In [1]:
import sounddevice as sd
import soundfile as sf
import threading
import time
import logging
import numpy as np
from statistics import mean, stdev
import os
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk
from datetime import datetime
import pandas as pd
import re
import pylsl  # LSL library for data streaming
import threading
import atexit
import sys
from pathlib import Path

# ======= CONFIGURATION VARIABLES =======
# Directory paths
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
AUDIO_OUTPUT_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "ExperimentAudio")
DESIGN_OUTPUT_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "ExperimentLog")
LSL_LOG_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "LSLLogs")

# LSL Stream Configuration
LSL_STREAM_NAME_CLICKS = "MouseClicks"
LSL_STREAM_NAME_MARKERS = "ExperimentMarkers"
LSL_STREAM_TYPE = "Markers"
LSL_SAMPLING_RATE = 0  # Irregular sampling rate
LSL_CHANNEL_FORMAT = pylsl.cf_string  # Using string format for markers

# Debug mode - set to True to print more detailed information
DEBUG_MODE = True

# Ensure directories exist
os.makedirs(AUDIO_OUTPUT_DIR, exist_ok=True)
os.makedirs(DESIGN_OUTPUT_DIR, exist_ok=True)
os.makedirs(LSL_LOG_DIR, exist_ok=True)

# Setup logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s',
                   handlers=[
                       logging.FileHandler(os.path.join(LSL_LOG_DIR, f"experiment_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")),
                       logging.StreamHandler()
                   ])

class PPSExperimentRunner:
    def __init__(self):
        # Initialize variables
        self.participant_id = None
        self.experiment_start_time = None
        self.stop_flag = False
        
        # Audio files and data
        self.looming_audio_file = None
        self.tactile_audio_file = None
        self.design_data = None
        
        # LSL outlets
        self.click_outlet = None
        self.marker_outlet = None
        
        # Recording
        self.recorder = None
        
        # Query audio devices
        self.devices = sd.query_devices()
        self.default_device = sd.default.device
        
        logging.info("PPS Experiment Runner initialized")
        
    def select_participant(self):
        """Display a window to select participant ID."""
        # Create root window
        root = tk.Tk()
        root.title("PPS Experiment - Select Participant")
        root.geometry("500x400")
        
        # Center the window
        root.update_idletasks()
        width = root.winfo_width()
        height = root.winfo_height()
        x = (root.winfo_screenwidth() // 2) - (width // 2)
        y = (root.winfo_screenheight() // 2) - (height // 2)
        root.geometry(f'+{x}+{y}')
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header label
        ttk.Label(main_frame, text="PPS Experiment Runner", font=("Arial", 16, "bold")).pack(pady=10)
        
        # Instructions
        ttk.Label(main_frame, text="Select a participant ID (0-99):").pack(pady=5)
        
        # Participant selection
        participant_frame = ttk.Frame(main_frame)
        participant_frame.pack(pady=10)
        
        # Check which participants have data
        available_participants = []
        for i in range(100):  # 0-99
            looming_file = os.path.join(AUDIO_OUTPUT_DIR, f"participant_{i}_design_looming.wav")
            tactile_file = os.path.join(AUDIO_OUTPUT_DIR, f"participant_{i}_design_tactile.wav")
            design_file = os.path.join(DESIGN_OUTPUT_DIR, f"participant_{i}_design.csv")
            
            if os.path.exists(looming_file) and os.path.exists(tactile_file) and os.path.exists(design_file):
                available_participants.append(i)
        
        # Participant listbox
        participant_listbox = tk.Listbox(participant_frame, height=10, width=10, font=("Arial", 12))
        participant_listbox.pack(side=tk.LEFT, fill=tk.Y)
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(participant_frame, orient=tk.VERTICAL, command=participant_listbox.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        participant_listbox.config(yscrollcommand=scrollbar.set)
        
        # Fill the listbox with available participants
        for p_id in available_participants:
            participant_listbox.insert(tk.END, str(p_id))
        
        # Information display
        info_frame = ttk.LabelFrame(main_frame, text="Participant Info", padding=10)
        info_frame.pack(fill=tk.X, padx=10, pady=10, expand=True)
        
        looming_label = ttk.Label(info_frame, text="Looming audio: Not selected")
        looming_label.pack(anchor=tk.W, pady=2)
        
        tactile_label = ttk.Label(info_frame, text="Tactile audio: Not selected")
        tactile_label.pack(anchor=tk.W, pady=2)
        
        design_label = ttk.Label(info_frame, text="Design file: Not selected")
        design_label.pack(anchor=tk.W, pady=2)
        
        status_label = ttk.Label(info_frame, text="Status: Select a participant", font=("Arial", 10, "italic"))
        status_label.pack(anchor=tk.W, pady=5)
        
        # Update info when participant is selected
        def on_participant_select(event):
            selected_indices = participant_listbox.curselection()
            if not selected_indices:
                return
                
            selected_id = participant_listbox.get(selected_indices[0])
            looming_file = os.path.join(AUDIO_OUTPUT_DIR, f"participant_{selected_id}_design_looming.wav")
            tactile_file = os.path.join(AUDIO_OUTPUT_DIR, f"participant_{selected_id}_design_tactile.wav")
            design_file = os.path.join(DESIGN_OUTPUT_DIR, f"participant_{selected_id}_design.csv")
            
            looming_label.config(text=f"Looming audio: {os.path.basename(looming_file)}")
            tactile_label.config(text=f"Tactile audio: {os.path.basename(tactile_file)}")
            design_label.config(text=f"Design file: {os.path.basename(design_file)}")
            
            # Check if files exist
            files_exist = all([os.path.exists(f) for f in [looming_file, tactile_file, design_file]])
            if files_exist:
                status_label.config(text=f"Status: Ready to run experiment for participant {selected_id}")
                start_button.config(state=tk.NORMAL)
            else:
                status_label.config(text=f"Status: Some files are missing for participant {selected_id}")
                start_button.config(state=tk.DISABLED)
        
        participant_listbox.bind('<<ListboxSelect>>', on_participant_select)
        
        # Function to handle Start button
        def on_start():
            selected_indices = participant_listbox.curselection()
            if not selected_indices:
                messagebox.showerror("Error", "Please select a participant")
                return
            
            self.participant_id = participant_listbox.get(selected_indices[0])
            
            # Load files
            looming_file = os.path.join(AUDIO_OUTPUT_DIR, f"participant_{self.participant_id}_design_looming.wav")
            tactile_file = os.path.join(AUDIO_OUTPUT_DIR, f"participant_{self.participant_id}_design_tactile.wav")
            design_file = os.path.join(DESIGN_OUTPUT_DIR, f"participant_{self.participant_id}_design.csv")
            
            # Check again if files exist
            if not all([os.path.exists(f) for f in [looming_file, tactile_file, design_file]]):
                messagebox.showerror("Error", f"Some files are missing for participant {self.participant_id}")
                return
            
            self.looming_audio_file = looming_file
            self.tactile_audio_file = tactile_file
            
            # Load design data
            try:
                self.design_data = pd.read_csv(design_file)
                logging.info(f"Loaded design data with {len(self.design_data)} trials for participant {self.participant_id}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load design file: {str(e)}")
                return
            
            # Close the window
            root.destroy()
            
            # Start the experiment
            self.run_experiment()
        
        # Function to handle Cancel button
        def on_cancel():
            root.destroy()
            sys.exit(0)
        
        # Buttons
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=20)
        
        start_button = ttk.Button(button_frame, text="Start Experiment", command=on_start, state=tk.DISABLED)
        start_button.pack(side=tk.LEFT, padx=10)
        
        ttk.Button(button_frame, text="Cancel", command=on_cancel).pack(side=tk.LEFT, padx=10)
        
        # Version info
        ttk.Label(
            main_frame, 
            text=f"PPS Experiment Runner v1.0 - {datetime.now().strftime('%Y-%m-%d')}", 
            font=("Arial", 8)
        ).pack(side=tk.BOTTOM, pady=5)
        
        # Start the GUI
        root.mainloop()
        
        # If participant_id is None, user cancelled
        if self.participant_id is None:
            sys.exit(0)
            
        return self.participant_id
    
    def setup_lsl_streams(self):
        """Set up the Lab Streaming Layer (LSL) outlets for markers and mouse clicks."""
        try:
            # Create LSL stream for experiment markers
            marker_info = pylsl.StreamInfo(
                name=f"{LSL_STREAM_NAME_MARKERS}_p{self.participant_id}",
                type=LSL_STREAM_TYPE,
                channel_count=2,  # [timestamp, marker_type]
                nominal_srate=LSL_SAMPLING_RATE,
                channel_format=LSL_CHANNEL_FORMAT,
                source_id=f"markers_p{self.participant_id}"
            )
            
            # Add metadata
            marker_desc = marker_info.desc()
            marker_desc.append_child_value("participant_id", self.participant_id)
            channels = marker_desc.append_child("channels")
            channels.append_child("channel").append_child_value("label", "timestamp")
            channels.append_child("channel").append_child_value("label", "marker_type")
            
            # Create outlet
            self.marker_outlet = pylsl.StreamOutlet(marker_info)
            logging.info(f"Created LSL outlet for experiment markers")
            
            # Create LSL stream for mouse clicks
            click_info = pylsl.StreamInfo(
                name=f"{LSL_STREAM_NAME_CLICKS}_p{self.participant_id}",
                type=LSL_STREAM_TYPE,
                channel_count=3,  # [timestamp, click_type, trial_info]
                nominal_srate=LSL_SAMPLING_RATE,
                channel_format=LSL_CHANNEL_FORMAT,
                source_id=f"clicks_p{self.participant_id}"
            )
            
            # Add metadata
            click_desc = click_info.desc()
            click_desc.append_child_value("participant_id", self.participant_id)
            channels = click_desc.append_child("channels")
            channels.append_child("channel").append_child_value("label", "timestamp")
            channels.append_child("channel").append_child_value("label", "click_type")
            channels.append_child("channel").append_child_value("label", "trial_info")
            
            # Create outlet
            self.click_outlet = pylsl.StreamOutlet(click_info)
            logging.info(f"Created LSL outlet for mouse clicks")
            
            return True
            
        except Exception as e:
            logging.error(f"Error setting up LSL streams: {e}")
            return False
    
    def setup_lsl_recorder(self):
        """Set up a recorder to save all LSL streams to a file."""
        try:
            # Create timestamp-based filename
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"participant_{self.participant_id}_lsl_data_{timestamp}.xdf"
            filepath = os.path.join(LSL_LOG_DIR, filename)
            
            # Start a separate thread to record LSL data
            def record_data():
                try:
                    logging.info(f"Starting LSL recording to {filepath}")
                    
                    # Record all streams
                    from pylsl import resolve_streams, StreamInlet, local_clock
                    
                    # Find all streams for this participant
                    streams = resolve_streams(wait_time=1.0)
                    participant_streams = [s for s in streams 
                                          if f"_p{self.participant_id}" in s.name()]
                    
                    if not participant_streams:
                        logging.warning("No LSL streams found for recording!")
                        return
                    
                    logging.info(f"Found {len(participant_streams)} streams to record")
                    
                    # Create inlets
                    inlets = [StreamInlet(stream) for stream in participant_streams]
                    
                    # Open file for recording
                    with open(filepath, 'w') as f:
                        f.write(f"# LSL Data for Participant {self.participant_id}\n")
                        f.write(f"# Recording started at: {datetime.now().isoformat()}\n")
                        f.write("# Stream,Timestamp,Data\n")
                        
                        # Record until stop_flag is set
                        while not self.stop_flag:
                            for i, inlet in enumerate(inlets):
                                sample, timestamp = inlet.pull_sample(timeout=0.0)
                                if sample:
                                    stream_name = participant_streams[i].name()
                                    data_str = ','.join([str(s) for s in sample])
                                    f.write(f"{stream_name},{timestamp},{data_str}\n")
                            
                            # Don't burn CPU
                            time.sleep(0.001)
                    
                    logging.info(f"LSL recording completed: {filepath}")
                    
                except Exception as e:
                    logging.error(f"Error in LSL recorder: {e}")
            
            # Start recording thread
            self.recording_thread = threading.Thread(target=record_data)
            self.recording_thread.daemon = True
            self.recording_thread.start()
            
            return True
            
        except Exception as e:
            logging.error(f"Error setting up LSL recorder: {e}")
            return False
    
    def setup_click_listener(self, window):
        """Set up a listener for mouse clicks to send to LSL."""
        try:
            # Dictionary to track trials by timestamp
            self.trial_timeline = {}
            
            # Build trial timeline from design data
            if self.design_data is not None:
                for _, row in self.design_data.iterrows():
                    # Convert timestamp to seconds
                    if 'timestamp_after_jitter' in row:
                        time_str = row['timestamp_after_jitter']
                        match = re.match(r'(\d+):(\d+\.\d+)', time_str)
                        if match:
                            minutes, seconds = match.groups()
                            time_sec = int(minutes) * 60 + float(seconds)
                            
                            # Add to timeline
                            self.trial_timeline[time_sec] = {
                                'trial_number': row['trial_number'],
                                'stimulus_type': row['stimulus_type'],
                                'soa_value_ms': row['soa_value_ms']
                            }
            
            logging.info(f"Built trial timeline with {len(self.trial_timeline)} entries")
            
            # Function to handle mouse clicks
            def on_click(event):
                # Ignore if we haven't started yet
                if self.experiment_start_time is None:
                    return
                
                # Get current time since experiment start
                current_time = time.perf_counter() - self.experiment_start_time
                
                # Determine click type
                if event.num == 1:
                    click_type = "left_click"
                elif event.num == 2:
                    click_type = "middle_click"
                elif event.num == 3:
                    click_type = "right_click"
                else:
                    click_type = f"button_{event.num}"
                
                # Find current trial
                current_trial = "none"
                for time_sec, trial_info in sorted(self.trial_timeline.items()):
                    if current_time >= time_sec:
                        current_trial = f"trial_{trial_info['trial_number']}"
                    else:
                        break
                
                # Send to LSL
                if self.click_outlet:
                    self.click_outlet.push_sample([
                        str(current_time),
                        click_type,
                        current_trial
                    ])
                    
                    if DEBUG_MODE:
                        logging.info(f"Mouse {click_type} at {current_time:.3f}s (Trial: {current_trial})")
            
            # Bind to window
            window.bind("<Button-1>", on_click)  # Left click
            window.bind("<Button-2>", on_click)  # Middle click
            window.bind("<Button-3>", on_click)  # Right click
            
            logging.info("Mouse click listener set up")
            return True
            
        except Exception as e:
            logging.error(f"Error setting up mouse click listener: {e}")
            return False
    
    def play_audio_files(self):
        """Play both audio files simultaneously using separate threads."""
        try:
            # Load audio files
            looming_data, looming_sr = sf.read(self.looming_audio_file, dtype='float32')
            tactile_data, tactile_sr = sf.read(self.tactile_audio_file, dtype='float32')
            
            logging.info(f"Loaded looming audio: {os.path.basename(self.looming_audio_file)}")
            logging.info(f"  Sample rate: {looming_sr} Hz, Duration: {len(looming_data)/looming_sr:.2f} seconds")
            
            logging.info(f"Loaded tactile audio: {os.path.basename(self.tactile_audio_file)}")
            logging.info(f"  Sample rate: {tactile_sr} Hz, Duration: {len(tactile_data)/tactile_sr:.2f} seconds")
            
            # Ensure we have matching sample rates
            if looming_sr != tactile_sr:
                logging.warning(f"Sample rates don't match! Looming: {looming_sr}Hz, Tactile: {tactile_sr}Hz")
                logging.warning("This might cause synchronization issues")
            
            # Create streams for both audio files
            looming_stream = sd.OutputStream(
                samplerate=looming_sr,
                channels=looming_data.shape[1] if len(looming_data.shape) > 1 else 1,
                dtype='float32',
                callback=None
            )
            
            tactile_stream = sd.OutputStream(
                samplerate=tactile_sr,
                channels=tactile_data.shape[1] if len(tactile_data.shape) > 1 else 1,
                dtype='float32',
                callback=None
            )
            
            # Set the experiment start time
            self.experiment_start_time = time.perf_counter()
            
            # Send start markers to LSL
            if self.marker_outlet:
                self.marker_outlet.push_sample(["0.000", "experiment_start"])
                logging.info("Sent LSL marker: experiment_start")
            
            # Start both streams
            looming_stream.start()
            tactile_stream.start()
            
            # Write data to the streams
            looming_stream.write(looming_data)
            tactile_stream.write(tactile_data)
            
            # Send end markers to LSL
            if self.marker_outlet:
                end_time = time.perf_counter() - self.experiment_start_time
                self.marker_outlet.push_sample([f"{end_time:.3f}", "experiment_end"])
                logging.info(f"Sent LSL marker: experiment_end at {end_time:.3f}s")
            
            # Close streams
            looming_stream.stop()
            tactile_stream.stop()
            looming_stream.close()
            tactile_stream.close()
            
            logging.info("Audio playback completed")
            return True
            
        except Exception as e:
            logging.error(f"Error playing audio files: {e}")
            return False
    
    def run_experiment(self):
        """Run the complete experiment with GUI."""
        try:
            # Create main experiment window
            root = tk.Tk()
            root.title(f"PPS Experiment - Participant {self.participant_id}")
            root.geometry("800x600")
            
            # Center the window
            root.update_idletasks()
            width = root.winfo_width()
            height = root.winfo_height()
            x = (root.winfo_screenwidth() // 2) - (width // 2)
            y = (root.winfo_screenheight() // 2) - (height // 2)
            root.geometry(f'+{x}+{y}')
            
            # Set up LSL streams
            if not self.setup_lsl_streams():
                messagebox.showerror("Error", "Failed to set up LSL streams")
                root.destroy()
                return False
            
            # Set up LSL recorder
            if not self.setup_lsl_recorder():
                messagebox.showerror("Error", "Failed to set up LSL recorder")
                root.destroy()
                return False
            
            # Set up click listener on the window
            if not self.setup_click_listener(root):
                messagebox.showerror("Error", "Failed to set up click listener")
                root.destroy()
                return False
            
            # Create main frame
            main_frame = ttk.Frame(root, padding="20")
            main_frame.pack(fill=tk.BOTH, expand=True)
            
            # Header
            ttk.Label(main_frame, text=f"PPS Experiment - Participant {self.participant_id}", 
                     font=("Arial", 18, "bold")).pack(pady=10)
            
            # Info frame
            info_frame = ttk.LabelFrame(main_frame, text="Experiment Information", padding=10)
            info_frame.pack(fill=tk.X, padx=10, pady=10)
            
            ttk.Label(info_frame, text=f"Looming Audio: {os.path.basename(self.looming_audio_file)}").pack(anchor=tk.W, pady=2)
            ttk.Label(info_frame, text=f"Tactile Audio: {os.path.basename(self.tactile_audio_file)}").pack(anchor=tk.W, pady=2)
            ttk.Label(info_frame, text=f"Trials: {len(self.design_data) if self.design_data is not None else 'Unknown'}").pack(anchor=tk.W, pady=2)
            
            # Instructions
            instructions_frame = ttk.LabelFrame(main_frame, text="Instructions", padding=10)
            instructions_frame.pack(fill=tk.X, padx=10, pady=10)
            
            ttk.Label(instructions_frame, 
                     text="1. Press Start to begin the experiment",
                     font=("Arial", 11)).pack(anchor=tk.W, pady=2)
            ttk.Label(instructions_frame, 
                     text="2. Click the mouse button when you detect a stimulus",
                     font=("Arial", 11)).pack(anchor=tk.W, pady=2)
            ttk.Label(instructions_frame, 
                     text="3. All responses will be recorded through LSL",
                     font=("Arial", 11)).pack(anchor=tk.W, pady=2)
            ttk.Label(instructions_frame, 
                     text="4. The experiment will end automatically when complete",
                     font=("Arial", 11)).pack(anchor=tk.W, pady=2)
            
            # Status frame
            status_frame = ttk.Frame(main_frame)
            status_frame.pack(fill=tk.X, padx=10, pady=20)
            
            status_var = tk.StringVar(value="Ready to start")
            status_label = ttk.Label(status_frame, textvariable=status_var, font=("Arial", 12, "bold"))
            status_label.pack(pady=10)
            
            # Progress bar
            progress_var = tk.DoubleVar(value=0.0)
            progress_bar = ttk.Progressbar(status_frame, variable=progress_var, maximum=100)
            progress_bar.pack(fill=tk.X, pady=5)
            
            # Function to start the experiment
            def start_experiment():
                nonlocal start_button
                
                # Disable the start button
                start_button.config(state=tk.DISABLED)
                
                # Update status
                status_var.set("Experiment running... Click to respond to stimuli")
                
                # Start progress updater
                def update_progress():
                    while not self.stop_flag:
                        if self.experiment_start_time is not None:
                            # Calculate progress based on audio duration (estimated at 15 minutes)
                            elapsed = time.perf_counter() - self.experiment_start_time
                            # Assume 15 minute duration
                            progress = min(100, (elapsed / (15 * 60)) * 100)
                            progress_var.set(progress)
                        
                        # Update every 100ms
                        time.sleep(0.1)
                
                progress_thread = threading.Thread(target=update_progress)
                progress_thread.daemon = True
                progress_thread.start()
                
                # Play audio files in a separate thread
                def run_audio():
                    try:
                        success = self.play_audio_files()
                        
                        # Update UI when complete
                        if success:
                            status_var.set("Experiment completed successfully")
                            messagebox.showinfo("Complete", 
                                             f"Experiment for participant {self.participant_id} has completed successfully.")
                        else:
                            status_var.set("Error during playback")
                        
                        # Set progress to 100%
                        progress_var.set(100)
                        
                        # Stop the experiment
                        self.stop_flag = True
                        
                        # Re-enable the start button for potential re-run
                        start_button.config(state=tk.NORMAL)
                        
                    except Exception as e:
                        logging.error(f"Error in audio thread: {e}")
                        status_var.set(f"Error: {str(e)}")
                        messagebox.showerror("Error", f"An error occurred during playback:\n\n{str(e)}")
                        self.stop_flag = True
                        start_button.config(state=tk.NORMAL)
                
                audio_thread = threading.Thread(target=run_audio)
                audio_thread.daemon = True
                audio_thread.start()
            
            # Buttons
            button_frame = ttk.Frame(main_frame)
            button_frame.pack(pady=20)
            
            start_button = ttk.Button(button_frame, text="Start Experiment", command=start_experiment)
            start_button.pack(side=tk.LEFT, padx=10)
            
            # Function to handle cancel
            def on_cancel():
                if messagebox.askyesno("Cancel Experiment", "Are you sure you want to cancel the experiment?"):
                    self.stop_flag = True
                    root.destroy()
            
            ttk.Button(button_frame, text="Cancel", command=on_cancel).pack(side=tk.LEFT, padx=10)
            
            # Set up protocol for window close
            def on_close():
                if self.experiment_start_time is not None and not self.stop_flag:
                    if not messagebox.askyesno("Exit", "Experiment is running. Are you sure you want to exit?"):
                        return
                
                self.stop_flag = True
                root.destroy()
            
            root.protocol("WM_DELETE_WINDOW", on_close)
            
            # Run the GUI
            root.mainloop()
            
            # Ensure we clean up
            self.stop_flag = True
            
            return True
            
        except Exception as e:
            logging.error(f"Error running experiment: {e}")
            messagebox.showerror("Error", f"An error occurred during the experiment:\n\n{str(e)}")
            return False
    
    def main(self):
        """Main entry point for the experiment runner."""
        try:
            # Select participant
            self.select_participant()
            
            # If we have a participant ID, run the experiment
            if self.participant_id is not None:
                self.run_experiment()
            
        except Exception as e:
            logging.error(f"Experiment failed: {e}")
            messagebox.showerror("Error", f"The experiment failed with an error:\n\n{str(e)}")
        
        finally:
            # Ensure we clean up
            self.stop_flag = True
            logging.info("Experiment runner completed")

# Entry point
if __name__ == "__main__":
    runner = PPSExperimentRunner()
    runner.main()

2025-03-19 16:22:49,569 - INFO - PPS Experiment Runner initialized
2025-03-19 16:23:06,118 - INFO - Loaded design data with 204 trials for participant 7
2025-03-19 16:23:06,236 - INFO - Created LSL outlet for experiment markers
2025-03-19 16:23:06,239 - INFO - Created LSL outlet for mouse clicks
2025-03-19 16:23:06,241 - INFO - Starting LSL recording to C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PPS_Experiment_Module\LSLLogs\participant_7_lsl_data_20250319_162306.xdf
2025-03-19 16:23:06,253 - INFO - Built trial timeline with 204 entries
2025-03-19 16:23:06,257 - INFO - Mouse click listener set up
2025-03-19 16:23:07,256 - INFO - Found 2 streams to record
2025-03-19 16:23:09,153 - INFO - Loaded looming audio: participant_7_design_looming.wav
2025-03-19 16:23:09,155 - INFO -   Sample rate: 48000 Hz, Duration: 1870.89 seconds
2025-03-19 16:23:09,155 - INFO - Loaded tactile audio: participant_7_design_tactile.wav
2025-03-19 16:23:09,156 - INFO -   Sample rate: 48000 Hz, Dura

In [2]:
import sounddevice as sd
import soundfile as sf
import threading
import time
import logging
import numpy as np
import os
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk
from datetime import datetime
import pandas as pd
import re
import pylsl  # LSL library for data streaming
import threading
import atexit
import sys
from pathlib import Path
import random
import queue

# ======= CONFIGURATION VARIABLES =======
# Directory paths
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
AUDIO_OUTPUT_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "ExperimentAudio")
DESIGN_OUTPUT_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "ExperimentLog")
LSL_LOG_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "LSLLogs")
PRACTICE_AUDIO_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "PracticeAudio")

# LSL Stream Configuration
LSL_STREAM_NAME_CLICKS = "MouseClicks"
LSL_STREAM_NAME_MARKERS = "ExperimentMarkers"
LSL_STREAM_TYPE = "Markers"
LSL_SAMPLING_RATE = 0  # Irregular sampling rate
LSL_CHANNEL_FORMAT = pylsl.cf_string  # Using string format for markers

# Practice mode configuration
PRACTICE_TRIALS = 10  # Number of practice trials
PRACTICE_TRIAL_DURATION = 6.0  # Duration of each practice trial in seconds
PRACTICE_RESPONSE_WINDOW = 1.0  # Response window after tactile stimulus (seconds)
PRACTICE_LOOMING_STIMULUS_FILE = os.path.join(BASE_DIR, "PPS_Experiment_Module", "front_az0_FABIAN_HRIR_natural.wav")
PRACTICE_TACTILE_STIMULUS_FILE = os.path.join(BASE_DIR, "PPS_Experiment_Module", "tactile_stimulus.wav")
PRACTICE_INSTRUCTION_FILE = os.path.join(BASE_DIR, "PPS_Experiment_Module", "practice_instructions.wav")

# Debug mode - set to True to print more detailed information
DEBUG_MODE = True

# Ensure directories exist
os.makedirs(AUDIO_OUTPUT_DIR, exist_ok=True)
os.makedirs(DESIGN_OUTPUT_DIR, exist_ok=True)
os.makedirs(LSL_LOG_DIR, exist_ok=True)
os.makedirs(PRACTICE_AUDIO_DIR, exist_ok=True)

# Setup logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s',
                   handlers=[
                       logging.FileHandler(os.path.join(LSL_LOG_DIR, f"experiment_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")),
                       logging.StreamHandler()
                   ])

class PPSExperimentRunner:
    def __init__(self):
        # Initialize variables
        self.participant_id = None
        self.participant_initials = None
        self.experiment_start_time = None
        self.stop_flag = False
        self.pause_flag = False
        self.practice_passed = False
        
        # Audio files and data
        self.looming_audio_file = None
        self.tactile_audio_file = None
        self.design_data = None
        
        # Audio streams
        self.looming_stream = None
        self.tactile_stream = None
        
        # LSL outlets
        self.click_outlet = None
        self.marker_outlet = None
        
        # Recording
        self.recorder = None
        self.lsl_recorder_confirmed = False
        
        # Practice trial data
        self.practice_responses = []
        self.practice_trials_info = []
        self.practice_trial_active = False
        self.practice_trial_type = None
        self.practice_tactile_time = None
        
        # Query audio devices
        self.devices = sd.query_devices()
        self.default_device = sd.default.device
        
        logging.info("PPS Experiment Runner initialized")
    
    def get_participant_info(self):
        """Get participant ID and name initials."""
        # Create root window
        root = tk.Tk()
        root.title("Participant Information")
        root.geometry("500x350")
        
        # Center the window
        root.update_idletasks()
        width = root.winfo_width()
        height = root.winfo_height()
        x = (root.winfo_screenwidth() // 2) - (width // 2)
        y = (root.winfo_screenheight() // 2) - (height // 2)
        root.geometry(f'+{x}+{y}')
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        ttk.Label(main_frame, text="Enter Participant Information", font=("Arial", 14, "bold")).pack(pady=10)
        
        # Participant ID
        id_frame = ttk.Frame(main_frame)
        id_frame.pack(fill=tk.X, pady=10)
        
        ttk.Label(id_frame, text="Participant ID (0-99):").pack(side=tk.LEFT, padx=5)
        id_var = tk.StringVar()
        id_entry = ttk.Entry(id_frame, textvariable=id_var, width=5)
        id_entry.pack(side=tk.LEFT, padx=5)
        
        # First name initial
        first_frame = ttk.Frame(main_frame)
        first_frame.pack(fill=tk.X, pady=10)
        
        ttk.Label(first_frame, text="First letter of first name:").pack(side=tk.LEFT, padx=5)
        first_var = tk.StringVar()
        first_entry = ttk.Entry(first_frame, textvariable=first_var, width=3)
        first_entry.pack(side=tk.LEFT, padx=5)
        
        # Last name initial
        last_frame = ttk.Frame(main_frame)
        last_frame.pack(fill=tk.X, pady=10)
        
        ttk.Label(last_frame, text="Last letter of last name:").pack(side=tk.LEFT, padx=5)
        last_var = tk.StringVar()
        last_entry = ttk.Entry(last_frame, textvariable=last_var, width=3)
        last_entry.pack(side=tk.LEFT, padx=5)
        
        # Preview
        preview_frame = ttk.LabelFrame(main_frame, text="Participant Identifier")
        preview_frame.pack(fill=tk.X, pady=15)
        
        preview_var = tk.StringVar(value="XX00")
        preview_label = ttk.Label(preview_frame, textvariable=preview_var, font=("Arial", 16, "bold"))
        preview_label.pack(pady=10)
        
        # Update preview when entries change
        def update_preview(*args):
            try:
                first = first_var.get().upper()
                last = last_var.get().upper()
                pid = id_var.get().zfill(2)
                
                if first and last and pid:
                    preview_var.set(f"{first[0]}{last[0]}{pid}")
            except:
                pass
        
        id_var.trace_add("write", update_preview)
        first_var.trace_add("write", update_preview)
        last_var.trace_add("write", update_preview)
        
        # Buttons
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=20)
        
        # Function to confirm entry
        result = [None, None]  # To store results
        
        def on_confirm():
            # Validate input
            pid = id_var.get().strip()
            first = first_var.get().strip().upper()
            last = last_var.get().strip().upper()
            
            if not pid or not pid.isdigit() or int(pid) < 0 or int(pid) > 99:
                messagebox.showerror("Error", "Participant ID must be a number between 0 and 99")
                return
            
            if not first or len(first) < 1:
                messagebox.showerror("Error", "Please enter the first letter of participant's first name")
                return
            
            if not last or len(last) < 1:
                messagebox.showerror("Error", "Please enter the last letter of participant's last name")
                return
            
            # Store results and close window
            result[0] = pid
            result[1] = f"{first[0]}{last[0]}{pid.zfill(2)}"
            root.destroy()
        
        def on_cancel():
            root.destroy()
        
        ttk.Button(button_frame, text="Confirm", command=on_confirm).pack(side=tk.LEFT, padx=10)
        ttk.Button(button_frame, text="Cancel", command=on_cancel).pack(side=tk.LEFT, padx=10)
        
        # Start the GUI
        root.mainloop()
        
        # Set participant info
        if result[0] is not None:
            self.participant_id = result[0]
            self.participant_initials = result[1]
            logging.info(f"Set participant ID: {self.participant_id}, Identifier: {self.participant_initials}")
            return True
        
        return False
    
    def confirm_lsl_recorder(self):
        """Confirm that the LSL recorder is running."""
        root = tk.Tk()
        root.title("LSL Recorder Confirmation")
        root.geometry("500x300")
        
        # Center the window
        root.update_idletasks()
        width = root.winfo_width()
        height = root.winfo_height()
        x = (root.winfo_screenwidth() // 2) - (width // 2)
        y = (root.winfo_screenheight() // 2) - (height // 2)
        root.geometry(f'+{x}+{y}')
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        ttk.Label(main_frame, text="Important!", font=("Arial", 16, "bold")).pack(pady=5)
        
        # Warning message
        msg = ("Please confirm that the LSL recorder is running!\n\n"
              "This is CRITICAL - without the LSL recorder, no data will be saved.\n\n"
              "Have you started the LSL recorder?")
        
        ttk.Label(main_frame, text=msg, wraplength=400, justify="center").pack(pady=20)
        
        # Checkbox
        var_confirmed = tk.BooleanVar(value=False)
        ttk.Checkbutton(main_frame, text="Yes, I have started the LSL recorder", 
                       variable=var_confirmed).pack(pady=10)
        
        # Buttons
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=20)
        
        # Function to confirm
        def on_confirm():
            if not var_confirmed.get():
                messagebox.showerror("Error", "You must confirm the LSL recorder is running.")
                return
            
            self.lsl_recorder_confirmed = True
            root.destroy()
        
        def on_cancel():
            root.destroy()
        
        ttk.Button(button_frame, text="Confirm", command=on_confirm).pack(side=tk.LEFT, padx=10)
        ttk.Button(button_frame, text="Cancel", command=on_cancel).pack(side=tk.LEFT, padx=10)
        
        # Start the GUI
        root.mainloop()
        
        return self.lsl_recorder_confirmed
        
    def find_participant_files(self):
        """Find the participant's audio and design files based on the participant ID."""
        # Look for design file
        design_file_pattern = f"participant_{self.participant_id}_design.csv"
        design_file_path = os.path.join(DESIGN_OUTPUT_DIR, design_file_pattern)
        
        if not os.path.exists(design_file_path):
            # Try to find files that match the pattern
            design_files = [f for f in os.listdir(DESIGN_OUTPUT_DIR) 
                          if f.startswith(f"participant_{self.participant_id}_") and f.endswith("_design.csv")]
            
            if not design_files:
                raise FileNotFoundError(f"No design file found for participant {self.participant_id}")
            
            design_file_path = os.path.join(DESIGN_OUTPUT_DIR, design_files[0])
        
        self.design_file_path = design_file_path
        logging.info(f"Found design file: {self.design_file_path}")
        
        # Load design data
        self.design_data = pd.read_csv(self.design_file_path)
        logging.info(f"Loaded design data with {len(self.design_data)} trials")
        
        # Look for looming audio file
        looming_file_pattern = f"participant_{self.participant_id}_design_looming.wav"
        looming_file_path = os.path.join(AUDIO_OUTPUT_DIR, looming_file_pattern)
        
        if not os.path.exists(looming_file_path):
            # Try to find files that match by participant ID
            looming_files = [f for f in os.listdir(AUDIO_OUTPUT_DIR) 
                           if f.startswith(f"participant_{self.participant_id}_") and "looming" in f.lower() and f.endswith(".wav")]
            
            if not looming_files:
                raise FileNotFoundError(f"No looming audio file found for participant {self.participant_id}")
            
            looming_file_path = os.path.join(AUDIO_OUTPUT_DIR, looming_files[0])
        
        self.looming_audio_file = looming_file_path
        logging.info(f"Found looming audio file: {self.looming_audio_file}")
        
        # Look for tactile audio file
        tactile_file_pattern = f"participant_{self.participant_id}_design_tactile.wav"
        tactile_file_path = os.path.join(AUDIO_OUTPUT_DIR, tactile_file_pattern)
        
        if not os.path.exists(tactile_file_path):
            # Try to find files that match by participant ID
            tactile_files = [f for f in os.listdir(AUDIO_OUTPUT_DIR) 
                           if f.startswith(f"participant_{self.participant_id}_") and "tactile" in f.lower() and f.endswith(".wav")]
            
            if not tactile_files:
                raise FileNotFoundError(f"No tactile audio file found for participant {self.participant_id}")
            
            tactile_file_path = os.path.join(AUDIO_OUTPUT_DIR, tactile_files[0])
        
        self.tactile_audio_file = tactile_file_path
        logging.info(f"Found tactile audio file: {self.tactile_audio_file}")
        
        return True
    
    def setup_lsl_streams(self):
        """Set up the Lab Streaming Layer (LSL) outlets for markers and mouse clicks."""
        try:
            # Create LSL stream for experiment markers
            marker_info = pylsl.StreamInfo(
                name=f"{LSL_STREAM_NAME_MARKERS}_{self.participant_initials}",
                type=LSL_STREAM_TYPE,
                channel_count=3,  # [timestamp, marker_type, participant_id]
                nominal_srate=LSL_SAMPLING_RATE,
                channel_format=LSL_CHANNEL_FORMAT,
                source_id=f"markers_{self.participant_initials}"
            )
            
            # Add metadata
            marker_desc = marker_info.desc()
            marker_desc.append_child_value("participant_id", self.participant_id)
            marker_desc.append_child_value("participant_initials", self.participant_initials)
            channels = marker_desc.append_child("channels")
            channels.append_child("channel").append_child_value("label", "timestamp")
            channels.append_child("channel").append_child_value("label", "marker_type")
            channels.append_child("channel").append_child_value("label", "participant_id")
            
            # Create outlet
            self.marker_outlet = pylsl.StreamOutlet(marker_info)
            logging.info(f"Created LSL outlet for experiment markers: {LSL_STREAM_NAME_MARKERS}_{self.participant_initials}")
            
            # Create LSL stream for mouse clicks
            click_info = pylsl.StreamInfo(
                name=f"{LSL_STREAM_NAME_CLICKS}_{self.participant_initials}",
                type=LSL_STREAM_TYPE,
                channel_count=4,  # [timestamp, click_type, trial_info, participant_id]
                nominal_srate=LSL_SAMPLING_RATE,
                channel_format=LSL_CHANNEL_FORMAT,
                source_id=f"clicks_{self.participant_initials}"
            )
            
            # Add metadata
            click_desc = click_info.desc()
            click_desc.append_child_value("participant_id", self.participant_id)
            click_desc.append_child_value("participant_initials", self.participant_initials)
            channels = click_desc.append_child("channels")
            channels.append_child("channel").append_child_value("label", "timestamp")
            channels.append_child("channel").append_child_value("label", "click_type")
            channels.append_child("channel").append_child_value("label", "trial_info")
            channels.append_child("channel").append_child_value("label", "participant_id")
            
            # Create outlet
            self.click_outlet = pylsl.StreamOutlet(click_info)
            logging.info(f"Created LSL outlet for mouse clicks: {LSL_STREAM_NAME_CLICKS}_{self.participant_initials}")
            
            return True
            
        except Exception as e:
            logging.error(f"Error setting up LSL streams: {e}")
            return False
    
    def setup_lsl_recorder(self):
        """Set up a recorder to save all LSL streams to a file."""
        try:
            # Create timestamp-based filename
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{self.participant_initials}_lsl_data_{timestamp}.xdf"
            filepath = os.path.join(LSL_LOG_DIR, filename)
            
            # Start a separate thread to record LSL data
            def record_data():
                try:
                    logging.info(f"Starting LSL recording to {filepath}")
                    
                    # Record all streams
                    from pylsl import resolve_streams, StreamInlet, local_clock
                    
                    # Find all streams for this participant
                    streams = resolve_streams(wait_time=1.0)
                    participant_streams = [s for s in streams 
                                          if f"_{self.participant_initials}" in s.name()]
                    
                    if not participant_streams:
                        logging.warning("No LSL streams found for recording!")
                        return
                    
                    logging.info(f"Found {len(participant_streams)} streams to record")
                    
                    # Create inlets
                    inlets = [StreamInlet(stream) for stream in participant_streams]
                    
                    # Open file for recording
                    with open(filepath, 'w') as f:
                        f.write(f"# LSL Data for Participant {self.participant_initials}\n")
                        f.write(f"# Recording started at: {datetime.now().isoformat()}\n")
                        f.write("# Stream,Timestamp,Data\n")
                        
                        # Record until stop_flag is set
                        while not self.stop_flag:
                            # Skip recording while paused
                            if self.pause_flag:
                                time.sleep(0.1)
                                continue
                                
                            for i, inlet in enumerate(inlets):
                                sample, timestamp = inlet.pull_sample(timeout=0.0)
                                if sample:
                                    stream_name = participant_streams[i].name()
                                    data_str = ','.join([str(s) for s in sample])
                                    f.write(f"{stream_name},{timestamp},{data_str}\n")
                            
                            # Don't burn CPU
                            time.sleep(0.001)
                    
                    logging.info(f"LSL recording completed: {filepath}")
                    
                except Exception as e:
                    logging.error(f"Error in LSL recorder: {e}")
            
            # Start recording thread
            self.recording_thread = threading.Thread(target=record_data)
            self.recording_thread.daemon = True
            self.recording_thread.start()
            
            return True
            
        except Exception as e:
            logging.error(f"Error setting up LSL recorder: {e}")
            return False
    
    def setup_click_listener(self, window):
        """Set up a listener for mouse clicks to send to LSL."""
        try:
            # Dictionary to track trials by timestamp
            self.trial_timeline = {}
            
            # Build trial timeline from design data
            if self.design_data is not None:
                for _, row in self.design_data.iterrows():
                    # Convert timestamp to seconds
                    if 'timestamp_after_jitter' in row:
                        time_str = row['timestamp_after_jitter']
                        match = re.match(r'(\d+):(\d+\.\d+)', time_str)
                        if match:
                            minutes, seconds = match.groups()
                            time_sec = int(minutes) * 60 + float(seconds)
                            
                            # Add to timeline
                            self.trial_timeline[time_sec] = {
                                'trial_number': row['trial_number'],
                                'stimulus_type': row['stimulus_type'],
                                'soa_value_ms': row['soa_value_ms'] if 'soa_value_ms' in row else 0
                            }
            
            logging.info(f"Built trial timeline with {len(self.trial_timeline)} entries")
            
            # Function to handle mouse clicks
            def on_click(event):
                # Ignore if we haven't started yet
                if self.experiment_start_time is None:
                    return
                
                # Ignore if experiment is paused
                if self.pause_flag:
                    return
                
                # Get current time since experiment start
                current_time = time.perf_counter() - self.experiment_start_time
                
                # Determine click type
                if event.num == 1:
                    click_type = "left_click"
                elif event.num == 2:
                    click_type = "middle_click"
                elif event.num == 3:
                    click_type = "right_click"
                else:
                    click_type = f"button_{event.num}"
                
                # If in practice mode, handle practice response
                if self.practice_trial_active:
                    if DEBUG_MODE:
                        logging.info(f"Practice click: {click_type} at {current_time:.3f}s")
                    
                    # Record the response for practice analysis
                    self.practice_responses.append({
                        'time': current_time,
                        'click_type': click_type,
                        'trial_type': self.practice_trial_type,
                        'tactile_time': self.practice_tactile_time
                    })
                    
                    # Send to LSL as well
                    if self.click_outlet:
                        self.click_outlet.push_sample([
                            str(current_time),
                            click_type,
                            f"practice_{self.practice_trial_type}",
                            self.participant_initials
                        ])
                    
                    return
                
                # Find current trial for regular experiment
                current_trial = "none"
                for time_sec, trial_info in sorted(self.trial_timeline.items()):
                    if current_time >= time_sec:
                        current_trial = f"trial_{trial_info['trial_number']}"
                    else:
                        break
                
                # Send to LSL
                if self.click_outlet:
                    self.click_outlet.push_sample([
                        str(current_time),
                        click_type,
                        current_trial,
                        self.participant_initials
                    ])
                    
                    if DEBUG_MODE:
                        logging.info(f"Mouse {click_type} at {current_time:.3f}s (Trial: {current_trial})")
            
            # Bind to window
            window.bind("<Button-1>", on_click)  # Left click
            window.bind("<Button-2>", on_click)  # Middle click
            window.bind("<Button-3>", on_click)  # Right click
            
            logging.info("Mouse click listener set up")
            return True
            
        except Exception as e:
            logging.error(f"Error setting up mouse click listener: {e}")
            return False
    
    def play_audio_files(self):
        """Play both audio files simultaneously, supporting pause/resume."""
        try:
            # Load audio files
            looming_data, looming_sr = sf.read(self.looming_audio_file, dtype='float32')
            tactile_data, tactile_sr = sf.read(self.tactile_audio_file, dtype='float32')
            
            logging.info(f"Loaded looming audio: {os.path.basename(self.looming_audio_file)}")
            logging.info(f"  Sample rate: {looming_sr} Hz, Duration: {len(looming_data)/looming_sr:.2f} seconds")
            
            logging.info(f"Loaded tactile audio: {os.path.basename(self.tactile_audio_file)}")
            logging.info(f"  Sample rate: {tactile_sr} Hz, Duration: {len(tactile_data)/tactile_sr:.2f} seconds")
            
            # Ensure we have matching sample rates
            if looming_sr != tactile_sr:
                logging.warning(f"Sample rates don't match! Looming: {looming_sr}Hz, Tactile: {tactile_sr}Hz")
                logging.warning("This might cause synchronization issues")
            
            # Store the data for pause/resume
            self.looming_data = looming_data
            self.looming_sr = looming_sr
            self.tactile_data = tactile_data
            self.tactile_sr = tactile_sr
            
            # Set up callback functions for streaming
            looming_pos = 0
            tactile_pos = 0
            
            # Looming callback
            def looming_callback(outdata, frames, time, status):
                nonlocal looming_pos
                if status:
                    logging.warning(f"Looming audio status: {status}")
                
                # Check if paused
                if self.pause_flag:
                    # Fill with silence
                    outdata.fill(0)
                    return
                
                if self.stop_flag:
                    raise sd.CallbackStop
                
                # Check if we've reached the end
                if looming_pos >= len(self.looming_data):
                    raise sd.CallbackStop
                
                # Copy data to output
                if len(self.looming_data.shape) == 1:  # Mono
                    remain = min(frames, len(self.looming_data) - looming_pos)
                    outdata[:remain, 0] = self.looming_data[looming_pos:looming_pos+remain]
                    if remain < frames:
                        outdata[remain:, 0] = 0
                else:  # Stereo
                    remain = min(frames, len(self.looming_data) - looming_pos)
                    outdata[:remain] = self.looming_data[looming_pos:looming_pos+remain]
                    if remain < frames:
                        outdata[remain:] = 0
                
                looming_pos += frames
            
            # Tactile callback
            def tactile_callback(outdata, frames, time, status):
                nonlocal tactile_pos
                if status:
                    logging.warning(f"Tactile audio status: {status}")
                
                # Check if paused
                if self.pause_flag:
                    # Fill with silence
                    outdata.fill(0)
                    return
                
                if self.stop_flag:
                    raise sd.CallbackStop
                
                # Check if we've reached the end
                if tactile_pos >= len(self.tactile_data):
                    raise sd.CallbackStop
                
                # Copy data to output
                if len(self.tactile_data.shape) == 1:  # Mono
                    remain = min(frames, len(self.tactile_data) - tactile_pos)
                    outdata[:remain, 0] = self.tactile_data[tactile_pos:tactile_pos+remain]
                    if remain < frames:
                        outdata[remain:, 0] = 0
                else:  # Stereo
                    remain = min(frames, len(self.tactile_data) - tactile_pos)
                    outdata[:remain] = self.tactile_data[tactile_pos:tactile_pos+remain]
                    if remain < frames:
                        outdata[remain:] = 0
                
                tactile_pos += frames
            
            # Create streams for both audio files
            looming_channels = self.looming_data.shape[1] if len(self.looming_data.shape) > 1 else 1
            tactile_channels = self.tactile_data.shape[1] if len(self.tactile_data.shape) > 1 else 1
            
            looming_stream = sd.OutputStream(
                samplerate=looming_sr,
                channels=looming_channels,
                callback=looming_callback,
                finished_callback=lambda: logging.info("Looming audio playback completed"),
                dtype='float32'
            )
            
            tactile_stream = sd.OutputStream(
                samplerate=tactile_sr,
                channels=tactile_channels,
                callback=tactile_callback,
                finished_callback=lambda: logging.info("Tactile audio playback completed"),
                dtype='float32'
            )
            
            # Store streams for pause/resume
            self.looming_stream = looming_stream
            self.tactile_stream = tactile_stream
            
            # Set the experiment start time
            self.experiment_start_time = time.perf_counter()
            
            # Send start markers to LSL
            if self.marker_outlet:
                self.marker_outlet.push_sample(["0.000", "experiment_start", self.participant_initials])
                logging.info("Sent LSL marker: experiment_start")
            
            # Start both streams
            looming_stream.start()
            tactile_stream.start()
            
            # Wait for both streams to finish or stop_flag
            while looming_stream.active and tactile_stream.active and not self.stop_flag:
                time.sleep(0.1)
            
            # Send end markers to LSL
            if self.marker_outlet and not self.pause_flag:
                end_time = time.perf_counter() - self.experiment_start_time
                self.marker_outlet.push_sample([f"{end_time:.3f}", "experiment_end", self.participant_initials])
                logging.info(f"Sent LSL marker: experiment_end at {end_time:.3f}s")
            
            # Stop streams if still active (this handles early termination)
            if looming_stream.active:
                looming_stream.stop()
            
            if tactile_stream.active:
                tactile_stream.stop()
            
            # Close streams
            looming_stream.close()
            tactile_stream.close()
            
            logging.info("Audio playback completed")
            return True
            
        except Exception as e:
            logging.error(f"Error playing audio files: {e}")
            
            # Clean up streams if they were created
            try:
                if self.looming_stream and self.looming_stream.active:
                    self.looming_stream.stop()
                    self.looming_stream.close()
                
                if self.tactile_stream and self.tactile_stream.active:
                    self.tactile_stream.stop()
                    self.tactile_stream.close()
            except:
                pass
                
            return False
    
    def pause_experiment(self):
        """Pause the experiment."""
        if self.pause_flag:
            return  # Already paused
        
        # Set pause flag
        self.pause_flag = True
        
        # Send pause marker to LSL
        if self.marker_outlet and self.experiment_start_time is not None:
            pause_time = time.perf_counter() - self.experiment_start_time
            self.marker_outlet.push_sample([f"{pause_time:.3f}", "experiment_pause", self.participant_initials])
            logging.info(f"Sent LSL marker: experiment_pause at {pause_time:.3f}s")
        
        logging.info("Experiment paused")
    
    def resume_experiment(self):
        """Resume the experiment after pause."""
        if not self.pause_flag:
            return  # Already running
        
        # Clear pause flag
        self.pause_flag = False
        
        # Send resume marker to LSL
        if self.marker_outlet and self.experiment_start_time is not None:
            resume_time = time.perf_counter() - self.experiment_start_time
            self.marker_outlet.push_sample([f"{resume_time:.3f}", "experiment_resume", self.participant_initials])
            logging.info(f"Sent LSL marker: experiment_resume at {resume_time:.3f}s")
        
        logging.info("Experiment resumed")
    
    def stop_experiment(self):
        """Stop the experiment and clean up resources."""
        # Set stop flag
        self.stop_flag = True
        
        # Send stop marker to LSL
        if self.marker_outlet and self.experiment_start_time is not None:
            stop_time = time.perf_counter() - self.experiment_start_time
            self.marker_outlet.push_sample([f"{stop_time:.3f}", "experiment_stop", self.participant_initials])
            logging.info(f"Sent LSL marker: experiment_stop at {stop_time:.3f}s")
        
        # Stop audio streams if active
        try:
            if self.looming_stream and self.looming_stream.active:
                self.looming_stream.stop()
                self.looming_stream.close()
            
            if self.tactile_stream and self.tactile_stream.active:
                self.tactile_stream.stop()
                self.tactile_stream.close()
        except:
            pass
        
        logging.info("Experiment stopped")
    
    def run_practice_trials(self, parent_window=None):
        """Run practice trials to ensure participant understands the task."""
        try:
            # Create window for practice trials
            if parent_window:
                parent_window.withdraw()  # Hide parent window
            
            root = tk.Tk()
            root.title(f"Practice Trials - Participant {self.participant_initials}")
            root.geometry("800x600")
            
            # Center the window
            root.update_idletasks()
            width = root.winfo_width()
            height = root.winfo_height()
            x = (root.winfo_screenwidth() // 2) - (width // 2)
            y = (root.winfo_screenheight() // 2) - (height // 2)
            root.geometry(f'+{x}+{y}')
            
            # Set up click listener
            self.setup_click_listener(root)
            
            # Create main frame
            main_frame = ttk.Frame(root, padding="20")
            main_frame.pack(fill=tk.BOTH, expand=True)
            
            # Header
            ttk.Label(main_frame, text=f"Practice Trials - {self.participant_initials}", 
                     font=("Arial", 18, "bold")).pack(pady=10)
            
            # Instructions
            instructions_frame = ttk.LabelFrame(main_frame, text="Instructions", padding=10)
            instructions_frame.pack(fill=tk.X, padx=10, pady=10)
            
            instruction_text = (
                "The participant will hear 10 practice trials.\n\n"
                "They should click ONLY when they feel a tactile stimulus.\n"
                "Some trials will have only looming sounds, some only tactile stimuli,\n"
                "and some will have both with various timings.\n\n"
                "The participant must:\n"
                "1. Click ONLY when they feel tactile stimulation\n"
                "2. Respond within 1 second of the tactile stimulus\n"
                "3. NOT click when there is no tactile stimulus\n\n"
                "The experiment will only proceed if the practice is completed successfully."
            )
            
            ttk.Label(instructions_frame, text=instruction_text, 
                     font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
            
            # Status frame
            status_frame = ttk.Frame(main_frame)
            status_frame.pack(fill=tk.X, padx=10, pady=10)
            
            status_var = tk.StringVar(value="Ready to start practice")
            status_label = ttk.Label(status_frame, textvariable=status_var, 
                                    font=("Arial", 12, "bold"))
            status_label.pack(pady=5)
            
            # Progress frame
            progress_frame = ttk.Frame(main_frame)
            progress_frame.pack(fill=tk.X, padx=10, pady=5)
            
            progress_var = tk.StringVar(value="Trial: 0 / 10")
            ttk.Label(progress_frame, textvariable=progress_var).pack(side=tk.LEFT, pady=5)
            
            progress_bar = ttk.Progressbar(progress_frame, maximum=PRACTICE_TRIALS)
            progress_bar.pack(fill=tk.X, side=tk.RIGHT, expand=True, padx=10, pady=5)
            
            # Results frame (initially hidden)
            results_frame = ttk.LabelFrame(main_frame, text="Results", padding=10)
            results_var = tk.StringVar(value="")
            results_label = ttk.Label(results_frame, textvariable=results_var, 
                                     font=("Arial", 11), justify="left")
            results_label.pack(anchor=tk.W, pady=5)
            
            # Button frame
            button_frame = ttk.Frame(main_frame)
            button_frame.pack(pady=20)
            
            # Start practice function
            def run_practice():
                # Reset practice data
                self.practice_responses = []
                self.practice_trials_info = []
                self.practice_trial_active = False
                
                nonlocal start_button
                
                # Disable buttons
                start_button.config(state=tk.DISABLED)
                cancel_button.config(state=tk.DISABLED)
                
                # Hide results if shown previously
                results_frame.pack_forget()
                
                # Update status
                status_var.set("Playing instructions...")
                root.update()
                
                # Send marker
                if self.marker_outlet:
                    self.marker_outlet.push_sample(["0.000", "practice_start", self.participant_initials])
                    logging.info("Sent LSL marker: practice_start")
                
                # Set experiment start time for click timing
                self.experiment_start_time = time.perf_counter()
                
                # TODO: Play instruction audio file here if it exists
                if os.path.exists(PRACTICE_INSTRUCTION_FILE):
                    try:
                        instruction_data, instruction_sr = sf.read(PRACTICE_INSTRUCTION_FILE, dtype='float32')
                        sd.play(instruction_data, instruction_sr)
                        sd.wait()  # Wait for instructions to finish
                    except Exception as e:
                        logging.error(f"Error playing instruction audio: {e}")
                
                # Prepare practice trials with randomized types
                trial_types = []
                # 1/3 looming only
                trial_types.extend(["looming_only"] * (PRACTICE_TRIALS // 3))
                # 1/3 tactile only
                trial_types.extend(["tactile_only"] * (PRACTICE_TRIALS // 3))
                # 1/3 both with random SOA
                trial_types.extend(["both"] * (PRACTICE_TRIALS - len(trial_types)))
                
                # Shuffle trial types
                random.shuffle(trial_types)
                
                # SOA values for "both" trials (milliseconds)
                soa_values = [190, 400, 700, 1000]
                
                # Run each practice trial
                for i, trial_type in enumerate(trial_types):
                    # Update progress
                    progress_var.set(f"Trial: {i+1} / {PRACTICE_TRIALS}")
                    progress_bar.config(value=i+1)
                    status_var.set(f"Running practice trial {i+1}...")
                    root.update()
                    
                    # Mark current trial as active
                    self.practice_trial_active = True
                    self.practice_trial_type = trial_type
                    self.practice_tactile_time = None
                    
                    # Send trial start marker
                    if self.marker_outlet:
                        trial_time = time.perf_counter() - self.experiment_start_time
                        self.marker_outlet.push_sample([f"{trial_time:.3f}", 
                                                     f"practice_trial_{i+1}_start_{trial_type}", 
                                                     self.participant_initials])
                    
                    # Load sounds based on trial type
                    if trial_type == "looming_only" or trial_type == "both":
                        try:
                            looming_data, looming_sr = sf.read(PRACTICE_LOOMING_STIMULUS_FILE, dtype='float32')
                        except Exception as e:
                            logging.error(f"Error loading looming stimulus: {e}")
                            looming_data = None
                    else:
                        looming_data = None
                    
                    if trial_type == "tactile_only" or trial_type == "both":
                        try:
                            tactile_data, tactile_sr = sf.read(PRACTICE_TACTILE_STIMULUS_FILE, dtype='float32')
                        except Exception as e:
                            logging.error(f"Error loading tactile stimulus: {e}")
                            tactile_data = None
                    else:
                        tactile_data = None
                    
                    # Set up trial timing
                    trial_start_time = time.perf_counter()
                    
                    # Calculate trial timings (in seconds from trial start)
                    looming_onset = 1.0  # Looming stimulus at 1 second
                    
                    # For "both" trials, use a random SOA
                    if trial_type == "both":
                        soa = random.choice(soa_values) / 1000.0  # Convert to seconds
                        tactile_onset = looming_onset + soa
                    else:
                        soa = 0
                        tactile_onset = 1.5  # Tactile only stimulus at 1.5 seconds
                    
                    # Record trial info
                    self.practice_trials_info.append({
                        'trial_number': i + 1,
                        'trial_type': trial_type,
                        'soa_ms': soa * 1000 if trial_type == "both" else 0
                    })
                    
                    # Wait for looming onset time
                    if looming_data is not None:
                        time_to_wait = looming_onset - (time.perf_counter() - trial_start_time)
                        if time_to_wait > 0:
                            time.sleep(time_to_wait)
                        
                        # Play looming sound
                        sd.play(looming_data, looming_sr, blocking=False)
                        
                        # Send looming marker
                        looming_time = time.perf_counter() - self.experiment_start_time
                        if self.marker_outlet:
                            self.marker_outlet.push_sample([f"{looming_time:.3f}", 
                                                         f"practice_trial_{i+1}_looming", 
                                                         self.participant_initials])
                    
                    # Wait for tactile onset time
                    if tactile_data is not None:
                        time_to_wait = tactile_onset - (time.perf_counter() - trial_start_time)
                        if time_to_wait > 0:
                            time.sleep(time_to_wait)
                        
                        # Play tactile sound
                        sd.play(tactile_data, tactile_sr, blocking=False)
                        
                        # Record tactile time for response window check
                        self.practice_tactile_time = time.perf_counter() - self.experiment_start_time
                        
                        # Send tactile marker
                        if self.marker_outlet:
                            self.marker_outlet.push_sample([f"{self.practice_tactile_time:.3f}", 
                                                         f"practice_trial_{i+1}_tactile", 
                                                         self.participant_initials])
                    
                    # Wait for trial to complete
                    trial_duration = max(PRACTICE_TRIAL_DURATION, 
                                        tactile_onset + 2.0 if tactile_data else 0,
                                        looming_onset + 2.0 if looming_data else 0)
                    
                    time_elapsed = time.perf_counter() - trial_start_time
                    if trial_duration > time_elapsed:
                        time.sleep(trial_duration - time_elapsed)
                    
                    # Mark trial as complete
                    self.practice_trial_active = False
                    
                    # Send trial end marker
                    if self.marker_outlet:
                        trial_end_time = time.perf_counter() - self.experiment_start_time
                        self.marker_outlet.push_sample([f"{trial_end_time:.3f}", 
                                                     f"practice_trial_{i+1}_end", 
                                                     self.participant_initials])
                    
                    # Brief pause between trials
                    time.sleep(1.0)
                
                # Analysis of practice results
                status_var.set("Analyzing practice performance...")
                root.update()
                
                # Calculate performance
                correct_responses = 0
                false_alarms = 0
                misses = 0
                
                for i, trial in enumerate(self.practice_trials_info):
                    trial_num = trial['trial_number']
                    trial_type = trial['trial_type']
                    
                    # Get responses for this trial
                    trial_responses = [r for r in self.practice_responses 
                                     if r['trial_type'] == trial_type and 
                                     abs(r['time'] - self.practice_tactile_time) < PRACTICE_TRIAL_DURATION 
                                     if self.practice_tactile_time is not None]
                    
                    if trial_type in ["tactile_only", "both"]:
                        # Should have a response within PRACTICE_RESPONSE_WINDOW
                        valid_responses = [r for r in trial_responses 
                                        if r['tactile_time'] is not None and 
                                        0 < r['time'] - r['tactile_time'] < PRACTICE_RESPONSE_WINDOW]
                        
                        if valid_responses:
                            correct_responses += 1
                        else:
                            misses += 1
                    
                    elif trial_type == "looming_only":
                        # Should NOT have a response
                        if trial_responses:
                            false_alarms += 1
                        else:
                            correct_responses += 1
                
                # Calculate accuracy
                total_trials = len(self.practice_trials_info)
                accuracy = (correct_responses / total_trials) * 100 if total_trials > 0 else 0
                
                # Determine if practice passed
                self.practice_passed = accuracy >= 80 and false_alarms <= 2 and misses <= 2
                
                # Display results
                results_text = (
                    f"Practice Results:\n"
                    f"Trials completed: {total_trials}\n"
                    f"Correct responses: {correct_responses} ({accuracy:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Misses: {misses}\n\n"
                )
                
                if self.practice_passed:
                    results_text += "PASSED: Participant understood the task!\n"
                    results_text += "You may now proceed to the main experiment."
                    status_var.set("Practice completed successfully!")
                else:
                    results_text += "FAILED: Participant did not meet criteria.\n"
                    results_text += "Please review the instructions and try again."
                    status_var.set("Practice failed - try again")
                
                results_var.set(results_text)
                results_frame.pack(fill=tk.X, padx=10, pady=10, before=button_frame)
                
                # Send practice end marker
                if self.marker_outlet:
                    practice_end_time = time.perf_counter() - self.experiment_start_time
                    result = "pass" if self.practice_passed else "fail"
                    self.marker_outlet.push_sample([f"{practice_end_time:.3f}", 
                                                 f"practice_end_{result}_{correct_responses}_{false_alarms}_{misses}", 
                                                 self.participant_initials])
                    logging.info(f"Sent LSL marker: practice_end_{result}")
                
                # Re-enable buttons
                start_button.config(state=tk.NORMAL)
                cancel_button.config(state=tk.NORMAL)
                
                if self.practice_passed:
                    done_button.config(state=tk.NORMAL)
            
            # Create buttons
            start_button = ttk.Button(button_frame, text="Start Practice", command=run_practice)
            start_button.pack(side=tk.LEFT, padx=10)
            
            done_button = ttk.Button(button_frame, text="Proceed to Experiment", 
                                  command=root.destroy, state=tk.DISABLED)
            done_button.pack(side=tk.LEFT, padx=10)
            
            # Function to handle cancel
            def on_cancel():
                if messagebox.askyesno("Cancel Practice", "Are you sure you want to cancel the practice?"):
                    self.practice_passed = False
                    root.destroy()
            
            cancel_button = ttk.Button(button_frame, text="Cancel", command=on_cancel)
            cancel_button.pack(side=tk.LEFT, padx=10)
            
            # Set up protocol for window close
            def on_close():
                if messagebox.askyesno("Exit", "Are you sure you want to exit practice?"):
                    self.practice_passed = False
                    root.destroy()
            
            root.protocol("WM_DELETE_WINDOW", on_close)
            
            # Start the GUI
            root.mainloop()
            
            # Restore parent window if provided
            if parent_window and parent_window.winfo_exists():
                parent_window.deiconify()
            
            return self.practice_passed
            
        except Exception as e:
            logging.error(f"Error in practice trials: {e}")
            self.practice_passed = False
            
            # Clean up
            if 'root' in locals() and root.winfo_exists():
                root.destroy()
            
            # Restore parent window if provided
            if parent_window and parent_window.winfo_exists():
                parent_window.deiconify()
            
            return False
    
    def run_experiment(self):
        """Run the complete experiment with GUI."""
        try:
            # Create main experiment window
            root = tk.Tk()
            root.title(f"PPS Experiment - Participant {self.participant_initials}")
            root.geometry("800x600")
            
            # Center the window
            root.update_idletasks()
            width = root.winfo_width()
            height = root.winfo_height()
            x = (root.winfo_screenwidth() // 2) - (width // 2)
            y = (root.winfo_screenheight() // 2) - (height // 2)
            root.geometry(f'+{x}+{y}')
            
            # Set up LSL streams
            if not self.setup_lsl_streams():
                messagebox.showerror("Error", "Failed to set up LSL streams")
                root.destroy()
                return False
            
            # Set up LSL recorder
            if not self.setup_lsl_recorder():
                messagebox.showerror("Error", "Failed to set up LSL recorder")
                root.destroy()
                return False
            
            # Set up click listener on the window
            if not self.setup_click_listener(root):
                messagebox.showerror("Error", "Failed to set up click listener")
                root.destroy()
                return False
            
            # Create main frame
            main_frame = ttk.Frame(root, padding="20")
            main_frame.pack(fill=tk.BOTH, expand=True)
            
            # Header
            ttk.Label(main_frame, text=f"PPS Experiment - {self.participant_initials}", 
                     font=("Arial", 18, "bold")).pack(pady=10)
            
            # Info frame
            info_frame = ttk.LabelFrame(main_frame, text="Experiment Information", padding=10)
            info_frame.pack(fill=tk.X, padx=10, pady=10)
            
            ttk.Label(info_frame, text=f"Participant: {self.participant_initials}", 
                     font=("Arial", 12, "bold")).pack(anchor=tk.W, pady=2)
            ttk.Label(info_frame, text=f"Looming Audio: {os.path.basename(self.looming_audio_file)}").pack(anchor=tk.W, pady=2)
            ttk.Label(info_frame, text=f"Tactile Audio: {os.path.basename(self.tactile_audio_file)}").pack(anchor=tk.W, pady=2)
            ttk.Label(info_frame, text=f"Trials: {len(self.design_data) if self.design_data is not None else 'Unknown'}").pack(anchor=tk.W, pady=2)
            
            # Status frame
            status_frame = ttk.Frame(main_frame)
            status_frame.pack(fill=tk.X, padx=10, pady=20)
            
            status_var = tk.StringVar(value="Ready to start")
            status_label = ttk.Label(status_frame, textvariable=status_var, font=("Arial", 12, "bold"))
            status_label.pack(pady=10)
            
            # Progress bar
            progress_var = tk.DoubleVar(value=0.0)
            progress_bar = ttk.Progressbar(status_frame, variable=progress_var, maximum=100)
            progress_bar.pack(fill=tk.X, pady=5)
            
            # Time elapsed
            time_var = tk.StringVar(value="Elapsed time: 00:00")
            time_label = ttk.Label(status_frame, textvariable=time_var)
            time_label.pack(pady=5)
            
            # Instructions
            instructions_frame = ttk.LabelFrame(main_frame, text="Instructions", padding=10)
            instructions_frame.pack(fill=tk.X, padx=10, pady=10)
            
            ttk.Label(instructions_frame, 
                     text="1. Click Start to begin the experiment",
                     font=("Arial", 11)).pack(anchor=tk.W, pady=2)
            ttk.Label(instructions_frame, 
                     text="2. Click the mouse button when you detect a tactile stimulus",
                     font=("Arial", 11)).pack(anchor=tk.W, pady=2)
            ttk.Label(instructions_frame, 
                     text="3. The experiment will end automatically when complete",
                     font=("Arial", 11)).pack(anchor=tk.W, pady=2)
            ttk.Label(instructions_frame, 
                     text="4. Use Pause if needed for a break",
                     font=("Arial", 11)).pack(anchor=tk.W, pady=2)
            
            # Function to start the experiment
            def start_experiment():
                nonlocal start_button, pause_button, practice_button
                
                # Check if practice was passed
                if not self.practice_passed:
                    messagebox.showerror("Practice Required", 
                                       "The participant must successfully complete the practice trials before starting the experiment.")
                    return
                
                # Disable buttons
                start_button.config(state=tk.DISABLED)
                practice_button.config(state=tk.DISABLED)
                pause_button.config(state=tk.NORMAL)
                
                # Reset flags
                self.stop_flag = False
                self.pause_flag = False
                
                # Update status
                status_var.set("Experiment running... Click to respond to stimuli")
                
                # Start progress and time updater
                def update_progress():
                    start_time = time.perf_counter()
                    while not self.stop_flag:
                        if self.experiment_start_time is not None and not self.pause_flag:
                            # Calculate progress based on audio duration (estimated at 15 minutes)
                            elapsed = time.perf_counter() - self.experiment_start_time
                            # Assume 15 minute duration
                            progress = min(100, (elapsed / (15 * 60)) * 100)
                            progress_var.set(progress)
                            
                            # Update time display
                            minutes = int(elapsed // 60)
                            seconds = int(elapsed % 60)
                            time_var.set(f"Elapsed time: {minutes:02d}:{seconds:02d}")
                        
                        # Update every 100ms
                        time.sleep(0.1)
                
                progress_thread = threading.Thread(target=update_progress)
                progress_thread.daemon = True
                progress_thread.start()
                
                # Play audio files in a separate thread
                def run_audio():
                    try:
                        success = self.play_audio_files()
                        
                        # Update UI when complete
                        if success and not self.stop_flag:
                            status_var.set("Experiment completed successfully")
                            messagebox.showinfo("Complete", 
                                             f"Experiment for participant {self.participant_initials} has completed successfully.")
                        elif self.stop_flag:
                            status_var.set("Experiment stopped")
                        else:
                            status_var.set("Error during playback")
                        
                        # Set progress to 100% if completed
                        if success and not self.stop_flag:
                            progress_var.set(100)
                        
                        # Re-enable buttons
                        start_button.config(state=tk.NORMAL)
                        practice_button.config(state=tk.NORMAL)
                        pause_button.config(state=tk.DISABLED)
                        resume_button.config(state=tk.DISABLED)
                        
                    except Exception as e:
                        logging.error(f"Error in audio thread: {e}")
                        status_var.set(f"Error: {str(e)}")
                        messagebox.showerror("Error", f"An error occurred during playback:\n\n{str(e)}")
                        
                        # Re-enable buttons
                        start_button.config(state=tk.NORMAL)
                        practice_button.config(state=tk.NORMAL)
                        pause_button.config(state=tk.DISABLED)
                        resume_button.config(state=tk.DISABLED)
                
                audio_thread = threading.Thread(target=run_audio)
                audio_thread.daemon = True
                audio_thread.start()
            
            # Function to pause experiment
            def pause_experiment():
                nonlocal pause_button, resume_button
                
                self.pause_experiment()
                status_var.set("Experiment PAUSED")
                
                pause_button.config(state=tk.DISABLED)
                resume_button.config(state=tk.NORMAL)
            
            # Function to resume experiment
            def resume_experiment():
                nonlocal pause_button, resume_button
                
                self.resume_experiment()
                status_var.set("Experiment running... Click to respond to stimuli")
                
                pause_button.config(state=tk.NORMAL)
                resume_button.config(state=tk.DISABLED)
            
            # Function to run practice trials
            def run_practice():
                self.run_practice_trials(parent_window=root)
                
                # Update button states based on practice result
                if self.practice_passed:
                    practice_button.config(text="Practice Passed ✓")
                    start_button.config(state=tk.NORMAL)
                else:
                    practice_button.config(text="Run Practice")
            
            # Buttons
            button_frame = ttk.Frame(main_frame)
            button_frame.pack(pady=20)
            
            # Practice button
            practice_button = ttk.Button(button_frame, text="Run Practice", command=run_practice)
            practice_button.pack(side=tk.LEFT, padx=10)
            
            # Start button (disabled until practice is passed)
            start_button = ttk.Button(button_frame, text="Start Experiment", 
                                    command=start_experiment, state=tk.DISABLED)
            start_button.pack(side=tk.LEFT, padx=10)
            
            # Pause button (initially disabled)
            pause_button = ttk.Button(button_frame, text="Pause", 
                                    command=pause_experiment, state=tk.DISABLED)
            pause_button.pack(side=tk.LEFT, padx=10)
            
            # Resume button (initially disabled)
            resume_button = ttk.Button(button_frame, text="Resume", 
                                     command=resume_experiment, state=tk.DISABLED)
            resume_button.pack(side=tk.LEFT, padx=10)
            
            # Function to handle stop/cancel
            def on_stop():
                if messagebox.askyesno("Stop Experiment", "Are you sure you want to stop the experiment?"):
                    self.stop_experiment()
                    status_var.set("Experiment stopped")
                    
                    # Reset button states
                    start_button.config(state=tk.NORMAL)
                    practice_button.config(state=tk.NORMAL)
                    pause_button.config(state=tk.DISABLED)
                    resume_button.config(state=tk.DISABLED)
            
            ttk.Button(button_frame, text="Stop", command=on_stop).pack(side=tk.LEFT, padx=10)
            
            # Set up protocol for window close
            def on_close():
                if self.experiment_start_time is not None and not self.stop_flag:
                    if not messagebox.askyesno("Exit", "Experiment is running. Are you sure you want to exit?"):
                        return
                
                self.stop_experiment()
                root.destroy()
            
            root.protocol("WM_DELETE_WINDOW", on_close)
            
            # Run the GUI
            root.mainloop()
            
            # Ensure we clean up
            self.stop_experiment()
            
            return True
            
        except Exception as e:
            logging.error(f"Error running experiment: {e}")
            messagebox.showerror("Error", f"An error occurred during the experiment:\n\n{str(e)}")
            return False
    
    def main(self):
        """Main entry point for the experiment runner."""
        try:
            # Get participant info
            if not self.get_participant_info():
                logging.info("No participant info provided. Exiting.")
                return
            
            # Find participant files
            try:
                self.find_participant_files()
            except FileNotFoundError as e:
                messagebox.showerror("Error", f"Could not find required files: {str(e)}")
                return
            
            # Confirm LSL recorder
            if not self.confirm_lsl_recorder():
                messagebox.showwarning("Warning", "LSL recorder confirmation required. Experiment cannot proceed.")
                return
            
            # Run practice trials and then the main experiment
            self.run_experiment()
            
        except Exception as e:
            logging.error(f"Experiment failed: {e}")
            messagebox.showerror("Error", f"The experiment failed with an error:\n\n{str(e)}")
        
        finally:
            # Ensure we clean up
            self.stop_flag = True
            logging.info("Experiment runner completed")

# Entry point
if __name__ == "__main__":
    runner = PPSExperimentRunner()
    runner.main()

2025-03-19 16:25:56,358 - INFO - PPS Experiment Runner initialized
2025-03-19 16:26:11,624 - INFO - Set participant ID: 1, Identifier: GF01
2025-03-19 16:26:11,626 - INFO - Found design file: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PPS_Experiment_Module\ExperimentLog\participant_1_design.csv
2025-03-19 16:26:11,648 - INFO - Loaded design data with 204 trials
2025-03-19 16:26:11,650 - INFO - Found looming audio file: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PPS_Experiment_Module\ExperimentAudio\participant_1_design_looming.wav
2025-03-19 16:26:11,651 - INFO - Found tactile audio file: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PPS_Experiment_Module\ExperimentAudio\participant_1_design_tactile.wav
2025-03-19 16:26:19,135 - INFO - Created LSL outlet for experiment markers: ExperimentMarkers_GF01
2025-03-19 16:26:19,138 - INFO - Created LSL outlet for mouse clicks: MouseClicks_GF01
2025-03-19 16:26:19,139 - INFO - Starting LSL recording to 

In [ ]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        self.root.geometry("800x600")
        
        # Center the window
        self.root.update_idletasks()
        width = self.root.winfo_width()
        height = self.root.winfo_height()
        x = (self.root.winfo_screenwidth() // 2) - (width // 2)
        y = (self.root.winfo_screenheight() // 2) - (height // 2)
        self.root.geometry(f'+{x}+{y}')
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=20)
        
        # Start Trial button
        self.start_button = ttk.Button(self.button_frame, text="Start Trial", 
                                  command=self.start_trial, width=20)
        self.start_button.pack(side=tk.LEFT, padx=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_style = ttk.Style()
        self.continue_style.configure("Continue.TButton", foreground="black", background="gray")
        
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Continue.TButton", width=20)
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
        
        # Set up closing protocol
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        if not self.experiment_running or self.start_time is None:
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat()
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        # Disable the start button during trial
        self.start_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            self.status_var.set("Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            
            # Play both audio files in separate threads
            def play_looming():
                sd.play(looming_data, looming_sr, device=0, blocking=True)
            
            def play_tactile():
                sd.play(tactile_data, tactile_sr, device=1, blocking=True)
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Update status periodically
            while time.perf_counter() < end_time and looming_thread.is_alive() or tactile_thread.is_alive():
                elapsed = time.perf_counter() - self.start_time
                self.status_var.set(f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                self.root.update()
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            self.status_var.set("Practice trial completed - analyzing results...")
            self.root.update()
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            self.status_var.set(f"Error during practice trial: {str(e)}")
            print(f"Error during practice trial: {e}")
        finally:
            self.experiment_running = False
            self.start_button.config(state=tk.NORMAL)
            self.save_log()
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        # Check if mouse clicks occurred within 1 second of tactile stimuli
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click
        for click in self.mouse_clicks:
            click_time = click["time"]
            
            # Find closest tactile stimulus
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                # Only consider stimuli that happened before the click
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
            else:
                false_alarms += 1
        
        # Count missed stimuli
        missed_stimuli = stimulus_responded.count(False)
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        
        # Update results display
        results_text = (
            f"Practice Results:\n\n"
            f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
            f"False alarms: {false_alarms}\n"
            f"Missed stimuli: {missed_stimuli}\n\n"
        )
        
        if self.practice_passed:
            results_text += "PASSED! You understood the task correctly."
            self.status_var.set("Practice passed successfully!")
            
            # Enable and highlight continue button
            self.continue_button.config(state=tk.NORMAL)
            self.continue_style.configure("Continue.TButton", background="green", foreground="white")
        else:
            results_text += "FAILED. Please try again and remember:\n"\
                           "- Click ONLY when you hear a tactile stimulus\n"\
                           "- You must click within 1 second after the stimulus\n"\
                           "- Don't click for looming sounds without tactile stimuli"
            self.status_var.set("Practice failed - try again")
        
        self.results_var.set(results_text)
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Set up closing protocol
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        self.root.geometry("800x600")
        
        # Center the window
        self.root.update_idletasks()
        width = self.root.winfo_width()
        height = self.root.winfo_height()
        x = (self.root.winfo_screenwidth() // 2) - (width // 2)
        y = (self.root.winfo_screenheight() // 2) - (height // 2)
        self.root.geometry(f'+{x}+{y}')
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators
        self.visual_canvas = tk.Canvas(visual_frame, width=760, height=100, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Timeline
        self.visual_canvas.create_line(400, 20, 740, 20)
        self.visual_canvas.create_line(400, 90, 740, 90)
        self.timeline_markers = []
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=20)
        
        # Start Trial button
        self.start_button = ttk.Button(self.button_frame, text="Start Trial", 
                                  command=self.start_trial, width=20)
        self.start_button.pack(side=tk.LEFT, padx=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20)
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        if not self.experiment_running or self.start_time is None:
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat()
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds")
        
        # Visual indicator for the experimenter
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline
        self.add_timeline_marker(current_time, "red")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        # Disable the start button during trial
        self.start_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter"""
        original_color = self.visual_canvas.itemcget(indicator, "fill")
        self.visual_canvas.itemconfig(indicator, fill=color)
        self.root.update()
        
        # Schedule color change back
        def reset_color():
            if self.visual_canvas.winfo_exists():
                self.visual_canvas.itemconfig(indicator, fill=original_color)
        
        self.root.after(int(duration * 1000), reset_color)
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time"""
        # Timeline spans from 400 to 740 pixels horizontally
        # Assuming max duration of 60 seconds for scaling
        max_duration = 60
        x_pos = 400 + min(time_sec / max_duration, 1.0) * 340
        
        # Create marker
        marker = self.visual_canvas.create_oval(x_pos-3, 87, x_pos+3, 93, fill=color, outline=color)
        self.timeline_markers.append(marker)
    
    def clear_timeline(self):
        """Clear all markers from the timeline"""
        for marker in self.timeline_markers:
            self.visual_canvas.delete(marker)
        self.timeline_markers = []
        
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                self.add_timeline_marker(t_time, "blue")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            self.status_var.set("Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                sd.play(looming_data, looming_sr, device=0, blocking=True)
            
            def play_tactile():
                sd.play(tactile_data, tactile_sr, device=1, blocking=True)
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.status_var.set(f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                self.root.update()
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            self.status_var.set("Practice trial completed - analyzing results...")
            self.root.update()
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            self.status_var.set(f"Error during practice trial: {str(e)}")
            print(f"Error during practice trial: {e}")
        finally:
            self.experiment_running = False
            self.start_button.config(state=tk.NORMAL)
            self.save_log()
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback"""
        if not tactile_times.size:
            return
            
        while self.experiment_running:
            current_time = time.perf_counter() - self.start_time
            
            # Check if we're at a tactile stimulus time
            for t_time in tactile_times:
                if abs(current_time - t_time) < 0.1:  # Within 100ms window
                    self.flash_indicator(self.tactile_indicator, "yellow")
                    break
                    
            time.sleep(0.05)  # Check frequently
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        # Check if mouse clicks occurred within 1 second of tactile stimuli
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click
        for click in self.mouse_clicks:
            click_time = click["time"]
            
            # Find closest tactile stimulus
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                # Only consider stimuli that happened before the click
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
            else:
                false_alarms += 1
        
        # Count missed stimuli
        missed_stimuli = stimulus_responded.count(False)
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        
        # Update results display
        results_text = (
            f"Practice Results:\n\n"
            f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
            f"False alarms: {false_alarms}\n"
            f"Missed stimuli: {missed_stimuli}\n\n"
        )
        
        if self.practice_passed:
            results_text += "PASSED! You understood the task correctly."
            self.status_var.set("Practice passed successfully!")
            
            # Enable and highlight continue button
            self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
        else:
            results_text += "FAILED. Please try again and remember:\n"\
                           "- Click ONLY when you hear a tactile stimulus\n"\
                           "- You must click within 1 second after the stimulus\n"\
                           "- Don't click for looming sounds without tactile stimuli"
            self.status_var.set("Practice failed - try again")
        
        self.results_var.set(results_text)
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        self.root.geometry("800x600")
        
        # Center the window
        self.root.update_idletasks()
        width = self.root.winfo_width()
        height = self.root.winfo_height()
        x = (self.root.winfo_screenwidth() // 2) - (width // 2)
        y = (self.root.winfo_screenheight() // 2) - (height // 2)
        self.root.geometry(f'+{x}+{y}')
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators
        self.visual_canvas = tk.Canvas(visual_frame, width=760, height=100, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Timeline
        self.visual_canvas.create_line(400, 20, 740, 20)
        self.visual_canvas.create_line(400, 90, 740, 90)
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 12, "bold"))
        
        # Create a distinctive frame just for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=10)
        start_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Start Trial button - large and prominent
        self.start_button = ttk.Button(start_frame, text="START TRIAL", 
                                  command=self.start_trial, width=30, style="Start.TButton")
        self.start_button.pack(pady=10, padx=10)
        
        # Button frame for other buttons
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # Display debug information
        debug_frame = ttk.LabelFrame(self.main_frame, text="Debug Info", padding=5)
        debug_frame.pack(fill=tk.X, padx=10, pady=5, before=self.button_frame)
        
        ttk.Label(debug_frame, text="If you don't see the START TRIAL button above, please check the 'Begin Practice' section.",
                 font=("Arial", 9), foreground="red").pack(pady=2)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        if not self.experiment_running or self.start_time is None:
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat()
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds")
        
        # Visual indicator for the experimenter
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline
        self.add_timeline_marker(current_time, "red")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter"""
        original_color = self.visual_canvas.itemcget(indicator, "fill")
        self.visual_canvas.itemconfig(indicator, fill=color)
        self.root.update()
        
        # Schedule color change back
        def reset_color():
            if self.visual_canvas.winfo_exists():
                self.visual_canvas.itemconfig(indicator, fill=original_color)
        
        self.root.after(int(duration * 1000), reset_color)
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time"""
        # Timeline spans from 400 to 740 pixels horizontally
        # Assuming max duration of 60 seconds for scaling
        max_duration = 60
        x_pos = 400 + min(time_sec / max_duration, 1.0) * 340
        
        # Create marker
        marker = self.visual_canvas.create_oval(x_pos-3, 87, x_pos+3, 93, fill=color, outline=color)
        self.timeline_markers.append(marker)
    
    def clear_timeline(self):
        """Clear all markers from the timeline"""
        for marker in self.timeline_markers:
            self.visual_canvas.delete(marker)
        self.timeline_markers = []
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        # Disable the start button during trial
        self.start_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                self.add_timeline_marker(t_time, "blue")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            self.status_var.set("Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                sd.play(looming_data, looming_sr, blocking=True)
            
            def play_tactile():
                sd.play(tactile_data, tactile_sr, blocking=True)
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.status_var.set(f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                self.root.update()
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            self.status_var.set("Practice trial completed - analyzing results...")
            self.root.update()
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            self.status_var.set(f"Error during practice trial: {str(e)}")
            print(f"Error during practice trial: {e}")
        finally:
            self.experiment_running = False
            self.start_button.config(state=tk.NORMAL)
            self.save_log()
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback"""
        if not tactile_times.size:
            return
            
        while self.experiment_running:
            current_time = time.perf_counter() - self.start_time
            
            # Check if we're at a tactile stimulus time
            for t_time in tactile_times:
                if abs(current_time - t_time) < 0.1:  # Within 100ms window
                    self.flash_indicator(self.tactile_indicator, "yellow")
                    break
                    
            time.sleep(0.05)  # Check frequently
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        # Check if mouse clicks occurred within 1 second of tactile stimuli
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click
        for click in self.mouse_clicks:
            click_time = click["time"]
            
            # Find closest tactile stimulus
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                # Only consider stimuli that happened before the click
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
            else:
                false_alarms += 1
        
        # Count missed stimuli
        missed_stimuli = stimulus_responded.count(False)
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        
        # Update results display
        results_text = (
            f"Practice Results:\n\n"
            f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
            f"False alarms: {false_alarms}\n"
            f"Missed stimuli: {missed_stimuli}\n\n"
        )
        
        if self.practice_passed:
            results_text += "PASSED! You understood the task correctly."
            self.status_var.set("Practice passed successfully!")
            
            # Enable and highlight continue button
            self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
        else:
            results_text += "FAILED. Please try again and remember:\n"\
                           "- Click ONLY when you hear a tactile stimulus\n"\
                           "- You must click within 1 second after the stimulus\n"\
                           "- Don't click for looming sounds without tactile stimuli"
            self.status_var.set("Practice failed - try again")
        
        self.results_var.set(results_text)
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Make window full screen from the start
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators
        self.visual_canvas = tk.Canvas(visual_frame, width=760, height=100, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Timeline
        self.visual_canvas.create_line(400, 20, 740, 20)
        self.visual_canvas.create_line(400, 90, 740, 90)
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 12, "bold"))
        
        # Create a distinctive frame just for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=10)
        start_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Start Trial button - large and prominent
        self.start_button = ttk.Button(start_frame, text="START TRIAL", 
                                  command=self.start_trial, width=30, style="Start.TButton")
        self.start_button.pack(pady=10, padx=10)
        
        # Try Again button (initially hidden)
        self.try_again_button = ttk.Button(start_frame, text="TRY AGAIN", 
                                     command=self.start_trial, width=30, style="Start.TButton")
        self.try_again_button.pack(pady=10, padx=10)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(self.results_frame, text="Show Detailed Diagnostics", 
                                          command=self.show_diagnostic_report, state=tk.DISABLED)
        self.show_diagnostic_button.pack(anchor=tk.E, pady=5)
        
        # Button frame for other buttons
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
    
    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window"""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        report_window.geometry("800x600")
        
        # Add text widget with scrollbar
        frame = ttk.Frame(report_window, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        scrollbar = ttk.Scrollbar(frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        text_widget = tk.Text(frame, wrap=tk.WORD, yscrollcommand=scrollbar.set)
        text_widget.pack(fill=tk.BOTH, expand=True)
        scrollbar.config(command=text_widget.yview)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # Display debug information
        debug_frame = ttk.LabelFrame(self.main_frame, text="Debug Info", padding=5)
        debug_frame.pack(fill=tk.X, padx=10, pady=5, before=self.button_frame)
        
        ttk.Label(debug_frame, text="If you don't see the START TRIAL button above, please check the 'Begin Practice' section.",
                 font=("Arial", 9), foreground="red").pack(pady=2)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        if not self.experiment_running or self.start_time is None:
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat()
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds")
        
        # Visual indicator for the experimenter
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline
        self.add_timeline_marker(current_time, "red")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter"""
        original_color = self.visual_canvas.itemcget(indicator, "fill")
        self.visual_canvas.itemconfig(indicator, fill=color)
        self.root.update()
        
        # Schedule color change back
        def reset_color():
            if self.visual_canvas.winfo_exists():
                self.visual_canvas.itemconfig(indicator, fill=original_color)
        
        self.root.after(int(duration * 1000), reset_color)
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time"""
        # Timeline spans from 400 to 740 pixels horizontally
        # Assuming max duration of 60 seconds for scaling
        max_duration = 60
        x_pos = 400 + min(time_sec / max_duration, 1.0) * 340
        
        # Create marker
        marker = self.visual_canvas.create_oval(x_pos-3, 87, x_pos+3, 93, fill=color, outline=color)
        self.timeline_markers.append(marker)
    
    def clear_timeline(self):
        """Clear all markers from the timeline"""
        for marker in self.timeline_markers:
            self.visual_canvas.delete(marker)
        self.timeline_markers = []
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                self.add_timeline_marker(t_time, "blue")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz")
            
            self.status_var.set("Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                try:
                    # Try to use device 0 first
                    sd.play(looming_data, looming_sr, device=0, blocking=True)
                except Exception as e:
                    print(f"Error playing on device 0: {e}")
                    # Fall back to default device
                    sd.play(looming_data, looming_sr, blocking=True)
            
            def play_tactile():
                try:
                    # Try to use device 1 first
                    sd.play(tactile_data, tactile_sr, device=1, blocking=True)
                except Exception as e:
                    print(f"Error playing on device 1: {e}")
                    # Fall back to default device
                    sd.play(tactile_data, tactile_sr, blocking=True)
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.status_var.set(f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                self.root.update()
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            self.status_var.set("Practice trial completed - analyzing results...")
            self.root.update()
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            self.status_var.set(f"Error during practice trial: {str(e)}")
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
        finally:
            self.experiment_running = False
            self.save_log()
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback"""
        if not tactile_times.size:
            return
            
        while self.experiment_running:
            current_time = time.perf_counter() - self.start_time
            
            # Check if we're at a tactile stimulus time
            for t_time in tactile_times:
                if abs(current_time - t_time) < 0.1:  # Within 100ms window
                    self.flash_indicator(self.tactile_indicator, "yellow")
                    break
                    
            time.sleep(0.05)  # Check frequently
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        # Detailed result tracking
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Lists to store detailed diagnostics
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click against tactile stimuli
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            
            # Find closest tactile stimulus that occurred before this click
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                print(f"  Tactile {i+1} at {stim_time:.3f}s: diff = {time_diff:.3f}s")
                
                # Only consider stimuli that happened before the click and within window
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > 1.0s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within 1 second")
                missed_stimuli += 1
                missed_details.append({
                    'tactile_time': tactile_times[i]
                })
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        # Create detailed diagnostic report
        diagnostic_report = "===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        
        # Correct responses
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
        else:
            diagnostic_report += "  None\n"
        
        # False alarms
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                else:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
        else:
            diagnostic_report += "  None\n"
        
        # Missed stimuli
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within 1 second\n"
        else:
            diagnostic_report += "  None\n"
        
        # Conclusion
        diagnostic_report += f"\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        # Update results display with basic summary
        results_text = (
            f"Practice Results:\n\n"
            f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
            f"False alarms: {false_alarms}\n"
            f"Missed stimuli: {missed_stimuli}\n\n"
        )
        
        if self.practice_passed:
            results_text += "PASSED! You understood the task correctly."
            self.status_var.set("Practice passed successfully!")
            
            # Enable and highlight continue button
            self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
            
            # Show diagnostic button
            self.show_diagnostic_button.config(state=tk.NORMAL)
            
            # Hide Start button, show Try Again
            self.start_button.pack_forget()
            self.try_again_button.pack()
        else:
            results_text += "FAILED. Please try again and remember:\n"\
                           "- Click ONLY when you hear a tactile stimulus\n"\
                           "- You must click within 1 second after the stimulus\n"\
                           "- Don't click for looming sounds without tactile stimuli"
            self.status_var.set("Practice failed - try again")
            
            # Show diagnostic button
            self.show_diagnostic_button.config(state=tk.NORMAL)
            
            # Hide Start button, show Try Again
            self.start_button.pack_forget()
            self.try_again_button.pack()
        
        self.results_var.set(results_text)
        
        # Save diagnostic report
        self.diagnostic_report = diagnostic_report
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Make window full screen from the start
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators
        self.visual_canvas = tk.Canvas(visual_frame, width=760, height=100, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Timeline
        self.visual_canvas.create_line(400, 20, 740, 20)
        self.visual_canvas.create_line(400, 90, 740, 90)
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 12, "bold"))
        
        # Create a distinctive frame just for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=10)
        start_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Start Trial button - large and prominent
        self.start_button = ttk.Button(start_frame, text="START TRIAL", 
                                  command=self.start_trial, width=30, style="Start.TButton")
        self.start_button.pack(pady=10, padx=10)
        
        # Try Again button (initially hidden)
        self.try_again_button = ttk.Button(start_frame, text="TRY AGAIN", 
                                     command=self.start_trial, width=30, style="Start.TButton")
        self.try_again_button.pack(pady=10, padx=10)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(self.results_frame, text="Show Detailed Diagnostics", 
                                          command=self.show_diagnostic_report, state=tk.DISABLED)
        self.show_diagnostic_button.pack(anchor=tk.E, pady=5)
        
        # Button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
    
    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window"""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        report_window.geometry("800x600")
        
        # Add text widget with scrollbar
        frame = ttk.Frame(report_window, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        scrollbar = ttk.Scrollbar(frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        text_widget = tk.Text(frame, wrap=tk.WORD, yscrollcommand=scrollbar.set)
        text_widget.pack(fill=tk.BOTH, expand=True)
        scrollbar.config(command=text_widget.yview)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # Display debug information
        debug_frame = ttk.LabelFrame(self.main_frame, text="Debug Info", padding=5)
        debug_frame.pack(fill=tk.X, padx=10, pady=5, before=self.button_frame)
        
        ttk.Label(debug_frame, text="If you don't see the START TRIAL button above, please check the 'Begin Practice' section.",
                 font=("Arial", 9), foreground="red").pack(pady=2)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        if not self.experiment_running or self.start_time is None:
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat()
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds")
        
        # Visual indicator for the experimenter
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline
        self.add_timeline_marker(current_time, "red")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter"""
        original_color = self.visual_canvas.itemcget(indicator, "fill")
        self.visual_canvas.itemconfig(indicator, fill=color)
        self.root.update()
        
        # Schedule color change back
        def reset_color():
            if self.visual_canvas.winfo_exists():
                self.visual_canvas.itemconfig(indicator, fill=original_color)
        
        self.root.after(int(duration * 1000), reset_color)
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time"""
        # Timeline spans from 400 to 740 pixels horizontally
        # Assuming max duration of 60 seconds for scaling
        max_duration = 60
        x_pos = 400 + min(time_sec / max_duration, 1.0) * 340
        
        # Create marker
        marker = self.visual_canvas.create_oval(x_pos-3, 87, x_pos+3, 93, fill=color, outline=color)
        self.timeline_markers.append(marker)
    
    def clear_timeline(self):
        """Clear all markers from the timeline"""
        for marker in self.timeline_markers:
            self.visual_canvas.delete(marker)
        self.timeline_markers = []
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                self.add_timeline_marker(t_time, "blue")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz")
            
            self.status_var.set("Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                try:
                    # Try to use device 0 first
                    sd.play(looming_data, looming_sr, device=0, blocking=True)
                except Exception as e:
                    print(f"Error playing on device 0: {e}")
                    # Fall back to default device
                    sd.play(looming_data, looming_sr, blocking=True)
            
            def play_tactile():
                try:
                    # Try to use device 1 first
                    sd.play(tactile_data, tactile_sr, device=1, blocking=True)
                except Exception as e:
                    print(f"Error playing on device 1: {e}")
                    # Fall back to default device
                    sd.play(tactile_data, tactile_sr, blocking=True)
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.status_var.set(f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                self.root.update()
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            self.status_var.set("Practice trial completed - analyzing results...")
            self.root.update()
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            self.status_var.set(f"Error during practice trial: {str(e)}")
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
        finally:
            self.experiment_running = False
            self.save_log()
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback"""
        if not tactile_times.size:
            return
            
        while self.experiment_running:
            current_time = time.perf_counter() - self.start_time
            
            # Check if we're at a tactile stimulus time
            for t_time in tactile_times:
                if abs(current_time - t_time) < 0.1:  # Within 100ms window
                    self.flash_indicator(self.tactile_indicator, "yellow")
                    break
                    
            time.sleep(0.05)  # Check frequently
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        # Detailed result tracking
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Lists to store detailed diagnostics
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click against tactile stimuli
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            
            # Find closest tactile stimulus that occurred before this click
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                print(f"  Tactile {i+1} at {stim_time:.3f}s: diff = {time_diff:.3f}s")
                
                # Only consider stimuli that happened before the click and within window
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > 1.0s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within 1 second")
                missed_stimuli += 1
                missed_details.append({
                    'tactile_time': tactile_times[i]
                })
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        # Create detailed diagnostic report
        diagnostic_report = "===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        
        # Correct responses
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
        else:
            diagnostic_report += "  None\n"
        
        # False alarms
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                else:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
        else:
            diagnostic_report += "  None\n"
        
        # Missed stimuli
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within 1 second\n"
        else:
            diagnostic_report += "  None\n"
        
        # Conclusion
        diagnostic_report += f"\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        # Update results display with basic summary
        results_text = (
            f"Practice Results:\n\n"
            f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
            f"False alarms: {false_alarms}\n"
            f"Missed stimuli: {missed_stimuli}\n\n"
        )
        
        if self.practice_passed:
            results_text += "PASSED! You understood the task correctly."
            self.status_var.set("Practice passed successfully!")
            
            # Enable and highlight continue button
            self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
            
            # Show diagnostic button
            self.show_diagnostic_button.config(state=tk.NORMAL)
            
            # Hide Start button, show Try Again
            self.start_button.pack_forget()
            self.try_again_button.pack()
        else:
            results_text += "FAILED. Please try again and remember:\n"\
                           "- Click ONLY when you hear a tactile stimulus\n"\
                           "- You must click within 1 second after the stimulus\n"\
                           "- Don't click for looming sounds without tactile stimuli"
            self.status_var.set("Practice failed - try again")
            
            # Show diagnostic button
            self.show_diagnostic_button.config(state=tk.NORMAL)
            
            # Hide Start button, show Try Again
            self.start_button.pack_forget()
            self.try_again_button.pack()
        
        self.results_var.set(results_text)
        
        # Save diagnostic report
        self.diagnostic_report = diagnostic_report
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Make window full screen from the start
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators
        self.visual_canvas = tk.Canvas(visual_frame, width=760, height=100, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Timeline
        self.visual_canvas.create_line(400, 20, 740, 20)
        self.visual_canvas.create_line(400, 90, 740, 90)
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 12, "bold"))
        
        # Create a distinctive frame just for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=10)
        start_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Start Trial button - large and prominent
        self.start_button = ttk.Button(start_frame, text="START TRIAL", 
                                  command=self.start_trial, width=30, style="Start.TButton")
        self.start_button.pack(pady=10, padx=10)
        
        # Try Again button (initially hidden)
        self.try_again_button = ttk.Button(start_frame, text="TRY AGAIN", 
                                     command=self.start_trial, width=30, style="Start.TButton")
        self.try_again_button.pack(pady=10, padx=10)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(self.results_frame, text="Show Detailed Diagnostics", 
                                          command=self.show_diagnostic_report, state=tk.DISABLED)
        self.show_diagnostic_button.pack(anchor=tk.E, pady=5)
        
        # Button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
    
    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window"""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        report_window.geometry("800x600")
        
        # Add text widget with scrollbar
        frame = ttk.Frame(report_window, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        scrollbar = ttk.Scrollbar(frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        text_widget = tk.Text(frame, wrap=tk.WORD, yscrollcommand=scrollbar.set)
        text_widget.pack(fill=tk.BOTH, expand=True)
        scrollbar.config(command=text_widget.yview)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # Display debug information
        debug_frame = ttk.LabelFrame(self.main_frame, text="Debug Info", padding=5)
        debug_frame.pack(fill=tk.X, padx=10, pady=5, before=self.button_frame)
        
        ttk.Label(debug_frame, text="If you don't see the START TRIAL button above, please check the 'Begin Practice' section.",
                 font=("Arial", 9), foreground="red").pack(pady=2)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        if not self.experiment_running or self.start_time is None:
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat()
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds")
        
        # Visual indicator for the experimenter - must be done on main thread
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline - must be done on main thread
        try:
            self.add_timeline_marker(current_time, "red")
            print(f"Added timeline marker at {current_time:.3f}s")
        except Exception as e:
            print(f"Error adding timeline marker: {e}")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter"""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            self.root.update_idletasks()
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            print(f"Error flashing indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time"""
        # Timeline spans from 400 to 740 pixels horizontally
        # Assuming max duration of 90 seconds for scaling
        max_duration = 90
        x_pos = 400 + min(time_sec / max_duration, 1.0) * 340
        
        # Create marker
        try:
            marker = self.visual_canvas.create_oval(x_pos-5, 85, x_pos+5, 95, fill=color, outline=color)
            self.timeline_markers.append(marker)
            return True
        except Exception as e:
            print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline"""
        for marker in self.timeline_markers:
            self.visual_canvas.delete(marker)
        self.timeline_markers = []
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                try:
                    self.add_timeline_marker(t_time, "blue")
                except Exception as e:
                    print(f"Error adding tactile marker: {e}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz, shape: {looming_data.shape}")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz, shape: {tactile_data.shape}")
            
            # Convert stereo to mono if needed for playback
            if len(looming_data.shape) > 1 and looming_data.shape[1] > 1:
                print("Converting looming audio from stereo to mono for playback")
                looming_data = np.mean(looming_data, axis=1)
            
            if len(tactile_data.shape) > 1 and tactile_data.shape[1] > 1:
                print("Converting tactile audio from stereo to mono for playback")
                tactile_data = np.mean(tactile_data, axis=1)
                
            # Update status in main thread using after()
            def update_status(msg):
                try:
                    self.status_var.set(msg)
                    self.root.update_idletasks()
                except Exception as e:
                    print(f"Error updating status: {e}")
                    
            self.root.after(0, update_status, "Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                try:
                    sd.play(looming_data, looming_sr, blocking=True)
                    print("Looming audio playback completed")
                except Exception as e:
                    print(f"Error playing looming audio: {e}")
            
            def play_tactile():
                try:
                    sd.play(tactile_data, tactile_sr, blocking=True)
                    print("Tactile audio playback completed")
                except Exception as e:
                    print(f"Error playing tactile audio: {e}")
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            self.root.after(0, update_status, "Practice trial completed - analyzing results...")
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
            
            # Update status in main thread
            def show_error(msg):
                self.status_var.set(msg)
                
            self.root.after(0, show_error, f"Error during practice trial: {str(e)}")
            
        finally:
            self.experiment_running = False
            
            # Re-enable buttons in main thread
            def enable_buttons():
                self.start_button.config(state=tk.NORMAL)
                self.try_again_button.config(state=tk.NORMAL)
                
            self.root.after(0, enable_buttons)
            self.save_log()
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback"""
        if not tactile_times.size:
            return
            
        try:
            while self.experiment_running:
                current_time = time.perf_counter() - self.start_time
                
                # Check if we're at a tactile stimulus time
                for t_time in tactile_times:
                    if abs(current_time - t_time) < 0.1:  # Within 100ms window
                        # Use after() to update the UI from the main thread
                        self.root.after(0, lambda: self.flash_indicator(self.tactile_indicator, "yellow"))
                        print(f"Tactile event detected at {current_time:.3f}s (expected {t_time:.3f}s)")
                        break
                        
                time.sleep(0.05)  # Check frequently
        except Exception as e:
            print(f"Error in tactile monitor: {e}")
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        # Detailed result tracking
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Lists to store detailed diagnostics
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click against tactile stimuli
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            
            # Find closest tactile stimulus that occurred before this click
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                print(f"  Tactile {i+1} at {stim_time:.3f}s: diff = {time_diff:.3f}s")
                
                # Only consider stimuli that happened before the click and within window
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > 1.0s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within 1 second")
                missed_stimuli += 1
                missed_details.append({
                    'tactile_time': tactile_times[i]
                })
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        # Create detailed diagnostic report
        diagnostic_report = "===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        
        # Correct responses
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
        else:
            diagnostic_report += "  None\n"
        
        # False alarms
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                else:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
        else:
            diagnostic_report += "  None\n"
        
        # Missed stimuli
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within 1 second\n"
        else:
            diagnostic_report += "  None\n"
        
        # Conclusion
        diagnostic_report += f"\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        # Save diagnostic report for later viewing
        self.diagnostic_report = diagnostic_report
        
        # Update UI in the main thread
        def update_ui():
            try:
                # Update results display with basic summary
                results_text = (
                    f"Practice Results:\n\n"
                    f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Missed stimuli: {missed_stimuli}\n\n"
                )
                
                if self.practice_passed:
                    results_text += "PASSED! You understood the task correctly."
                    self.status_var.set("Practice passed successfully!")
                    
                    # Enable and highlight continue button
                    self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
                else:
                    results_text += "FAILED. Please try again and remember:\n"\
                                   "- Click ONLY when you hear a tactile stimulus\n"\
                                   "- You must click within 1 second after the stimulus\n"\
                                   "- Don't click for looming sounds without tactile stimuli"
                    self.status_var.set("Practice failed - try again")
                
                self.results_var.set(results_text)
                
                # Show Try Again button instead of Start
                self.start_button.pack_forget()
                self.try_again_button.pack()
                
                # Enable diagnostic button
                self.show_diagnostic_button.config(state=tk.NORMAL)
            except Exception as e:
                print(f"Error updating UI with results: {e}")
        
        # Schedule UI updates on main thread
        self.root.after(0, update_ui)
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

In [1]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        self.click_count = 0
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Make window full screen from the start
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Bind mouse clicks for the entire application - do this FIRST
        self.root.bind("<Button-1>", self.on_mouse_click)
        print("Mouse click binding established")
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators
        self.visual_canvas = tk.Canvas(visual_frame, width=760, height=140, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Timeline
        self.visual_canvas.create_line(400, 20, 740, 20)
        self.visual_canvas.create_line(400, 90, 740, 90)
        
        # Click test area
        self.visual_canvas.create_rectangle(50, 100, 740, 130, fill="lightyellow", outline="black")
        self.visual_canvas.create_text(395, 115, text="Click Test Area (click here to verify mouse tracking)", font=("Arial", 10))
        
        # Add click counter display
        self.click_counter_text = self.visual_canvas.create_text(650, 75, 
                                                            text="Total Clicks: 0", 
                                                            font=("Arial", 12, "bold"),
                                                            fill="blue")
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 12, "bold"))
        
        # Create a distinctive frame just for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=10)
        start_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Start Trial button - large and prominent
        self.start_button = ttk.Button(start_frame, text="START TRIAL", 
                                  command=self.start_trial, width=30, style="Start.TButton")
        self.start_button.pack(pady=10, padx=10)
        
        # Try Again button (initially hidden)
        self.try_again_button = ttk.Button(start_frame, text="TRY AGAIN", 
                                     command=self.start_trial, width=30, style="Start.TButton")
        self.try_again_button.pack(pady=10, padx=10)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(self.results_frame, text="Show Detailed Diagnostics", 
                                          command=self.show_diagnostic_report, state=tk.DISABLED)
        self.show_diagnostic_button.pack(anchor=tk.E, pady=5)
        
        # Button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
    
    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window"""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        report_window.geometry("800x600")
        
        # Add text widget with scrollbar
        frame = ttk.Frame(report_window, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        scrollbar = ttk.Scrollbar(frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        text_widget = tk.Text(frame, wrap=tk.WORD, yscrollcommand=scrollbar.set)
        text_widget.pack(fill=tk.BOTH, expand=True)
        scrollbar.config(command=text_widget.yview)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # Display debug information
        debug_frame = ttk.LabelFrame(self.main_frame, text="Debug Info", padding=5)
        debug_frame.pack(fill=tk.X, padx=10, pady=5, before=self.button_frame)
        
        ttk.Label(debug_frame, text="If you don't see the START TRIAL button above, please check the 'Begin Practice' section.",
                 font=("Arial", 9), foreground="red").pack(pady=2)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        print(f"Mouse click detected at screen position ({event.x}, {event.y})")
        
        # Always show click visually even if not in experiment
        try:
            # Create a temporary visual flash where the click happened
            flash = self.visual_canvas.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, 
                                                fill="orange", outline="red", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            
            # Update click counter
            self.click_count += 1
            self.visual_canvas.itemconfig(self.click_counter_text, 
                                       text=f"Total Clicks: {self.click_count}")
        except Exception as e:
            print(f"Error creating click flash: {e}")
            
        if not self.experiment_running or self.start_time is None:
            print("Click ignored for scoring (experiment not running)")
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat(),
            "x": event.x,
            "y": event.y
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds (WILL BE SCORED)")
        
        # Visual indicator for the experimenter - must be done on main thread
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline - must be done on main thread
        try:
            self.add_timeline_marker(current_time, "red")
            print(f"Added timeline marker at {current_time:.3f}s")
        except Exception as e:
            print(f"Error adding timeline marker: {e}")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter"""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            self.root.update_idletasks()
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            print(f"Error flashing indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time"""
        # Timeline spans from 400 to 740 pixels horizontally
        # Assuming max duration of 90 seconds for scaling
        max_duration = 90
        x_pos = 400 + min(time_sec / max_duration, 1.0) * 340
        
        # Create marker - make it bigger and more prominent
        try:
            # Create with a border to make it more visible
            marker = self.visual_canvas.create_oval(x_pos-7, 83, x_pos+7, 97, 
                                                fill=color, outline="black", width=1)
            self.timeline_markers.append(marker)
            
            # Add a timestamp label
            label = self.visual_canvas.create_text(x_pos, 77, 
                                           text=f"{time_sec:.1f}s", 
                                           font=("Arial", 7), 
                                           fill="black")
            self.timeline_markers.append(label)
            
            # Flash the timeline area briefly
            flash = self.visual_canvas.create_rectangle(395, 80, 745, 100, 
                                                  outline="red", width=2)
            self.root.after(200, lambda f=flash: self.visual_canvas.delete(f))
            
            return True
        except Exception as e:
            print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline"""
        print("Clearing timeline markers...")
        try:
            for marker in self.timeline_markers:
                self.visual_canvas.delete(marker)
            self.timeline_markers = []
            
            # Add a flash effect to show timeline has been cleared
            flash = self.visual_canvas.create_rectangle(395, 15, 745, 95, outline="blue", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            print("Timeline cleared successfully")
        except Exception as e:
            print(f"Error clearing timeline: {e}")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Reset click counter for new trial
        self.click_count = 0
        self.visual_canvas.itemconfig(self.click_counter_text, text=f"Total Clicks: {self.click_count}")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Verify mouse tracking before starting trial
        verify_msg = "Verifying mouse tracking..."
        print(verify_msg)
        self.status_var.set(verify_msg)
        self.root.update()
        
        # Flash the click test area to draw attention
        self.flash_area_rect = self.visual_canvas.create_rectangle(50, 100, 740, 130, 
                                                              fill="yellow", outline="red", width=3)
        self.flash_area_text = self.visual_canvas.create_text(395, 115, 
                                               text="CLICK HERE to verify mouse tracking is working!", 
                                               font=("Arial", 12, "bold"), fill="red")
        
        # Wait a moment for user to notice
        self.root.after(2000, self.remove_verification_prompt)
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def remove_verification_prompt(self):
        """Remove the verification prompt and continue with trial"""
        try:
            # Delete the flash objects
            self.visual_canvas.delete(self.flash_area_rect)
            self.visual_canvas.delete(self.flash_area_text)
            print("Verification prompt removed, continuing with trial")
        except:
            print("Error removing verification prompt")
    
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                try:
                    self.add_timeline_marker(t_time, "blue")
                except Exception as e:
                    print(f"Error adding tactile marker: {e}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz, shape: {looming_data.shape}")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz, shape: {tactile_data.shape}")
            
            # Convert stereo to mono if needed for playback
            if len(looming_data.shape) > 1 and looming_data.shape[1] > 1:
                print("Converting looming audio from stereo to mono for playback")
                looming_data = np.mean(looming_data, axis=1)
            
            if len(tactile_data.shape) > 1 and tactile_data.shape[1] > 1:
                print("Converting tactile audio from stereo to mono for playback")
                tactile_data = np.mean(tactile_data, axis=1)
                
            # Update status in main thread using after()
            def update_status(msg):
                try:
                    self.status_var.set(msg)
                    self.root.update_idletasks()
                except Exception as e:
                    print(f"Error updating status: {e}")
                    
            self.root.after(0, update_status, "Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                try:
                    sd.play(looming_data, looming_sr, blocking=True)
                    print("Looming audio playback completed")
                except Exception as e:
                    print(f"Error playing looming audio: {e}")
            
            def play_tactile():
                try:
                    sd.play(tactile_data, tactile_sr, blocking=True)
                    print("Tactile audio playback completed")
                except Exception as e:
                    print(f"Error playing tactile audio: {e}")
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            self.root.after(0, update_status, "Practice trial completed - analyzing results...")
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
            
            # Update status in main thread
            def show_error(msg):
                self.status_var.set(msg)
                
            self.root.after(0, show_error, f"Error during practice trial: {str(e)}")
            
        finally:
            self.experiment_running = False
            
            # Re-enable buttons in main thread
            def enable_buttons():
                self.start_button.config(state=tk.NORMAL)
                self.try_again_button.config(state=tk.NORMAL)
                
            self.root.after(0, enable_buttons)
            self.save_log()
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback"""
        if not tactile_times.size:
            return
            
        try:
            while self.experiment_running:
                current_time = time.perf_counter() - self.start_time
                
                # Check if we're at a tactile stimulus time
                for t_time in tactile_times:
                    if abs(current_time - t_time) < 0.1:  # Within 100ms window
                        # Use after() to update the UI from the main thread
                        self.root.after(0, lambda: self.flash_indicator(self.tactile_indicator, "yellow"))
                        print(f"Tactile event detected at {current_time:.3f}s (expected {t_time:.3f}s)")
                        break
                        
                time.sleep(0.05)  # Check frequently
        except Exception as e:
            print(f"Error in tactile monitor: {e}")
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        # Detailed result tracking
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Lists to store detailed diagnostics
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click against tactile stimuli
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            
            # Find closest tactile stimulus that occurred before this click
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                print(f"  Tactile {i+1} at {stim_time:.3f}s: diff = {time_diff:.3f}s")
                
                # Only consider stimuli that happened before the click and within window
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > 1.0s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within 1 second")
                missed_stimuli += 1
                missed_details.append({
                    'tactile_time': tactile_times[i]
                })
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        # Create detailed diagnostic report
        diagnostic_report = "===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        
        # Correct responses
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
        else:
            diagnostic_report += "  None\n"
        
        # False alarms
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                else:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
        else:
            diagnostic_report += "  None\n"
        
        # Missed stimuli
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within 1 second\n"
        else:
            diagnostic_report += "  None\n"
        
        # Conclusion
        diagnostic_report += f"\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        # Save diagnostic report for later viewing
        self.diagnostic_report = diagnostic_report
        
        # Update UI in the main thread
        def update_ui():
            try:
                # Update results display with basic summary
                results_text = (
                    f"Practice Results:\n\n"
                    f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Missed stimuli: {missed_stimuli}\n\n"
                )
                
                if self.practice_passed:
                    results_text += "PASSED! You understood the task correctly."
                    self.status_var.set("Practice passed successfully!")
                    
                    # Enable and highlight continue button
                    self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
                else:
                    results_text += "FAILED. Please try again and remember:\n"\
                                   "- Click ONLY when you hear a tactile stimulus\n"\
                                   "- You must click within 1 second after the stimulus\n"\
                                   "- Don't click for looming sounds without tactile stimuli"
                    self.status_var.set("Practice failed - try again")
                
                self.results_var.set(results_text)
                
                # Show Try Again button instead of Start
                self.start_button.pack_forget()
                self.try_again_button.pack()
                
                # Enable diagnostic button
                self.show_diagnostic_button.config(state=tk.NORMAL)
            except Exception as e:
                print(f"Error updating UI with results: {e}")
        
        # Schedule UI updates on main thread
        self.root.after(0, update_ui)
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

Loaded previously played iterations: {1, 2, 3, 4, 5}
Mouse click binding established
Mouse click detected at screen position (143, 20)
Click ignored for scoring (experiment not running)

===== STARTING PRACTICE TRIAL =====
Selected iteration 5
Verifying mouse tracking...
Clearing timeline markers...
Timeline cleared successfully
Looking for files:
- Looming: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_5_looming.wav
- Tactile: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_5_tactile.wav
- Log: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_5_log.csv
Loaded trial log with 12 trials
    trial_number trial_type  tactile_time_seconds
0              1      catch              2.200000
1              2   baseline              8.856229
2              3   baseline             15.712500
3              4      catch             22.768750
4              5      

In [4]:
def update_progress_line(self, elapsed_time):
        """Update the position of the progress line based on elapsed time"""
        try:
            # Calculate position
            timeline_start_x = 400
            timeline_width = self.timeline_end_x - timeline_start_x
            max_duration = 90  # Maximum duration in seconds
            
            x_pos = timeline_start_x + min(elapsed_time / max_duration, 1.0) * timeline_width
            
            # Update line position
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y-30, 
                x_pos, self.timeline_y+30
            )
            return True
        except Exception as e:
            print(f"Error updating progress line: {e}")
            return False
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        self.click_count = 0
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Make window full screen from the start
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Bind mouse clicks for the entire application - do this FIRST
        self.root.bind("<Button-1>", self.on_mouse_click)
        print("Mouse click binding established")
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators - make it wider to extend timeline
        self.visual_canvas = tk.Canvas(visual_frame, width=screen_width-100, height=180, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Timeline - make it span almost the entire width
        timeline_start_x = 400
        self.timeline_end_x = screen_width - 120
        self.timeline_y = 90
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y, self.timeline_end_x, self.timeline_y, width=2)
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y-30, self.timeline_end_x, self.timeline_y-30, width=2)
        
        # Add labels to the timeline
        self.visual_canvas.create_text(timeline_start_x-30, self.timeline_y, text="Timeline:", font=("Arial", 10, "bold"))
        
        # Add time markers every 10 seconds
        for sec in range(0, 91, 10):
            x_pos = timeline_start_x + ((self.timeline_end_x - timeline_start_x) * sec / 90)
            self.visual_canvas.create_line(x_pos, self.timeline_y-5, x_pos, self.timeline_y+5, width=1)
            self.visual_canvas.create_text(x_pos, self.timeline_y+15, text=f"{sec}s", font=("Arial", 8))
        
        # Create progress indicator (vertical line) - initially hidden
        self.progress_line = self.visual_canvas.create_line(
            timeline_start_x, self.timeline_y-30, timeline_start_x, self.timeline_y+30, 
            width=2, fill="green", dash=(4, 4), state="hidden")
        
        # Create large click test area (square)
        test_area_size = 300  # much larger square
        test_area_x = (screen_width - 100) // 2 - test_area_size // 2
        test_area_y = 110
        self.visual_canvas.create_rectangle(
            test_area_x, test_area_y, 
            test_area_x + test_area_size, test_area_y + test_area_size, 
            fill="lightyellow", outline="black")
        self.visual_canvas.create_text(
            test_area_x + test_area_size//2, test_area_y + test_area_size//2, 
            text="Click Test Area\n(click here to verify mouse tracking)", 
            font=("Arial", 12), justify="center")
        
        # Add click counter display
        self.click_counter_text = self.visual_canvas.create_text(650, 75, 
                                                            text="Total Clicks: 0", 
                                                            font=("Arial", 12, "bold"),
                                                            fill="blue")
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 12, "bold"))
        
        # Create a distinctive frame just for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=10)
        start_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Start Trial button - large and prominent
        self.start_button = ttk.Button(start_frame, text="START TRIAL", 
                                  command=self.start_trial, width=30, style="Start.TButton")
        self.start_button.pack(pady=10, padx=10)
        
        # Try Again button (initially hidden)
        self.try_again_button = ttk.Button(start_frame, text="TRY AGAIN", 
                                     command=self.start_trial, width=30, style="Start.TButton")
        self.try_again_button.pack(pady=10, padx=10)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(self.results_frame, text="Show Detailed Diagnostics", 
                                          command=self.show_diagnostic_report, state=tk.DISABLED)
        self.show_diagnostic_button.pack(anchor=tk.E, pady=5)
        
        # Button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
    
    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window"""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        report_window.geometry("800x600")
        
        # Add text widget with scrollbar
        frame = ttk.Frame(report_window, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        scrollbar = ttk.Scrollbar(frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        text_widget = tk.Text(frame, wrap=tk.WORD, yscrollcommand=scrollbar.set)
        text_widget.pack(fill=tk.BOTH, expand=True)
        scrollbar.config(command=text_widget.yview)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # Display debug information
        debug_frame = ttk.LabelFrame(self.main_frame, text="Debug Info", padding=5)
        debug_frame.pack(fill=tk.X, padx=10, pady=5, before=self.button_frame)
        
        ttk.Label(debug_frame, text="If you don't see the START TRIAL button above, please check the 'Begin Practice' section.",
                 font=("Arial", 9), foreground="red").pack(pady=2)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        print(f"Mouse click detected at screen position ({event.x}, {event.y})")
        
        # Always show click visually even if not in experiment
        try:
            # Create a temporary visual flash where the click happened
            flash = self.visual_canvas.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, 
                                                fill="orange", outline="red", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            
            # Update click counter
            self.click_count += 1
            self.visual_canvas.itemconfig(self.click_counter_text, 
                                       text=f"Total Clicks: {self.click_count}")
        except Exception as e:
            print(f"Error creating click flash: {e}")
            
        if not self.experiment_running or self.start_time is None:
            print("Click ignored for scoring (experiment not running)")
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat(),
            "x": event.x,
            "y": event.y
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds (WILL BE SCORED)")
        
        # Visual indicator for the experimenter - must be done on main thread
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline - must be done on main thread
        try:
            self.add_timeline_marker(current_time, "red")
            print(f"Added timeline marker at {current_time:.3f}s")
        except Exception as e:
            print(f"Error adding timeline marker: {e}")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter"""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            self.root.update_idletasks()
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            print(f"Error flashing indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time"""
        # Timeline now spans from timeline_start_x to timeline_end_x
        timeline_start_x = 400
        timeline_width = self.timeline_end_x - timeline_start_x
        
        # Calculate position based on time
        max_duration = 90  # Maximum duration in seconds
        x_pos = timeline_start_x + min(time_sec / max_duration, 1.0) * timeline_width
        
        # Create marker - make it bigger and more prominent
        try:
            # Create with a border to make it more visible
            marker = self.visual_canvas.create_oval(x_pos-7, self.timeline_y-7, x_pos+7, self.timeline_y+7, 
                                                fill=color, outline="black", width=1)
            self.timeline_markers.append(marker)
            
            # Flash the timeline area briefly
            flash = self.visual_canvas.create_rectangle(
                x_pos-15, self.timeline_y-15, x_pos+15, self.timeline_y+15, 
                outline="red", width=2)
            self.root.after(200, lambda f=flash: self.visual_canvas.delete(f))
            
            # Update progress line
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y-30, 
                x_pos, self.timeline_y+30
            )
            
            return True
        except Exception as e:
            print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline"""
        print("Clearing timeline markers...")
        try:
            for marker in self.timeline_markers:
                self.visual_canvas.delete(marker)
            self.timeline_markers = []
            
            # Add a flash effect to show timeline has been cleared
            flash = self.visual_canvas.create_rectangle(395, 15, 745, 95, outline="blue", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            print("Timeline cleared successfully")
        except Exception as e:
            print(f"Error clearing timeline: {e}")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Reset click counter for new trial
        self.click_count = 0
        self.visual_canvas.itemconfig(self.click_counter_text, text=f"Total Clicks: {self.click_count}")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Hide progress line initially
        self.visual_canvas.itemconfig(self.progress_line, state="hidden")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Verify mouse tracking before starting trial
        verify_msg = "Verifying mouse tracking..."
        print(verify_msg)
        self.status_var.set(verify_msg)
        self.root.update()
        
        # Get the click test area coordinates
        test_area_size = 300
        test_area_x = (self.root.winfo_width() - 100) // 2 - test_area_size // 2
        test_area_y = 110
        
        # Flash the click test area to draw attention
        self.flash_area_rect = self.visual_canvas.create_rectangle(
            test_area_x-5, test_area_y-5, 
            test_area_x + test_area_size+5, test_area_y + test_area_size+5, 
            fill="yellow", outline="red", width=3)
        self.flash_area_text = self.visual_canvas.create_text(
            test_area_x + test_area_size//2, test_area_y + test_area_size//2, 
            text="CLICK HERE TO VERIFY MOUSE TRACKING IS WORKING!", 
            font=("Arial", 16, "bold"), fill="red")
        
        # Wait a moment for user to notice
        self.root.after(2000, self.remove_verification_prompt)
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def remove_verification_prompt(self):
        """Remove the verification prompt and continue with trial"""
        try:
            # Delete the flash objects
            self.visual_canvas.delete(self.flash_area_rect)
            self.visual_canvas.delete(self.flash_area_text)
            print("Verification prompt removed, continuing with trial")
        except:
            print("Error removing verification prompt")
    
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                try:
                    self.add_timeline_marker(t_time, "blue")
                except Exception as e:
                    print(f"Error adding tactile marker: {e}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz, shape: {looming_data.shape}")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz, shape: {tactile_data.shape}")
            
            # Convert stereo to mono if needed for playback
            if len(looming_data.shape) > 1 and looming_data.shape[1] > 1:
                print("Converting looming audio from stereo to mono for playback")
                looming_data = np.mean(looming_data, axis=1)
            
            if len(tactile_data.shape) > 1 and tactile_data.shape[1] > 1:
                print("Converting tactile audio from stereo to mono for playback")
                tactile_data = np.mean(tactile_data, axis=1)
                
            # Update status in main thread using after()
            def update_status(msg):
                try:
                    self.status_var.set(msg)
                    self.root.update_idletasks()
                except Exception as e:
                    print(f"Error updating status: {e}")
                    
            self.root.after(0, update_status, "Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                try:
                    sd.play(looming_data, looming_sr, blocking=True)
                    print("Looming audio playback completed")
                except Exception as e:
                    print(f"Error playing looming audio: {e}")
            
            def play_tactile():
                try:
                    sd.play(tactile_data, tactile_sr, blocking=True)
                    print("Tactile audio playback completed")
                except Exception as e:
                    print(f"Error playing tactile audio: {e}")
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                
                # Update progress line every 100ms
                self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                
                # Update status text
                self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            self.root.after(0, update_status, "Practice trial completed - analyzing results...")
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
            
            # Update status in main thread
            def show_error(msg):
                self.status_var.set(msg)
                
            self.root.after(0, show_error, f"Error during practice trial: {str(e)}")
            
        finally:
            self.experiment_running = False
            
            # Re-enable buttons in main thread
            def enable_buttons():
                self.start_button.config(state=tk.NORMAL)
                self.try_again_button.config(state=tk.NORMAL)
                
            self.root.after(0, enable_buttons)
            self.save_log()
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback"""
        if not tactile_times.size:
            return
            
        try:
            while self.experiment_running:
                current_time = time.perf_counter() - self.start_time
                
                # Check if we're at a tactile stimulus time
                for t_time in tactile_times:
                    if abs(current_time - t_time) < 0.1:  # Within 100ms window
                        # Use after() to update the UI from the main thread
                        self.root.after(0, lambda: self.flash_indicator(self.tactile_indicator, "yellow"))
                        print(f"Tactile event detected at {current_time:.3f}s (expected {t_time:.3f}s)")
                        break
                        
                time.sleep(0.05)  # Check frequently
        except Exception as e:
            print(f"Error in tactile monitor: {e}")
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        # Detailed result tracking
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Lists to store detailed diagnostics
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click against tactile stimuli
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            
            # Find closest tactile stimulus that occurred before this click
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                print(f"  Tactile {i+1} at {stim_time:.3f}s: diff = {time_diff:.3f}s")
                
                # Only consider stimuli that happened before the click and within window
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > 1.0s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within 1 second")
                missed_stimuli += 1
                missed_details.append({
                    'tactile_time': tactile_times[i]
                })
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        # Create detailed diagnostic report
        diagnostic_report = "===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        
        # Correct responses
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
        else:
            diagnostic_report += "  None\n"
        
        # False alarms
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                else:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
        else:
            diagnostic_report += "  None\n"
        
        # Missed stimuli
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within 1 second\n"
        else:
            diagnostic_report += "  None\n"
        
        # Conclusion
        diagnostic_report += f"\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        # Save diagnostic report for later viewing
        self.diagnostic_report = diagnostic_report
        
        # Update UI in the main thread
        def update_ui():
            try:
                # Update results display with basic summary
                results_text = (
                    f"Practice Results:\n\n"
                    f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Missed stimuli: {missed_stimuli}\n\n"
                )
                
                if self.practice_passed:
                    results_text += "PASSED! You understood the task correctly."
                    self.status_var.set("Practice passed successfully!")
                    
                    # Enable and highlight continue button
                    self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
                else:
                    results_text += "FAILED. Please try again and remember:\n"\
                                   "- Click ONLY when you hear a tactile stimulus\n"\
                                   "- You must click within 1 second after the stimulus\n"\
                                   "- Don't click for looming sounds without tactile stimuli"
                    self.status_var.set("Practice failed - try again")
                
                self.results_var.set(results_text)
                
                # Show Try Again button instead of Start
                self.start_button.pack_forget()
                self.try_again_button.pack()
                
                # Enable diagnostic button
                self.show_diagnostic_button.config(state=tk.NORMAL)
            except Exception as e:
                print(f"Error updating UI with results: {e}")
        
        # Schedule UI updates on main thread
        self.root.after(0, update_ui)
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

Loaded previously played iterations: {3}
Mouse click binding established
Mouse click detected at screen position (653, 166)
Click ignored for scoring (experiment not running)
Mouse click detected at screen position (663, 156)
Click ignored for scoring (experiment not running)
Mouse click detected at screen position (764, 144)
Click ignored for scoring (experiment not running)
Mouse click detected at screen position (708, 131)
Click ignored for scoring (experiment not running)
Mouse click detected at screen position (639, 139)
Click ignored for scoring (experiment not running)
Mouse click detected at screen position (614, 152)
Click ignored for scoring (experiment not running)
Mouse click detected at screen position (631, 159)
Click ignored for scoring (experiment not running)
Mouse click detected at screen position (123, 3)
Click ignored for scoring (experiment not running)

===== STARTING PRACTICE TRIAL =====
Selected iteration 5
Verifying mouse tracking...
Clearing timeline markers..

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\ipykernel_15064\1376450973.py", line 621, in <lambda>
    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'PracticeTrialRunner' object has no attribute 'update_progress_line'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\i

Mouse click detected at screen position (1192, 145)
Mouse click at 1.040 seconds (WILL BE SCORED)
Added timeline marker at 1.040s


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\ipykernel_15064\1376450973.py", line 621, in <lambda>
    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'PracticeTrialRunner' object has no attribute 'update_progress_line'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\i

Mouse click detected at screen position (1282, 163)
Mouse click at 1.691 seconds (WILL BE SCORED)
Added timeline marker at 1.691s
Verification prompt removed, continuing with trial


...

Tactile event detected at 57.435s (expected 57.498s)
Tactile event detected at 57.486s (expected 57.498s)
Tactile event detected at 57.537s (expected 57.498s)
Tactile event detected at 57.589s (expected 57.498s)


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\ipykernel_15064\1376450973.py", line 621, in <lambda>
    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'PracticeTrialRunner' object has no attribute 'update_progress_line'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\i

Tactile event detected at 63.202s (expected 63.267s)
Tactile event detected at 63.252s (expected 63.267s)
Tactile event detected at 63.304s (expected 63.267s)
Tactile event detected at 63.354s (expected 63.267s)


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\ipykernel_15064\1376450973.py", line 621, in <lambda>
    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'PracticeTrialRunner' object has no attribute 'update_progress_line'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\i

Looming audio playback completed
Tactile audio playback completed
Audio playback completed at 16:47:25.493254

===== ANALYZING PRACTICE RESULTS =====
Tactile stimuli at: [ 8.85622917 15.7125     30.385      36.93122917 42.73747917 49.79375
 57.498      63.26725   ]
Mouse clicks at: [1.0399518000012904, 1.690908300002775, 1.998044200001459, 2.2977150000006077, 3.0180274000013014, 3.6727354000031482, 5.44662960000278, 6.057360700000572, 6.566034100000252, 7.008246600002167, 9.26977330000227, 14.779337300002226, 15.259761200002686, 15.831248400001641, 16.12773020000168, 16.341340099999798, 16.53126230000271, 16.750139499999932, 17.101498800002446, 17.330353500001365, 17.505173400000785, 17.713814900002035, 17.915620600000693, 18.129453100002138, 18.45110950000162, 18.71004670000184, 18.95943150000312, 19.19825050000145, 19.41783370000121, 19.638474600000336, 26.781526400001894, 27.239019100001315, 27.545862200000556, 27.79173720000108, 28.00854380000237, 28.218195800000103, 28.41148340000

In [ ]:
def update_progress_line(self, elapsed_time):
        """Update the position of the progress line based on elapsed time"""
        try:
            # Calculate position
            timeline_start_x = 400
            timeline_width = self.timeline_end_x - timeline_start_x
            max_duration = 90  # Maximum duration in seconds
            
            x_pos = timeline_start_x + min(elapsed_time / max_duration, 1.0) * timeline_width
            
            # Update line position
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y-30, 
                x_pos, self.timeline_y+30
            )
            return True
        except Exception as e:
            print(f"Error updating progress line: {e}")
            return False
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        self.click_count = 0
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()
    
    def load_played_iterations(self):
        """Load list of previously played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations"""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI"""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Make window full screen from the start
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Bind mouse clicks for the entire application - do this FIRST
        self.root.bind("<Button-1>", self.on_mouse_click)
        print("Mouse click binding established")
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                 font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                 font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators - make it wider to extend timeline
        self.visual_canvas = tk.Canvas(visual_frame, width=screen_width-100, height=120, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Add click counter display
        self.click_counter_text = self.visual_canvas.create_text(650, 75, 
                                                            text="Total Clicks: 0", 
                                                            font=("Arial", 12, "bold"),
                                                            fill="blue")
        
        # Timeline - make it span almost the entire width
        timeline_start_x = 400
        self.timeline_end_x = screen_width - 120
        self.timeline_y = 90
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y, self.timeline_end_x, self.timeline_y, width=2)
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y-30, self.timeline_end_x, self.timeline_y-30, width=2)
        
        # Add labels to the timeline
        self.visual_canvas.create_text(timeline_start_x-30, self.timeline_y, text="Timeline:", font=("Arial", 10, "bold"))
        
        # Add time markers every 10 seconds
        for sec in range(0, 91, 10):
            x_pos = timeline_start_x + ((self.timeline_end_x - timeline_start_x) * sec / 90)
            self.visual_canvas.create_line(x_pos, self.timeline_y-5, x_pos, self.timeline_y+5, width=1)
            self.visual_canvas.create_text(x_pos, self.timeline_y+15, text=f"{sec}s", font=("Arial", 8))
        
        # Create progress indicator (vertical line) - represents current playback position
        self.progress_line = self.visual_canvas.create_line(
            timeline_start_x, self.timeline_y-30, timeline_start_x, self.timeline_y+30, 
            width=3, fill="green", state="hidden")
            
        # Create a separate frame for the click test area
        click_test_frame = ttk.LabelFrame(self.main_frame, text="Click Test Area", padding=10)
        click_test_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create a canvas for the click test area
        test_area_height = 250
        self.click_test_canvas = tk.Canvas(click_test_frame, width=screen_width-100, height=test_area_height, bg="lightyellow")
        self.click_test_canvas.pack(fill=tk.X, pady=5)
        
        # Add text to the click test area
        self.click_test_canvas.create_text(
            (screen_width-100)//2, test_area_height//2,
            text="CLICK ANYWHERE IN THIS AREA TO VERIFY MOUSE TRACKING",
            font=("Arial", 18, "bold"), fill="blue")
            
        # Add a visual indicator for clicks in the test area
        self.click_feedback_circle = self.click_test_canvas.create_oval(
            (screen_width-100)//2 - 50, test_area_height//2 - 50,
            (screen_width-100)//2 + 50, test_area_height//2 + 50,
            outline="gray", width=2, fill="white")
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                 font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 12, "bold"))
        
        # Create a distinctive frame just for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=10)
        start_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Start Trial button - large and prominent
        self.start_button = ttk.Button(start_frame, text="START TRIAL", 
                                  command=self.start_trial, width=30, style="Start.TButton")
        self.start_button.pack(pady=10, padx=10)
        
        # Try Again button (initially hidden)
        self.try_again_button = ttk.Button(start_frame, text="TRY AGAIN", 
                                     command=self.start_trial, width=30, style="Start.TButton")
        self.try_again_button.pack(pady=10, padx=10)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(self.results_frame, text="Show Detailed Diagnostics", 
                                          command=self.show_diagnostic_report, state=tk.DISABLED)
        self.show_diagnostic_button.pack(anchor=tk.E, pady=5)
        
        # Button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
    
    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window"""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        report_window.geometry("800x600")
        
        # Add text widget with scrollbar
        frame = ttk.Frame(report_window, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        scrollbar = ttk.Scrollbar(frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        text_widget = tk.Text(frame, wrap=tk.WORD, yscrollcommand=scrollbar.set)
        text_widget.pack(fill=tk.BOTH, expand=True)
        scrollbar.config(command=text_widget.yview)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment", 
                                     state=tk.DISABLED, style="Red.TButton", width=20,
                                     command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(self.button_frame, text="Cancel", 
                                   command=self.on_close, width=15)
        self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # Display debug information
        debug_frame = ttk.LabelFrame(self.main_frame, text="Debug Info", padding=5)
        debug_frame.pack(fill=tk.X, padx=10, pady=5, before=self.button_frame)
        
        ttk.Label(debug_frame, text="If you don't see the START TRIAL button above, please check the 'Begin Practice' section.",
                 font=("Arial", 9), foreground="red").pack(pady=2)
        
        # Bind mouse clicks for the entire window
        self.root.bind("<Button-1>", self.on_mouse_click)
    
    def on_close(self):
        """Handle window close event"""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment"""
        print(f"Mouse click detected at screen position ({event.x}, {event.y})")
        
        # Show visual feedback on click test canvas if it exists
        try:
            if hasattr(self, 'click_test_canvas') and self.click_test_canvas.winfo_exists():
                # Check if click is within the click test canvas
                test_canvas_x = self.click_test_canvas.winfo_rootx()
                test_canvas_y = self.click_test_canvas.winfo_rooty()
                test_canvas_width = self.click_test_canvas.winfo_width()
                test_canvas_height = self.click_test_canvas.winfo_height()
                
                # If click is within the test canvas
                if (test_canvas_x <= event.x_root <= test_canvas_x + test_canvas_width and
                    test_canvas_y <= event.y_root <= test_canvas_y + test_canvas_height):
                    # Convert to canvas coordinates
                    canvas_x = event.x_root - test_canvas_x
                    canvas_y = event.y_root - test_canvas_y
                    
                    # Move the feedback circle to click position
                    self.click_test_canvas.coords(
                        self.click_feedback_circle,
                        canvas_x - 25, canvas_y - 25,
                        canvas_x + 25, canvas_y + 25
                    )
                    # Change color to indicate click
                    self.click_test_canvas.itemconfig(self.click_feedback_circle, fill="orange", outline="red")
                    # Reset after a short delay
                    self.root.after(300, lambda: self.click_test_canvas.itemconfig(
                        self.click_feedback_circle, fill="white", outline="gray"))
        except Exception as e:
            print(f"Error updating click test feedback: {e}")
        
        # Always show click visually on the main canvas
        try:
            if event.widget == self.visual_canvas:
                # Create a temporary visual flash where the click happened
                flash = self.visual_canvas.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, 
                                                    fill="orange", outline="red", width=2)
                self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
        except Exception as e:
            print(f"Error creating click flash: {e}")
            
        if not self.experiment_running or self.start_time is None:
            print("Click ignored for scoring (experiment not running)")
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat(),
            "x": event.x,
            "y": event.y
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds (WILL BE SCORED)")
        
        # Visual indicator for the experimenter - must be done on main thread
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline - must be done on main thread
        try:
            self.add_timeline_marker(current_time, "red")
            print(f"Added timeline marker at {current_time:.3f}s")
        except Exception as e:
            print(f"Error adding timeline marker: {e}")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter"""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            self.root.update_idletasks()
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            print(f"Error flashing indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time"""
        # Timeline now spans from timeline_start_x to timeline_end_x
        timeline_start_x = 400
        timeline_width = self.timeline_end_x - timeline_start_x
        
        # Calculate position based on time
        max_duration = 90  # Maximum duration in seconds
        x_pos = timeline_start_x + min(time_sec / max_duration, 1.0) * timeline_width
        
        # Create marker - make it bigger and more prominent
        try:
            # Create with a border to make it more visible
            marker = self.visual_canvas.create_oval(x_pos-7, self.timeline_y-7, x_pos+7, self.timeline_y+7, 
                                                fill=color, outline="black", width=1)
            self.timeline_markers.append(marker)
            
            # Flash the timeline area briefly
            flash = self.visual_canvas.create_rectangle(
                x_pos-15, self.timeline_y-15, x_pos+15, self.timeline_y+15, 
                outline="red", width=2)
            self.root.after(200, lambda f=flash: self.visual_canvas.delete(f))
            
            # Update progress line
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y-30, 
                x_pos, self.timeline_y+30
            )
            
            return True
        except Exception as e:
            print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline"""
        print("Clearing timeline markers...")
        try:
            for marker in self.timeline_markers:
                self.visual_canvas.delete(marker)
            self.timeline_markers = []
            
            # Add a flash effect to show timeline has been cleared
            flash = self.visual_canvas.create_rectangle(395, 15, 745, 95, outline="blue", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            print("Timeline cleared successfully")
        except Exception as e:
            print(f"Error clearing timeline: {e}")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet"""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial"""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Reset click counter for new trial
        self.click_count = 0
        self.visual_canvas.itemconfig(self.click_counter_text, text=f"Total Clicks: {self.click_count}")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Hide progress line initially
        self.visual_canvas.itemconfig(self.progress_line, state="hidden")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Verify mouse tracking before starting trial
        verify_msg = "Verifying mouse tracking..."
        print(verify_msg)
        self.status_var.set(verify_msg)
        self.root.update()
        
        # Flash the click test canvas to draw attention
        test_canvas_width = self.click_test_canvas.winfo_width()
        test_canvas_height = self.click_test_canvas.winfo_height()
        
        # Add a flashing rectangle over the entire test canvas
        self.flash_area_rect = self.click_test_canvas.create_rectangle(
            5, 5, test_canvas_width-5, test_canvas_height-5,
            outline="red", width=5)
            
        # Add text overlay
        self.flash_area_text = self.click_test_canvas.create_text(
            test_canvas_width//2, test_canvas_height//2,
            text="CLICK ANYWHERE TO VERIFY MOUSE TRACKING!",
            font=("Arial", 24, "bold"), fill="red")
        
        # Wait a moment for user to notice
        self.root.after(2000, self.remove_verification_prompt)
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def remove_verification_prompt(self):
        """Remove the verification prompt and continue with trial"""
        try:
            # Delete the flash objects
            self.click_test_canvas.delete(self.flash_area_rect)
            self.click_test_canvas.delete(self.flash_area_text)
            print("Verification prompt removed, continuing with trial")
        except Exception as e:
            print(f"Error removing verification prompt: {e}")
    
    def run_trial(self):
        """Run the actual practice trial"""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                try:
                    self.add_timeline_marker(t_time, "blue")
                except Exception as e:
                    print(f"Error adding tactile marker: {e}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz, shape: {looming_data.shape}")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz, shape: {tactile_data.shape}")
            
            # Convert stereo to mono if needed for playback
            if len(looming_data.shape) > 1 and looming_data.shape[1] > 1:
                print("Converting looming audio from stereo to mono for playback")
                looming_data = np.mean(looming_data, axis=1)
            
            if len(tactile_data.shape) > 1 and tactile_data.shape[1] > 1:
                print("Converting tactile audio from stereo to mono for playback")
                tactile_data = np.mean(tactile_data, axis=1)
                
            # Update status in main thread using after()
            def update_status(msg):
                try:
                    self.status_var.set(msg)
                    self.root.update_idletasks()
                except Exception as e:
                    print(f"Error updating status: {e}")
                    
            self.root.after(0, update_status, "Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Reset progress line to start position
            self.update_progress_line(0)
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                try:
                    sd.play(looming_data, looming_sr, blocking=True)
                    print("Looming audio playback completed")
                except Exception as e:
                    print(f"Error playing looming audio: {e}")
            
            def play_tactile():
                try:
                    sd.play(tactile_data, tactile_sr, blocking=True)
                    print("Tactile audio playback completed")
                except Exception as e:
                    print(f"Error playing tactile audio: {e}")
            
            # Create and start threads
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish (use the longer of the two)
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            
            # Add a little buffer to ensure playback completes
            end_time = self.start_time + max_duration + 1.0
            
            # Start a progress update thread
            progress_update_thread = threading.Thread(
                target=self.continuous_progress_update,
                args=(max_duration,),
                daemon=True
            )
            progress_update_thread.start()
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            self.root.after(0, update_status, "Practice trial completed - analyzing results...")
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
            
            # Update status in main thread
            def show_error(msg):
                self.status_var.set(msg)
                
            self.root.after(0, show_error, f"Error during practice trial: {str(e)}")
            
        finally:
            self.experiment_running = False
            
            # Re-enable buttons in main thread
            def enable_buttons():
                self.start_button.config(state=tk.NORMAL)
                self.try_again_button.config(state=tk.NORMAL)
                
            self.root.after(0, enable_buttons)
            self.save_log()
            
    def continuous_progress_update(self, max_duration):
        """Continuously update the progress line during experiment"""
        try:
            update_interval = 0.05  # Update every 50ms for smooth movement
            
            while self.experiment_running:
                if self.start_time is not None:
                    # Calculate elapsed time
                    elapsed = time.perf_counter() - self.start_time
                    
                    # Stop if we've reached the end
                    if elapsed > max_duration:
                        break
                        
                    # Schedule an update to the progress line
                    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                    
                    # Small sleep to avoid excessive updates
                    time.sleep(update_interval)
                else:
                    time.sleep(0.1)
                    
            print("Progress update thread completed")
        except Exception as e:
            print(f"Error in progress update thread: {e}")
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback"""
        if not tactile_times.size:
            return
            
        try:
            while self.experiment_running:
                current_time = time.perf_counter() - self.start_time
                
                # Check if we're at a tactile stimulus time
                for t_time in tactile_times:
                    if abs(current_time - t_time) < 0.1:  # Within 100ms window
                        # Use after() to update the UI from the main thread
                        self.root.after(0, lambda: self.flash_indicator(self.tactile_indicator, "yellow"))
                        print(f"Tactile event detected at {current_time:.3f}s (expected {t_time:.3f}s)")
                        break
                        
                time.sleep(0.05)  # Check frequently
        except Exception as e:
            print(f"Error in tactile monitor: {e}")
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial"""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        # Detailed result tracking
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Lists to store detailed diagnostics
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        # Initialize arrays to track which stimuli were responded to
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click against tactile stimuli
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            
            # Find closest tactile stimulus that occurred before this click
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                print(f"  Tactile {i+1} at {stim_time:.3f}s: diff = {time_diff:.3f}s")
                
                # Only consider stimuli that happened before the click and within window
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Check if click was within 1 second of a tactile stimulus
            if closest_idx >= 0 and closest_diff <= 1.0:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > 1.0s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within 1 second")
                missed_stimuli += 1
                missed_details.append({
                    'tactile_time': tactile_times[i]
                })
        
        # Calculate total expected tactile stimuli
        total_tactile = len(tactile_times)
        
        # Calculate score
        if total_tactile > 0:
            hit_rate = correct_clicks / total_tactile
        else:
            hit_rate = 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        # Determine if practice passed (criteria: at least 80% hit rate, no more than 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        # Create detailed diagnostic report
        diagnostic_report = "===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        
        # Correct responses
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
        else:
            diagnostic_report += "  None\n"
        
        # False alarms
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                else:
                    diagnostic_report += f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
        else:
            diagnostic_report += "  None\n"
        
        # Missed stimuli
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within 1 second\n"
        else:
            diagnostic_report += "  None\n"
        
        # Conclusion
        diagnostic_report += f"\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        # Save diagnostic report for later viewing
        self.diagnostic_report = diagnostic_report
        
        # Update UI in the main thread
        def update_ui():
            try:
                # Update results display with basic summary
                results_text = (
                    f"Practice Results:\n\n"
                    f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Missed stimuli: {missed_stimuli}\n\n"
                )
                
                if self.practice_passed:
                    results_text += "PASSED! You understood the task correctly."
                    self.status_var.set("Practice passed successfully!")
                    
                    # Enable and highlight continue button
                    self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
                else:
                    results_text += "FAILED. Please try again and remember:\n"\
                                   "- Click ONLY when you hear a tactile stimulus\n"\
                                   "- You must click within 1 second after the stimulus\n"\
                                   "- Don't click for looming sounds without tactile stimuli"
                    self.status_var.set("Practice failed - try again")
                
                self.results_var.set(results_text)
                
                # Show Try Again button instead of Start
                self.start_button.pack_forget()
                self.try_again_button.pack()
                
                # Enable diagnostic button
                self.show_diagnostic_button.config(state=tk.NORMAL)
            except Exception as e:
                print(f"Error updating UI with results: {e}")
        
        # Schedule UI updates on main thread
        self.root.after(0, update_ui)
    
    def save_log(self):
        """Save the mouse click log to a CSV file"""
        if not self.mouse_clicks:
            return
            
        try:
            # Convert to DataFrame
            df = pd.DataFrame(self.mouse_clicks)
            
            # Add additional info
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            
            # Save to CSV
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
            
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

Loaded previously played iterations: {1, 2, 3, 4, 5}
Mouse click binding established
Error in progress update thread: main thread is not in main loop
ERROR during practice trial: main thread is not in main loop


Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\ipykernel_23440\3629334649.py", line 675, in run_trial
    self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 872, in after
    name = self._register(callit)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1603, in _register
    self.tk.createcommand(name, f)
RuntimeError: main thread is not in main loop
Exception in thread Thread-10 (run_trial):
Traceback (most recent call last):
  File "C:\Users\cogpsy-vrlab\AppData\Local\Temp\ipykernel_23440\3629334649.py", line 675, in run_trial
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 872, in after
    name = self._register(callit)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\cogpsy-vrlab\anaconda3\Lib\tkinter\__init__.py", line 1603, in _register
    self

In [2]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        self.click_count = 0
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()

    def update_progress_line(self, elapsed_time):
        """Update the position of the progress line based on elapsed time."""
        try:
            # Calculate position
            timeline_start_x = 400
            timeline_width = self.timeline_end_x - timeline_start_x
            max_duration = 90  # Maximum duration in seconds
            
            x_pos = timeline_start_x + min(elapsed_time / max_duration, 1.0) * timeline_width
            
            # Update line position
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y - 30, 
                x_pos, self.timeline_y + 30
            )
            return True
        except Exception as e:
            print(f"Error updating progress line: {e}")
            return False
    
    def load_played_iterations(self):
        """Load list of previously played iterations."""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations."""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI."""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Make window full screen from the start
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        
        # Create main frame
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Bind mouse clicks for the entire application - do this FIRST
        self.root.bind("<Button-1>", self.on_mouse_click)
        print("Mouse click binding established")
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                  font=("Arial", 18, "bold")).pack(pady=10)
        
        # Instructions frame
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=10)
        instructions_frame.pack(fill=tk.X, padx=10, pady=10)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                  font=("Arial", 11), justify="left").pack(anchor=tk.W, pady=5)
        
        # Status frame
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=10, pady=10)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                      font=("Arial", 12, "bold"))
        self.status_label.pack(pady=5)
        
        # Experimenter visualization frame (not shown to participant)
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=10)
        visual_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create canvas for visual indicators - make it wider to extend timeline
        self.visual_canvas = tk.Canvas(visual_frame, width=screen_width - 100, height=120, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=5)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 50, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 35, text="Tactile Stimulus", font=("Arial", 10))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 60, 350, 90, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 75, text="Mouse Click", font=("Arial", 10))
        
        # Add click counter display
        self.click_counter_text = self.visual_canvas.create_text(
            650, 75,
            text="Total Clicks: 0",
            font=("Arial", 12, "bold"),
            fill="blue"
        )
        
        # Timeline - make it span almost the entire width
        timeline_start_x = 400
        self.timeline_end_x = screen_width - 120
        self.timeline_y = 90
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y, self.timeline_end_x, self.timeline_y, width=2)
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y - 30, self.timeline_end_x, self.timeline_y - 30, width=2)
        
        # Add labels to the timeline
        self.visual_canvas.create_text(timeline_start_x - 30, self.timeline_y, text="Timeline:", font=("Arial", 10, "bold"))
        
        # Add time markers every 10 seconds
        for sec in range(0, 91, 10):
            x_pos = timeline_start_x + ((self.timeline_end_x - timeline_start_x) * sec / 90)
            self.visual_canvas.create_line(x_pos, self.timeline_y - 5, x_pos, self.timeline_y + 5, width=1)
            self.visual_canvas.create_text(x_pos, self.timeline_y + 15, text=f"{sec}s", font=("Arial", 8))
        
        # Create progress indicator (vertical line) - represents current playback position
        self.progress_line = self.visual_canvas.create_line(
            timeline_start_x, self.timeline_y - 30, timeline_start_x, self.timeline_y + 30, 
            width=3, fill="green", state="hidden"
        )
            
        # Create a separate frame for the click test area
        click_test_frame = ttk.LabelFrame(self.main_frame, text="Click Test Area", padding=10)
        click_test_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Create a canvas for the click test area
        test_area_height = 250
        self.click_test_canvas = tk.Canvas(click_test_frame, width=screen_width - 100, height=test_area_height, bg="lightyellow")
        self.click_test_canvas.pack(fill=tk.X, pady=5)
        
        # Add text to the click test area
        self.click_test_canvas.create_text(
            (screen_width - 100)//2, test_area_height//2,
            text="CLICK ANYWHERE IN THIS AREA TO VERIFY MOUSE TRACKING",
            font=("Arial", 18, "bold"), fill="blue"
        )
            
        # Add a visual indicator for clicks in the test area
        self.click_feedback_circle = self.click_test_canvas.create_oval(
            (screen_width - 100)//2 - 50, test_area_height//2 - 50,
            (screen_width - 100)//2 + 50, test_area_height//2 + 50,
            outline="gray", width=2, fill="white"
        )
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=10)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                       font=("Arial", 11), justify="left")
        self.results_label.pack(anchor=tk.W, pady=5)
        self.results_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 12, "bold"))
        
        # Create a distinctive frame just for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=10)
        start_frame.pack(fill=tk.X, padx=10, pady=10)
        
        # Start Trial button - large and prominent
        self.start_button = ttk.Button(
            start_frame, text="START TRIAL",
            command=self.start_trial, width=30, style="Start.TButton"
        )
        self.start_button.pack(pady=10, padx=10)
        
        # Try Again button (initially hidden)
        self.try_again_button = ttk.Button(
            start_frame, text="TRY AGAIN", 
            command=self.start_trial, width=30, style="Start.TButton"
        )
        self.try_again_button.pack(pady=10, padx=10)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(
            self.results_frame, text="Show Detailed Diagnostics",
            command=self.show_diagnostic_report, state=tk.DISABLED
        )
        self.show_diagnostic_button.pack(anchor=tk.E, pady=5)
        
        # Button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
        
        # Continue to Experiment button (initially disabled)
        self.continue_button = ttk.Button(
            self.button_frame, text="Continue to Experiment",
            state=tk.DISABLED, style="Red.TButton", width=20,
            command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment...")
        )
        self.continue_button.pack(side=tk.LEFT, padx=10)
        
        # Cancel button
        self.cancel_button = ttk.Button(
            self.button_frame, text="Cancel",
            command=self.on_close, width=15
        )
        self.cancel_button.pack(side=tk.LEFT, padx=10)
    
    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window."""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        report_window.geometry("800x600")
        
        # Add text widget with scrollbar
        frame = ttk.Frame(report_window, padding=10)
        frame.pack(fill=tk.BOTH, expand=True)
        
        scrollbar = ttk.Scrollbar(frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        text_widget = tk.Text(frame, wrap=tk.WORD, yscrollcommand=scrollbar.set)
        text_widget.pack(fill=tk.BOTH, expand=True)
        scrollbar.config(command=text_widget.yview)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
        
        # (Re)create these buttons if needed inside the window – or as placeholders:
        # In your code, these lines appear repeated, possibly an accidental duplication.
        # If needed, they can be commented out or removed.
        #
        # self.continue_button = ttk.Button(self.button_frame, text="Continue to Experiment",
        #                              state=tk.DISABLED, style="Red.TButton", width=20,
        #                              command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment..."))
        # self.continue_button.pack(side=tk.LEFT, padx=10)
        #
        # self.cancel_button = ttk.Button(self.button_frame, text="Cancel",
        #                            command=self.on_close, width=15)
        # self.cancel_button.pack(side=tk.LEFT, padx=10)
        
        # If you want to add any extra debug info, you could do it here:
        # debug_frame = ttk.LabelFrame(self.main_frame, text="Debug Info", padding=5)
        # debug_frame.pack(fill=tk.X, padx=10, pady=5, before=self.button_frame)
        # ttk.Label(debug_frame, text="Some debug info", font=("Arial", 9), foreground="red").pack(pady=2)
    
    def on_close(self):
        """Handle window close event."""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment."""
        print(f"Mouse click detected at screen position ({event.x}, {event.y})")
        
        # Show visual feedback on click test canvas if it exists
        try:
            if hasattr(self, 'click_test_canvas') and self.click_test_canvas.winfo_exists():
                # Check if click is within the click test canvas
                test_canvas_x = self.click_test_canvas.winfo_rootx()
                test_canvas_y = self.click_test_canvas.winfo_rooty()
                test_canvas_width = self.click_test_canvas.winfo_width()
                test_canvas_height = self.click_test_canvas.winfo_height()
                
                # If click is within the test canvas
                if (test_canvas_x <= event.x_root <= test_canvas_x + test_canvas_width and
                    test_canvas_y <= event.y_root <= test_canvas_y + test_canvas_height):
                    # Convert to canvas coordinates
                    canvas_x = event.x_root - test_canvas_x
                    canvas_y = event.y_root - test_canvas_y
                    
                    # Move the feedback circle to click position
                    self.click_test_canvas.coords(
                        self.click_feedback_circle,
                        canvas_x - 25, canvas_y - 25,
                        canvas_x + 25, canvas_y + 25
                    )
                    # Change color to indicate click
                    self.click_test_canvas.itemconfig(self.click_feedback_circle, fill="orange", outline="red")
                    # Reset after a short delay
                    self.root.after(300, lambda: self.click_test_canvas.itemconfig(
                        self.click_feedback_circle, fill="white", outline="gray"))
        except Exception as e:
            print(f"Error updating click test feedback: {e}")
        
        # Always show click visually on the main canvas
        try:
            if event.widget == self.visual_canvas:
                # Create a temporary visual flash where the click happened
                flash = self.visual_canvas.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, 
                                                       fill="orange", outline="red", width=2)
                self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
        except Exception as e:
            print(f"Error creating click flash: {e}")
            
        if not self.experiment_running or self.start_time is None:
            print("Click ignored for scoring (experiment not running)")
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat(),
            "x": event.x,
            "y": event.y
        })
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds (WILL BE SCORED)")
        
        # Visual indicator for the experimenter
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline
        try:
            self.add_timeline_marker(current_time, "red")
            print(f"Added timeline marker at {current_time:.3f}s")
        except Exception as e:
            print(f"Error adding timeline marker: {e}")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter."""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            self.root.update_idletasks()
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            print(f"Error flashing indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time."""
        timeline_start_x = 400
        timeline_width = self.timeline_end_x - timeline_start_x
        
        # Calculate position based on time
        max_duration = 90  # Maximum duration in seconds
        x_pos = timeline_start_x + min(time_sec / max_duration, 1.0) * timeline_width
        
        try:
            # Create marker
            marker = self.visual_canvas.create_oval(
                x_pos-7, self.timeline_y-7, x_pos+7, self.timeline_y+7, 
                fill=color, outline="black", width=1
            )
            self.timeline_markers.append(marker)
            
            # Flash the timeline area briefly
            flash = self.visual_canvas.create_rectangle(
                x_pos-15, self.timeline_y-15, x_pos+15, self.timeline_y+15, 
                outline="red", width=2
            )
            self.root.after(200, lambda f=flash: self.visual_canvas.delete(f))
            
            # Update progress line
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y-30, 
                x_pos, self.timeline_y+30
            )
            
            return True
        except Exception as e:
            print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline."""
        print("Clearing timeline markers...")
        try:
            for marker in self.timeline_markers:
                self.visual_canvas.delete(marker)
            self.timeline_markers = []
            
            # Add a flash effect to show timeline has been cleared
            flash = self.visual_canvas.create_rectangle(395, 15, 745, 95, outline="blue", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            print("Timeline cleared successfully")
        except Exception as e:
            print(f"Error clearing timeline: {e}")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet."""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial."""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Reset click counter for new trial
        self.click_count = 0
        self.visual_canvas.itemconfig(self.click_counter_text, text=f"Total Clicks: {self.click_count}")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Hide progress line initially
        self.visual_canvas.itemconfig(self.progress_line, state="hidden")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Verify mouse tracking before starting trial
        verify_msg = "Verifying mouse tracking..."
        print(verify_msg)
        self.status_var.set(verify_msg)
        self.root.update()
        
        # Flash the click test canvas to draw attention
        test_canvas_width = self.click_test_canvas.winfo_width()
        test_canvas_height = self.click_test_canvas.winfo_height()
        
        # Add a flashing rectangle over the entire test canvas
        self.flash_area_rect = self.click_test_canvas.create_rectangle(
            5, 5, test_canvas_width - 5, test_canvas_height - 5,
            outline="red", width=5
        )
            
        # Add text overlay
        self.flash_area_text = self.click_test_canvas.create_text(
            test_canvas_width//2, test_canvas_height//2,
            text="CLICK ANYWHERE TO VERIFY MOUSE TRACKING!",
            font=("Arial", 24, "bold"), fill="red"
        )
        
        # Wait a moment for user to notice
        self.root.after(2000, self.remove_verification_prompt)
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def remove_verification_prompt(self):
        """Remove the verification prompt and continue with trial."""
        try:
            self.click_test_canvas.delete(self.flash_area_rect)
            self.click_test_canvas.delete(self.flash_area_text)
            print("Verification prompt removed, continuing with trial")
        except Exception as e:
            print(f"Error removing verification prompt: {e}")
    
    def run_trial(self):
        """Run the actual practice trial."""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                try:
                    self.add_timeline_marker(t_time, "blue")
                except Exception as e:
                    print(f"Error adding tactile marker: {e}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz, shape: {looming_data.shape}")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz, shape: {tactile_data.shape}")
            
            # Convert stereo to mono if needed
            if len(looming_data.shape) > 1 and looming_data.shape[1] > 1:
                print("Converting looming audio from stereo to mono for playback")
                looming_data = np.mean(looming_data, axis=1)
            if len(tactile_data.shape) > 1 and tactile_data.shape[1] > 1:
                print("Converting tactile audio from stereo to mono for playback")
                tactile_data = np.mean(tactile_data, axis=1)
            
            def update_status(msg):
                try:
                    self.status_var.set(msg)
                    self.root.update_idletasks()
                except Exception as e:
                    print(f"Error updating status: {e}")
            
            self.root.after(0, update_status, "Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Reset progress line to start position
            self.update_progress_line(0)
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                try:
                    sd.play(looming_data, looming_sr, blocking=True)
                    print("Looming audio playback completed")
                except Exception as e:
                    print(f"Error playing looming audio: {e}")
            
            def play_tactile():
                try:
                    sd.play(tactile_data, tactile_sr, blocking=True)
                    print("Tactile audio playback completed")
                except Exception as e:
                    print(f"Error playing tactile audio: {e}")
            
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            end_time = self.start_time + max_duration + 1.0
            
            # Start a progress update thread
            progress_update_thread = threading.Thread(
                target=self.continuous_progress_update,
                args=(max_duration,),
                daemon=True
            )
            progress_update_thread.start()
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            self.root.after(0, update_status, "Practice trial completed - analyzing results...")
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
            
            def show_error(msg):
                self.status_var.set(msg)
            self.root.after(0, show_error, f"Error during practice trial: {str(e)}")
            
        finally:
            self.experiment_running = False
            
            def enable_buttons():
                self.start_button.config(state=tk.NORMAL)
                self.try_again_button.config(state=tk.NORMAL)
            self.root.after(0, enable_buttons)
            self.save_log()
    
    def continuous_progress_update(self, max_duration):
        """Continuously update the progress line during the experiment."""
        try:
            update_interval = 0.05  # Update every 50ms
            
            while self.experiment_running:
                if self.start_time is not None:
                    elapsed = time.perf_counter() - self.start_time
                    if elapsed > max_duration:
                        break
                    # Schedule an update to the progress line
                    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                    time.sleep(update_interval)
                else:
                    time.sleep(0.1)
            print("Progress update thread completed")
        except Exception as e:
            print(f"Error in progress update thread: {e}")
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback."""
        if not tactile_times.size:
            return
        try:
            while self.experiment_running:
                current_time = time.perf_counter() - self.start_time
                for t_time in tactile_times:
                    if abs(current_time - t_time) < 0.1:  # within 100ms
                        self.root.after(0, lambda: self.flash_indicator(self.tactile_indicator, "yellow"))
                        print(f"Tactile event detected at {current_time:.3f}s (expected {t_time:.3f}s)")
                        break
                time.sleep(0.05)
        except Exception as e:
            print(f"Error in tactile monitor: {e}")
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial."""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            if closest_idx >= 0 and closest_diff <= 1.0:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > 1.0s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within 1 second")
                missed_stimuli += 1
                missed_details.append({'tactile_time': tactile_times[i]})
        
        total_tactile = len(tactile_times)
        hit_rate = (correct_clicks / total_tactile) if total_tactile > 0 else 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        diagnostic_report = "===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += (
                    f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile "
                    f"at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                )
        else:
            diagnostic_report += "  None\n"
        
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += (
                        f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for "
                        f"tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                    )
                else:
                    diagnostic_report += (
                        f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
                    )
        else:
            diagnostic_report += "  None\n"
        
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += (
                    f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within 1 second\n"
                )
        else:
            diagnostic_report += "  None\n"
        
        diagnostic_report += "\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        self.diagnostic_report = diagnostic_report
        
        def update_ui():
            try:
                results_text = (
                    f"Practice Results:\n\n"
                    f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Missed stimuli: {missed_stimuli}\n\n"
                )
                
                if self.practice_passed:
                    results_text += "PASSED! You understood the task correctly."
                    self.status_var.set("Practice passed successfully!")
                    self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
                else:
                    results_text += (
                        "FAILED. Please try again and remember:\n"
                        "- Click ONLY when you hear a tactile stimulus\n"
                        "- You must click within 1 second after the stimulus\n"
                        "- Don't click for looming sounds without tactile stimuli"
                    )
                    self.status_var.set("Practice failed - try again")
                
                self.results_var.set(results_text)
                
                # Show Try Again button instead of Start
                self.start_button.pack_forget()
                self.try_again_button.pack()
                
                # Enable diagnostic button
                self.show_diagnostic_button.config(state=tk.NORMAL)
            except Exception as e:
                print(f"Error updating UI with results: {e}")
        
        self.root.after(0, update_ui)
    
    def save_log(self):
        """Save the mouse click log to a CSV file."""
        if not self.mouse_clicks:
            return
        try:
            df = pd.DataFrame(self.mouse_clicks)
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

Loaded previously played iterations: {1, 2, 3, 4}
Mouse click binding established
Mouse click detected at screen position (144, 17)
Click ignored for scoring (experiment not running)

===== STARTING PRACTICE TRIAL =====
Selected iteration 5
Verifying mouse tracking...
Clearing timeline markers...
Timeline cleared successfully
Looking for files:
- Looming: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_5_looming.wav
- Tactile: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_5_tactile.wav
- Log: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_5_log.csv
Loaded trial log with 12 trials
    trial_number trial_type  tactile_time_seconds
0              1      catch              2.200000
1              2   baseline              8.856229
2              3   baseline             15.712500
3              4      catch             22.768750
4              5        p

In [5]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        self.click_count = 0
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()

    def update_progress_line(self, elapsed_time):
        """Update the position of the progress line based on elapsed time."""
        try:
            # Calculate position
            timeline_start_x = 400
            timeline_width = self.timeline_end_x - timeline_start_x
            max_duration = 90  # Maximum duration in seconds
            
            x_pos = timeline_start_x + min(elapsed_time / max_duration, 1.0) * timeline_width
            
            # Update line position
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y - 30, 
                x_pos, self.timeline_y + 30
            )
            return True
        except Exception as e:
            print(f"Error updating progress line: {e}")
            return False
    
    def load_played_iterations(self):
        """Load list of previously played iterations."""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations."""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI."""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Get screen dimensions
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        
        # Set window to 90% of screen size to account for taskbar and window decorations
        window_width = int(screen_width * 0.9)
        window_height = int(screen_height * 0.9)
        self.root.geometry(f"{window_width}x{window_height}+{int(screen_width*0.05)}+{int(screen_height*0.05)}")
        
        # Allow window resizing
        self.root.resizable(True, True)
        
        # Create main canvas with scrollbar
        self.main_canvas = tk.Canvas(self.root)
        self.main_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Add vertical scrollbar to canvas
        self.vsb = ttk.Scrollbar(self.root, orient=tk.VERTICAL, command=self.main_canvas.yview)
        self.vsb.pack(side=tk.RIGHT, fill=tk.Y)
        self.main_canvas.configure(yscrollcommand=self.vsb.set)
        
        # Create a frame inside the canvas for all content
        self.main_frame = ttk.Frame(self.main_canvas, padding="10")
        self.main_canvas.create_window((0, 0), window=self.main_frame, anchor=tk.NW, tags="self.main_frame")
        
        # Configure the canvas to resize with the frame
        self.main_frame.bind("<Configure>", lambda e: self.main_canvas.configure(scrollregion=self.main_canvas.bbox("all")))
        
        # Bind mousewheel to scroll
        self.root.bind("<MouseWheel>", lambda e: self.main_canvas.yview_scroll(int(-1*(e.delta/120)), "units"))  # For Windows
        self.root.bind("<Button-4>", lambda e: self.main_canvas.yview_scroll(-1, "units"))  # For Linux
        self.root.bind("<Button-5>", lambda e: self.main_canvas.yview_scroll(1, "units"))   # For Linux
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Bind mouse clicks for the entire application - do this FIRST
        self.root.bind("<Button-1>", self.on_mouse_click)
        print("Mouse click binding established")
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                  font=("Arial", 16, "bold")).pack(pady=5)
        
        # Instructions frame - more compact
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=5)
        instructions_frame.pack(fill=tk.X, padx=5, pady=5)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            "4. You must click within 1 second after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                  font=("Arial", 10), justify="left").pack(anchor=tk.W, pady=2)
        
        # Status frame - more compact
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=5, pady=2)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                      font=("Arial", 11, "bold"))
        self.status_label.pack(pady=2)
        
        # Experimenter visualization frame
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=5)
        visual_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create canvas for visual indicators - reduced height
        self.visual_canvas = tk.Canvas(visual_frame, width=window_width - 100, height=100, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=2)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 45, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 32, text="Tactile Stimulus", font=("Arial", 9))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 55, 350, 80, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 67, text="Mouse Click", font=("Arial", 9))
        
        # Add click counter display
        self.click_counter_text = self.visual_canvas.create_text(
            650, 67,
            text="Total Clicks: 0",
            font=("Arial", 11, "bold"),
            fill="blue"
        )
        
        # Timeline - make it span almost the entire width
        timeline_start_x = 400
        self.timeline_end_x = window_width - 120
        self.timeline_y = 75
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y, self.timeline_end_x, self.timeline_y, width=2)
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y - 20, self.timeline_end_x, self.timeline_y - 20, width=2)
        
        # Add labels to the timeline
        self.visual_canvas.create_text(timeline_start_x - 30, self.timeline_y, text="Timeline:", font=("Arial", 9, "bold"))
        
        # Add time markers every 10 seconds
        for sec in range(0, 91, 10):
            x_pos = timeline_start_x + ((self.timeline_end_x - timeline_start_x) * sec / 90)
            self.visual_canvas.create_line(x_pos, self.timeline_y - 5, x_pos, self.timeline_y + 5, width=1)
            self.visual_canvas.create_text(x_pos, self.timeline_y + 12, text=f"{sec}s", font=("Arial", 7))
        
        # Create progress indicator (vertical line) - represents current playback position
        self.progress_line = self.visual_canvas.create_line(
            timeline_start_x, self.timeline_y - 20, timeline_start_x, self.timeline_y + 20, 
            width=3, fill="green", state="hidden"
        )
            
        # Create a separate frame for the click test area - reduced height
        click_test_frame = ttk.LabelFrame(self.main_frame, text="Click Test Area", padding=5)
        click_test_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create a canvas for the click test area - reduced height
        test_area_height = 120
        self.click_test_canvas = tk.Canvas(click_test_frame, width=window_width - 100, height=test_area_height, bg="lightyellow")
        self.click_test_canvas.pack(fill=tk.X, pady=2)
        
        # Add text to the click test area
        self.click_test_canvas.create_text(
            (window_width - 100)//2, test_area_height//2,
            text="CLICK ANYWHERE IN THIS AREA TO VERIFY MOUSE TRACKING",
            font=("Arial", 14, "bold"), fill="blue"
        )
            
        # Add a visual indicator for clicks in the test area
        self.click_feedback_circle = self.click_test_canvas.create_oval(
            (window_width - 100)//2 - 40, test_area_height//2 - 40,
            (window_width - 100)//2 + 40, test_area_height//2 + 40,
            outline="gray", width=2, fill="white"
        )
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=5)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                       font=("Arial", 10), justify="left")
        self.results_label.pack(anchor=tk.W, pady=2)
        self.results_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 11, "bold"))
        
        # Create a distinctive frame for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=5)
        start_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create a horizontal frame for the buttons
        buttons_horizontal = ttk.Frame(start_frame)
        buttons_horizontal.pack(fill=tk.X, pady=5)
        
        # Start Trial button and Try Again button side by side
        self.start_button = ttk.Button(
            buttons_horizontal, text="START TRIAL",
            command=self.start_trial, width=25, style="Start.TButton"
        )
        self.start_button.pack(side=tk.LEFT, padx=5)
        
        self.try_again_button = ttk.Button(
            buttons_horizontal, text="TRY AGAIN", 
            command=self.start_trial, width=25, style="Start.TButton"
        )
        self.try_again_button.pack(side=tk.RIGHT, padx=5)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(
            self.results_frame, text="Show Detailed Diagnostics",
            command=self.show_diagnostic_report, state=tk.DISABLED
        )
        self.show_diagnostic_button.pack(anchor=tk.E, pady=2)
        
        # Bottom button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=5)
        
        # Continue and Cancel buttons side by side
        self.continue_button = ttk.Button(
            self.button_frame, text="Continue to Experiment",
            state=tk.DISABLED, style="Red.TButton", width=20,
            command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment...")
        )
        self.continue_button.pack(side=tk.LEFT, padx=5)
        
        self.cancel_button = ttk.Button(
            self.button_frame, text="Cancel",
            command=self.on_close, width=15
        )
        self.cancel_button.pack(side=tk.LEFT, padx=5)
        
        # Ensure scrolling works properly by updating the scroll region
        self.main_frame.update_idletasks()
        self.main_canvas.config(scrollregion=self.main_canvas.bbox("all"))
        
        # Scroll to top when starting
        self.main_canvas.yview_moveto(0)

    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window."""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report that fits screen better
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        
        # Set size relative to screen
        window_width = min(800, self.root.winfo_screenwidth() - 100)
        window_height = min(600, self.root.winfo_screenheight() - 100)
        report_window.geometry(f"{window_width}x{window_height}")
        
        # Add scrolled text widget instead of manual scrollbar
        text_widget = scrolledtext.ScrolledText(
            report_window, wrap=tk.WORD, width=80, height=25
        )
        text_widget.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
    
    def on_close(self):
        """Handle window close event."""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment."""
        print(f"Mouse click detected at screen position ({event.x}, {event.y})")
        
        # Show visual feedback on click test canvas if it exists
        try:
            if hasattr(self, 'click_test_canvas') and self.click_test_canvas.winfo_exists():
                # Check if click is within the click test canvas
                test_canvas_x = self.click_test_canvas.winfo_rootx()
                test_canvas_y = self.click_test_canvas.winfo_rooty()
                test_canvas_width = self.click_test_canvas.winfo_width()
                test_canvas_height = self.click_test_canvas.winfo_height()
                
                # If click is within the test canvas
                if (test_canvas_x <= event.x_root <= test_canvas_x + test_canvas_width and
                    test_canvas_y <= event.y_root <= test_canvas_y + test_canvas_height):
                    # Convert to canvas coordinates
                    canvas_x = event.x_root - test_canvas_x
                    canvas_y = event.y_root - test_canvas_y
                    
                    # Move the feedback circle to click position
                    self.click_test_canvas.coords(
                        self.click_feedback_circle,
                        canvas_x - 25, canvas_y - 25,
                        canvas_x + 25, canvas_y + 25
                    )
                    # Change color to indicate click
                    self.click_test_canvas.itemconfig(self.click_feedback_circle, fill="orange", outline="red")
                    # Reset after a short delay
                    self.root.after(300, lambda: self.click_test_canvas.itemconfig(
                        self.click_feedback_circle, fill="white", outline="gray"))
        except Exception as e:
            print(f"Error updating click test feedback: {e}")
        
        # Always show click visually on the main canvas
        try:
            if event.widget == self.visual_canvas:
                # Create a temporary visual flash where the click happened
                flash = self.visual_canvas.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, 
                                                       fill="orange", outline="red", width=2)
                self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
        except Exception as e:
            print(f"Error creating click flash: {e}")
            
        if not self.experiment_running or self.start_time is None:
            print("Click ignored for scoring (experiment not running)")
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat(),
            "x": event.x,
            "y": event.y
        })
        
        # Update click count
        self.click_count += 1
        try:
            self.visual_canvas.itemconfig(
                self.click_counter_text, 
                text=f"Total Clicks: {self.click_count}"
            )
        except Exception as e:
            print(f"Error updating click counter: {e}")
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds (WILL BE SCORED)")
        
        # Visual indicator for the experimenter
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline
        try:
            self.add_timeline_marker(current_time, "red")
            print(f"Added timeline marker at {current_time:.3f}s")
        except Exception as e:
            print(f"Error adding timeline marker: {e}")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter."""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            self.root.update_idletasks()
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            print(f"Error flashing indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time."""
        timeline_start_x = 400
        timeline_width = self.timeline_end_x - timeline_start_x
        
        # Calculate position based on time
        max_duration = 90  # Maximum duration in seconds
        x_pos = timeline_start_x + min(time_sec / max_duration, 1.0) * timeline_width
        
        try:
            # Create marker
            marker = self.visual_canvas.create_oval(
                x_pos-5, self.timeline_y-5, x_pos+5, self.timeline_y+5, 
                fill=color, outline="black", width=1
            )
            self.timeline_markers.append(marker)
            
            # Flash the timeline area briefly
            flash = self.visual_canvas.create_rectangle(
                x_pos-10, self.timeline_y-10, x_pos+10, self.timeline_y+10, 
                outline="red", width=2
            )
            self.root.after(200, lambda f=flash: self.visual_canvas.delete(f))
            
            # Update progress line
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y-20, 
                x_pos, self.timeline_y+20
            )
            
            return True
        except Exception as e:
            print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline."""
        print("Clearing timeline markers...")
        try:
            for marker in self.timeline_markers:
                self.visual_canvas.delete(marker)
            self.timeline_markers = []
            
            # Add a flash effect to show timeline has been cleared
            flash = self.visual_canvas.create_rectangle(395, 15, 745, 95, outline="blue", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            print("Timeline cleared successfully")
        except Exception as e:
            print(f"Error clearing timeline: {e}")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet."""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial."""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Reset click counter for new trial
        self.click_count = 0
        self.visual_canvas.itemconfig(self.click_counter_text, text=f"Total Clicks: {self.click_count}")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Hide progress line initially
        self.visual_canvas.itemconfig(self.progress_line, state="hidden")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Verify mouse tracking before starting trial
        verify_msg = "Verifying mouse tracking..."
        print(verify_msg)
        self.status_var.set(verify_msg)
        self.root.update()
        
        # Flash the click test canvas to draw attention
        test_canvas_width = self.click_test_canvas.winfo_width()
        test_canvas_height = self.click_test_canvas.winfo_height()
        
        # Add a flashing rectangle over the entire test canvas
        self.flash_area_rect = self.click_test_canvas.create_rectangle(
            5, 5, test_canvas_width - 5, test_canvas_height - 5,
            outline="red", width=5
        )
            
        # Add text overlay
        self.flash_area_text = self.click_test_canvas.create_text(
            test_canvas_width//2, test_canvas_height//2,
            text="CLICK ANYWHERE TO VERIFY MOUSE TRACKING!",
            font=("Arial", 14, "bold"), fill="red"
        )
        
        # Wait a moment for user to notice
        self.root.after(2000, self.remove_verification_prompt)
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def remove_verification_prompt(self):
        """Remove the verification prompt and continue with trial."""
        try:
            self.click_test_canvas.delete(self.flash_area_rect)
            self.click_test_canvas.delete(self.flash_area_text)
            print("Verification prompt removed, continuing with trial")
        except Exception as e:
            print(f"Error removing verification prompt: {e}")
    
    def run_trial(self):
        """Run the actual practice trial."""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                try:
                    self.add_timeline_marker(t_time, "blue")
                except Exception as e:
                    print(f"Error adding tactile marker: {e}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz, shape: {looming_data.shape}")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz, shape: {tactile_data.shape}")
            
            # Convert stereo to mono if needed
            if len(looming_data.shape) > 1 and looming_data.shape[1] > 1:
                print("Converting looming audio from stereo to mono for playback")
                looming_data = np.mean(looming_data, axis=1)
            if len(tactile_data.shape) > 1 and tactile_data.shape[1] > 1:
                print("Converting tactile audio from stereo to mono for playback")
                tactile_data = np.mean(tactile_data, axis=1)
            
            def update_status(msg):
                try:
                    self.status_var.set(msg)
                    self.root.update_idletasks()
                except Exception as e:
                    print(f"Error updating status: {e}")
            
            self.root.after(0, update_status, "Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Reset progress line to start position
            self.update_progress_line(0)
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads
            def play_looming():
                try:
                    sd.play(looming_data, looming_sr, blocking=True)
                    print("Looming audio playback completed")
                except Exception as e:
                    print(f"Error playing looming audio: {e}")
            
            def play_tactile():
                try:
                    sd.play(tactile_data, tactile_sr, blocking=True)
                    print("Tactile audio playback completed")
                except Exception as e:
                    print(f"Error playing tactile audio: {e}")
            
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            end_time = self.start_time + max_duration + 1.0
            
            # Start a progress update thread
            progress_update_thread = threading.Thread(
                target=self.continuous_progress_update,
                args=(max_duration,),
                daemon=True
            )
            progress_update_thread.start()
            
            # Update status periodically
            while time.perf_counter() < end_time and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                time.sleep(0.1)
            
            # Wait for threads to complete
            looming_thread.join(timeout=1.0)
            tactile_thread.join(timeout=1.0)
            
            print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            self.root.after(0, update_status, "Practice trial completed - analyzing results...")
            
            # Analyze results
            self.analyze_results(tactile_times)
            
        except Exception as e:
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
            
            def show_error(msg):
                self.status_var.set(msg)
            self.root.after(0, show_error, f"Error during practice trial: {str(e)}")
            
        finally:
            self.experiment_running = False
            
            def enable_buttons():
                self.start_button.config(state=tk.NORMAL)
                self.try_again_button.config(state=tk.NORMAL)
            self.root.after(0, enable_buttons)
            self.save_log()
    
    def continuous_progress_update(self, max_duration):
        """Continuously update the progress line during the experiment."""
        try:
            update_interval = 0.05  # Update every 50ms
            
            while self.experiment_running:
                if self.start_time is not None:
                    elapsed = time.perf_counter() - self.start_time
                    if elapsed > max_duration:
                        break
                    # Schedule an update to the progress line
                    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                    time.sleep(update_interval)
                else:
                    time.sleep(0.1)
            print("Progress update thread completed")
        except Exception as e:
            print(f"Error in progress update thread: {e}")
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback."""
        if not tactile_times.size:
            return
        try:
            while self.experiment_running:
                current_time = time.perf_counter() - self.start_time
                for t_time in tactile_times:
                    if abs(current_time - t_time) < 0.1:  # within 100ms
                        self.root.after(0, lambda: self.flash_indicator(self.tactile_indicator, "yellow"))
                        print(f"Tactile event detected at {current_time:.3f}s (expected {t_time:.3f}s)")
                        break
                time.sleep(0.05)
        except Exception as e:
            print(f"Error in tactile monitor: {e}")
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial."""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            closest_idx = -1
            closest_diff = float('inf')
            
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            if closest_idx >= 0 and closest_diff <= 1.0:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > 1.0s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within 1 second")
                missed_stimuli += 1
                missed_details.append({'tactile_time': tactile_times[i]})
        
        total_tactile = len(tactile_times)
        hit_rate = (correct_clicks / total_tactile) if total_tactile > 0 else 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        diagnostic_report = "===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += (
                    f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile "
                    f"at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                )
        else:
            diagnostic_report += "  None\n"
        
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += (
                        f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for "
                        f"tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                    )
                else:
                    diagnostic_report += (
                        f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
                    )
        else:
            diagnostic_report += "  None\n"
        
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += (
                    f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within 1 second\n"
                )
        else:
            diagnostic_report += "  None\n"
        
        diagnostic_report += "\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        self.diagnostic_report = diagnostic_report
        
        def update_ui():
            try:
                results_text = (
                    f"Practice Results:\n\n"
                    f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Missed stimuli: {missed_stimuli}\n\n"
                )
                
                if self.practice_passed:
                    results_text += "PASSED! You understood the task correctly."
                    self.status_var.set("Practice passed successfully!")
                    self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
                else:
                    results_text += (
                        "FAILED. Please try again and remember:\n"
                        "- Click ONLY when you hear a tactile stimulus\n"
                        "- You must click within 1 second after the stimulus\n"
                        "- Don't click for looming sounds without tactile stimuli"
                    )
                    self.status_var.set("Practice failed - try again")
                
                self.results_var.set(results_text)
                
                # Show Try Again button instead of Start
                self.start_button.pack_forget()
                self.try_again_button.pack()
                
                # Enable diagnostic button
                self.show_diagnostic_button.config(state=tk.NORMAL)
                
                # Ensure buttons are visible by scrolling to bottom if needed
                self.main_canvas.yview_moveto(1.0)
            except Exception as e:
                print(f"Error updating UI with results: {e}")
        
        self.root.after(0, update_ui)
    
    def save_log(self):
        """Save the mouse click log to a CSV file."""
        if not self.mouse_clicks:
            return
        try:
            df = pd.DataFrame(self.mouse_clicks)
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

Loaded previously played iterations: {3}
Mouse click binding established
Mouse click detected at screen position (165, 15)
Click ignored for scoring (experiment not running)

===== STARTING PRACTICE TRIAL =====
Selected iteration 2
Verifying mouse tracking...
Clearing timeline markers...
Timeline cleared successfully
Looking for files:
- Looming: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_2_looming.wav
- Tactile: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_2_tactile.wav
- Log: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_2_log.csv
Loaded trial log with 12 trials
    trial_number trial_type  tactile_time_seconds
0              1      catch               2.10000
1              2        pps              10.72925
2              3      catch              15.91250
3              4      catch              22.36875
4              5   baseline       

In [1]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Response window in seconds (changed from 1.0 to 1.5)
RESPONSE_WINDOW = 1.5

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        self.click_count = 0
        # Flag to control audio playback
        self.stop_audio = False
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()

    def update_progress_line(self, elapsed_time):
        """Update the position of the progress line based on elapsed time."""
        try:
            # Calculate position
            timeline_start_x = 400
            timeline_width = self.timeline_end_x - timeline_start_x
            max_duration = 90  # Maximum duration in seconds
            
            x_pos = timeline_start_x + min(elapsed_time / max_duration, 1.0) * timeline_width
            
            # Update line position
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y - 30, 
                x_pos, self.timeline_y + 30
            )
            return True
        except Exception as e:
            print(f"Error updating progress line: {e}")
            return False
    
    def load_played_iterations(self):
        """Load list of previously played iterations."""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations."""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI."""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Set protocol for window close
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Get screen dimensions
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        
        # Set window to 90% of screen size to account for taskbar and window decorations
        window_width = int(screen_width * 0.9)
        window_height = int(screen_height * 0.9)
        self.root.geometry(f"{window_width}x{window_height}+{int(screen_width*0.05)}+{int(screen_height*0.05)}")
        
        # Allow window resizing
        self.root.resizable(True, True)
        
        # Create main canvas with scrollbar
        self.main_canvas = tk.Canvas(self.root)
        self.main_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Add vertical scrollbar to canvas
        self.vsb = ttk.Scrollbar(self.root, orient=tk.VERTICAL, command=self.main_canvas.yview)
        self.vsb.pack(side=tk.RIGHT, fill=tk.Y)
        self.main_canvas.configure(yscrollcommand=self.vsb.set)
        
        # Create a frame inside the canvas for all content
        self.main_frame = ttk.Frame(self.main_canvas, padding="10")
        self.main_canvas.create_window((0, 0), window=self.main_frame, anchor=tk.NW, tags="self.main_frame")
        
        # Configure the canvas to resize with the frame
        self.main_frame.bind("<Configure>", lambda e: self.main_canvas.configure(scrollregion=self.main_canvas.bbox("all")))
        
        # Bind mousewheel to scroll
        self.root.bind("<MouseWheel>", lambda e: self.main_canvas.yview_scroll(int(-1*(e.delta/120)), "units"))  # For Windows
        self.root.bind("<Button-4>", lambda e: self.main_canvas.yview_scroll(-1, "units"))  # For Linux
        self.root.bind("<Button-5>", lambda e: self.main_canvas.yview_scroll(1, "units"))   # For Linux
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Bind mouse clicks for the entire application - do this FIRST
        self.root.bind("<Button-1>", self.on_mouse_click)
        print("Mouse click binding established")
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                  font=("Arial", 16, "bold")).pack(pady=5)
        
        # Instructions frame - more compact
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=5)
        instructions_frame.pack(fill=tk.X, padx=5, pady=5)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            f"4. You must click within {RESPONSE_WINDOW} seconds after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                  font=("Arial", 10), justify="left").pack(anchor=tk.W, pady=2)
        
        # Status frame - more compact
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=5, pady=2)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                      font=("Arial", 11, "bold"))
        self.status_label.pack(pady=2)
        
        # Experimenter visualization frame
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=5)
        visual_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create canvas for visual indicators - reduced height
        self.visual_canvas = tk.Canvas(visual_frame, width=window_width - 100, height=100, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=2)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 45, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 32, text="Tactile Stimulus", font=("Arial", 9))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 55, 350, 80, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 67, text="Mouse Click", font=("Arial", 9))
        
        # Add click counter display
        self.click_counter_text = self.visual_canvas.create_text(
            650, 67,
            text="Total Clicks: 0",
            font=("Arial", 11, "bold"),
            fill="blue"
        )
        
        # Timeline - make it span almost the entire width
        timeline_start_x = 400
        self.timeline_end_x = window_width - 120
        self.timeline_y = 75
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y, self.timeline_end_x, self.timeline_y, width=2)
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y - 20, self.timeline_end_x, self.timeline_y - 20, width=2)
        
        # Add labels to the timeline
        self.visual_canvas.create_text(timeline_start_x - 30, self.timeline_y, text="Timeline:", font=("Arial", 9, "bold"))
        
        # Add time markers every 10 seconds
        for sec in range(0, 91, 10):
            x_pos = timeline_start_x + ((self.timeline_end_x - timeline_start_x) * sec / 90)
            self.visual_canvas.create_line(x_pos, self.timeline_y - 5, x_pos, self.timeline_y + 5, width=1)
            self.visual_canvas.create_text(x_pos, self.timeline_y + 12, text=f"{sec}s", font=("Arial", 7))
        
        # Create progress indicator (vertical line) - represents current playback position
        self.progress_line = self.visual_canvas.create_line(
            timeline_start_x, self.timeline_y - 20, timeline_start_x, self.timeline_y + 20, 
            width=3, fill="green", state="hidden"
        )
            
        # Create a separate frame for the click test area - reduced height
        click_test_frame = ttk.LabelFrame(self.main_frame, text="Click Test Area", padding=5)
        click_test_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create a canvas for the click test area - reduced height
        test_area_height = 120
        self.click_test_canvas = tk.Canvas(click_test_frame, width=window_width - 100, height=test_area_height, bg="lightyellow")
        self.click_test_canvas.pack(fill=tk.X, pady=2)
        
        # Add text to the click test area
        self.click_test_canvas.create_text(
            (window_width - 100)//2, test_area_height//2,
            text="CLICK ANYWHERE IN THIS AREA TO VERIFY MOUSE TRACKING",
            font=("Arial", 14, "bold"), fill="blue"
        )
            
        # Add a visual indicator for clicks in the test area
        self.click_feedback_circle = self.click_test_canvas.create_oval(
            (window_width - 100)//2 - 40, test_area_height//2 - 40,
            (window_width - 100)//2 + 40, test_area_height//2 + 40,
            outline="gray", width=2, fill="white"
        )
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=5)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                       font=("Arial", 10), justify="left")
        self.results_label.pack(anchor=tk.W, pady=2)
        self.results_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 11, "bold"))
        
        # Create a distinctive frame for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=5)
        start_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create a horizontal frame for the buttons
        buttons_horizontal = ttk.Frame(start_frame)
        buttons_horizontal.pack(fill=tk.X, pady=5)
        
        # Start Trial button and Try Again button side by side
        self.start_button = ttk.Button(
            buttons_horizontal, text="START TRIAL",
            command=self.start_trial, width=25, style="Start.TButton"
        )
        self.start_button.pack(side=tk.LEFT, padx=5)
        
        self.try_again_button = ttk.Button(
            buttons_horizontal, text="TRY AGAIN", 
            command=self.start_trial, width=25, style="Start.TButton"
        )
        self.try_again_button.pack(side=tk.RIGHT, padx=5)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(
            self.results_frame, text="Show Detailed Diagnostics",
            command=self.show_diagnostic_report, state=tk.DISABLED
        )
        self.show_diagnostic_button.pack(anchor=tk.E, pady=2)
        
        # Bottom button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=5)
        
        # Continue and Cancel buttons side by side
        self.continue_button = ttk.Button(
            self.button_frame, text="Continue to Experiment",
            state=tk.DISABLED, style="Red.TButton", width=20,
            command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment...")
        )
        self.continue_button.pack(side=tk.LEFT, padx=5)
        
        self.cancel_button = ttk.Button(
            self.button_frame, text="Cancel",
            command=self.on_close, width=15
        )
        self.cancel_button.pack(side=tk.LEFT, padx=5)
        
        # Ensure scrolling works properly by updating the scroll region
        self.main_frame.update_idletasks()
        self.main_canvas.config(scrollregion=self.main_canvas.bbox("all"))
        
        # Scroll to top when starting
        self.main_canvas.yview_moveto(0)

    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window."""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report that fits screen better
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        
        # Set size relative to screen
        window_width = min(800, self.root.winfo_screenwidth() - 100)
        window_height = min(600, self.root.winfo_screenheight() - 100)
        report_window.geometry(f"{window_width}x{window_height}")
        
        # Add scrolled text widget instead of manual scrollbar
        text_widget = scrolledtext.ScrolledText(
            report_window, wrap=tk.WORD, width=80, height=25
        )
        text_widget.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
    
    def on_close(self):
        """Handle window close event."""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                # Stop audio playback when closing the window
                self.stop_audio = True
                sd.stop()
                print("Audio playback stopped")
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment."""
        print(f"Mouse click detected at screen position ({event.x}, {event.y})")
        
        # Show visual feedback on click test canvas if it exists
        try:
            if hasattr(self, 'click_test_canvas') and self.click_test_canvas.winfo_exists():
                # Check if click is within the click test canvas
                test_canvas_x = self.click_test_canvas.winfo_rootx()
                test_canvas_y = self.click_test_canvas.winfo_rooty()
                test_canvas_width = self.click_test_canvas.winfo_width()
                test_canvas_height = self.click_test_canvas.winfo_height()
                
                # If click is within the test canvas
                if (test_canvas_x <= event.x_root <= test_canvas_x + test_canvas_width and
                    test_canvas_y <= event.y_root <= test_canvas_y + test_canvas_height):
                    # Convert to canvas coordinates
                    canvas_x = event.x_root - test_canvas_x
                    canvas_y = event.y_root - test_canvas_y
                    
                    # Move the feedback circle to click position
                    self.click_test_canvas.coords(
                        self.click_feedback_circle,
                        canvas_x - 25, canvas_y - 25,
                        canvas_x + 25, canvas_y + 25
                    )
                    # Change color to indicate click
                    self.click_test_canvas.itemconfig(self.click_feedback_circle, fill="orange", outline="red")
                    # Reset after a short delay
                    self.root.after(300, lambda: self.click_test_canvas.itemconfig(
                        self.click_feedback_circle, fill="white", outline="gray"))
        except Exception as e:
            print(f"Error updating click test feedback: {e}")
        
        # Always show click visually on the main canvas
        try:
            if event.widget == self.visual_canvas:
                # Create a temporary visual flash where the click happened
                flash = self.visual_canvas.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, 
                                                       fill="orange", outline="red", width=2)
                self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
        except Exception as e:
            print(f"Error creating click flash: {e}")
            
        if not self.experiment_running or self.start_time is None:
            print("Click ignored for scoring (experiment not running)")
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat(),
            "x": event.x,
            "y": event.y
        })
        
        # Update click count
        self.click_count += 1
        try:
            self.visual_canvas.itemconfig(
                self.click_counter_text, 
                text=f"Total Clicks: {self.click_count}"
            )
        except Exception as e:
            print(f"Error updating click counter: {e}")
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds (WILL BE SCORED)")
        
        # Visual indicator for the experimenter
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline
        try:
            self.add_timeline_marker(current_time, "red")
            print(f"Added timeline marker at {current_time:.3f}s")
        except Exception as e:
            print(f"Error adding timeline marker: {e}")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter."""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            self.root.update_idletasks()
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            print(f"Error flashing indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time."""
        timeline_start_x = 400
        timeline_width = self.timeline_end_x - timeline_start_x
        
        # Calculate position based on time
        max_duration = 90  # Maximum duration in seconds
        x_pos = timeline_start_x + min(time_sec / max_duration, 1.0) * timeline_width
        
        try:
            # Create marker
            marker = self.visual_canvas.create_oval(
                x_pos-5, self.timeline_y-5, x_pos+5, self.timeline_y+5, 
                fill=color, outline="black", width=1
            )
            self.timeline_markers.append(marker)
            
            # Flash the timeline area briefly
            flash = self.visual_canvas.create_rectangle(
                x_pos-10, self.timeline_y-10, x_pos+10, self.timeline_y+10, 
                outline="red", width=2
            )
            self.root.after(200, lambda f=flash: self.visual_canvas.delete(f))
            
            # Update progress line
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y-20, 
                x_pos, self.timeline_y+20
            )
            
            return True
        except Exception as e:
            print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline."""
        print("Clearing timeline markers...")
        try:
            for marker in self.timeline_markers:
                self.visual_canvas.delete(marker)
            self.timeline_markers = []
            
            # Add a flash effect to show timeline has been cleared
            flash = self.visual_canvas.create_rectangle(395, 15, 745, 95, outline="blue", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            print("Timeline cleared successfully")
        except Exception as e:
            print(f"Error clearing timeline: {e}")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet."""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial."""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Reset stop audio flag
        self.stop_audio = False
        
        # Reset click counter for new trial
        self.click_count = 0
        self.visual_canvas.itemconfig(self.click_counter_text, text=f"Total Clicks: {self.click_count}")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Hide progress line initially
        self.visual_canvas.itemconfig(self.progress_line, state="hidden")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Verify mouse tracking before starting trial
        verify_msg = "Verifying mouse tracking..."
        print(verify_msg)
        self.status_var.set(verify_msg)
        self.root.update()
        
        # Flash the click test canvas to draw attention
        test_canvas_width = self.click_test_canvas.winfo_width()
        test_canvas_height = self.click_test_canvas.winfo_height()
        
        # Add a flashing rectangle over the entire test canvas
        self.flash_area_rect = self.click_test_canvas.create_rectangle(
            5, 5, test_canvas_width - 5, test_canvas_height - 5,
            outline="red", width=5
        )
            
        # Add text overlay
        self.flash_area_text = self.click_test_canvas.create_text(
            test_canvas_width//2, test_canvas_height//2,
            text="CLICK ANYWHERE TO VERIFY MOUSE TRACKING!",
            font=("Arial", 14, "bold"), fill="red"
        )
        
        # Wait a moment for user to notice
        self.root.after(2000, self.remove_verification_prompt)
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def remove_verification_prompt(self):
        """Remove the verification prompt and continue with trial."""
        try:
            self.click_test_canvas.delete(self.flash_area_rect)
            self.click_test_canvas.delete(self.flash_area_text)
            print("Verification prompt removed, continuing with trial")
        except Exception as e:
            print(f"Error removing verification prompt: {e}")
    
    def run_trial(self):
        """Run the actual practice trial."""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                try:
                    self.add_timeline_marker(t_time, "blue")
                except Exception as e:
                    print(f"Error adding tactile marker: {e}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz, shape: {looming_data.shape}")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz, shape: {tactile_data.shape}")
            
            # Convert stereo to mono if needed
            if len(looming_data.shape) > 1 and looming_data.shape[1] > 1:
                print("Converting looming audio from stereo to mono for playback")
                looming_data = np.mean(looming_data, axis=1)
            if len(tactile_data.shape) > 1 and tactile_data.shape[1] > 1:
                print("Converting tactile audio from stereo to mono for playback")
                tactile_data = np.mean(tactile_data, axis=1)
            
            def update_status(msg):
                try:
                    self.status_var.set(msg)
                    self.root.update_idletasks()
                except Exception as e:
                    print(f"Error updating status: {e}")
            
            self.root.after(0, update_status, "Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Reset progress line to start position
            self.update_progress_line(0)
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads with checks for the stop flag
            def play_looming():
                try:
                    with sd.OutputStream(samplerate=looming_sr, channels=1 if len(looming_data.shape) == 1 else looming_data.shape[1]) as stream:
                        chunk_size = 1024
                        for i in range(0, len(looming_data), chunk_size):
                            if self.stop_audio:
                                print("Looming audio playback stopped")
                                break
                            chunk = looming_data[i:min(i+chunk_size, len(looming_data))]
                            stream.write(chunk.astype(np.float32))
                    print("Looming audio playback completed")
                except Exception as e:
                    print(f"Error playing looming audio: {e}")
            
            def play_tactile():
                try:
                    with sd.OutputStream(samplerate=tactile_sr, channels=1 if len(tactile_data.shape) == 1 else tactile_data.shape[1]) as stream:
                        chunk_size = 1024
                        for i in range(0, len(tactile_data), chunk_size):
                            if self.stop_audio:
                                print("Tactile audio playback stopped")
                                break
                            chunk = tactile_data[i:min(i+chunk_size, len(tactile_data))]
                            stream.write(chunk.astype(np.float32))
                    print("Tactile audio playback completed")
                except Exception as e:
                    print(f"Error playing tactile audio: {e}")
            
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            end_time = self.start_time + max_duration + 1.0
            
            # Start a progress update thread
            progress_update_thread = threading.Thread(
                target=self.continuous_progress_update,
                args=(max_duration,),
                daemon=True
            )
            progress_update_thread.start()
            
            # Update status periodically
            while time.perf_counter() < end_time and not self.stop_audio and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                time.sleep(0.1)
            
            if not self.stop_audio:
                # Wait for threads to complete
                looming_thread.join(timeout=1.0)
                tactile_thread.join(timeout=1.0)
                
                print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
                self.root.after(0, update_status, "Practice trial completed - analyzing results...")
                
                # Analyze results
                self.analyze_results(tactile_times)
            else:
                print("Audio playback was stopped by user")
                self.root.after(0, update_status, "Practice trial stopped")
            
        except Exception as e:
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
            
            def show_error(msg):
                self.status_var.set(msg)
            self.root.after(0, show_error, f"Error during practice trial: {str(e)}")
            
        finally:
            self.experiment_running = False
            
            def enable_buttons():
                self.start_button.config(state=tk.NORMAL)
                self.try_again_button.config(state=tk.NORMAL)
            self.root.after(0, enable_buttons)
            self.save_log()
    
    def continuous_progress_update(self, max_duration):
        """Continuously update the progress line during the experiment."""
        try:
            update_interval = 0.05  # Update every 50ms
            
            while self.experiment_running and not self.stop_audio:
                if self.start_time is not None:
                    elapsed = time.perf_counter() - self.start_time
                    if elapsed > max_duration:
                        break
                    # Schedule an update to the progress line
                    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                    time.sleep(update_interval)
                else:
                    time.sleep(0.1)
            print("Progress update thread completed")
        except Exception as e:
            print(f"Error in progress update thread: {e}")
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback."""
        if not tactile_times.size:
            return
        try:
            while self.experiment_running and not self.stop_audio:
                current_time = time.perf_counter() - self.start_time
                for t_time in tactile_times:
                    if abs(current_time - t_time) < 0.1:  # within 100ms
                        self.root.after(0, lambda: self.flash_indicator(self.tactile_indicator, "yellow"))
                        print(f"Tactile event detected at {current_time:.3f}s (expected {t_time:.3f}s)")
                        break
                time.sleep(0.05)
        except Exception as e:
            print(f"Error in tactile monitor: {e}")
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial."""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        # Initialize counters
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Initialize result details
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        # Mark each stimulus as not responded yet
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            
            # Initialize variables for finding the closest tactile stimulus
            closest_idx = -1
            closest_diff = float('inf')
            
            # Find the closest preceding tactile stimulus
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                # Only consider tactile stimuli that preceded this click and are closest
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Determine if the click is a correct response or false alarm
            # Changed from 1.0 to 1.5 seconds window
            if closest_idx >= 0 and closest_diff <= RESPONSE_WINDOW:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > {RESPONSE_WINDOW}s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli (tactile events without click responses)
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within {RESPONSE_WINDOW} seconds")
                missed_stimuli += 1
                missed_details.append({'tactile_time': tactile_times[i]})
        
        # Calculate hit rate
        total_tactile = len(tactile_times)
        hit_rate = (correct_clicks / total_tactile) if total_tactile > 0 else 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        # Determine if practice passed (80% correct and <= 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        # Create detailed diagnostic report
        diagnostic_report = f"===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        diagnostic_report += f"RESPONSE WINDOW: {RESPONSE_WINDOW} seconds\n\n"
        
        # Add correct responses section
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += (
                    f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile "
                    f"at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                )
        else:
            diagnostic_report += "  None\n"
        
        # Add false alarms section
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += (
                        f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for "
                        f"tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                    )
                else:
                    diagnostic_report += (
                        f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
                    )
        else:
            diagnostic_report += "  None\n"
        
        # Add missed stimuli section
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += (
                    f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within {RESPONSE_WINDOW} seconds\n"
                )
        else:
            diagnostic_report += "  None\n"
        
        # Add conclusion
        diagnostic_report += "\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        self.diagnostic_report = diagnostic_report
        
        # Enhanced display of results in the GUI
        def update_ui():
            try:
                detailed_results = (
                    f"Practice Results:\n\n"
                    f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Missed stimuli: {missed_stimuli}\n\n"
                )
                
                # Add specific details about misses
                if missed_details:
                    detailed_results += "MISSED STIMULI:\n"
                    for i, detail in enumerate(missed_details[:3]):  # Show first 3 for brevity
                        detailed_results += f"- Tactile at {detail['tactile_time']:.1f}s had no response\n"
                    if len(missed_details) > 3:
                        detailed_results += f"  (and {len(missed_details) - 3} more...)\n"
                    detailed_results += "\n"
                
                # Add specific details about false alarms
                if false_alarm_details:
                    detailed_results += "FALSE ALARMS:\n"
                    for i, detail in enumerate(false_alarm_details[:3]):  # Show first 3 for brevity
                        if detail['closest_tactile'] is not None:
                            detailed_results += f"- Click at {detail['click_time']:.1f}s (too late, {detail['difference']:.1f}s after tactile)\n"
                        else:
                            detailed_results += f"- Click at {detail['click_time']:.1f}s (no corresponding tactile)\n"
                    if len(false_alarm_details) > 3:
                        detailed_results += f"  (and {len(false_alarm_details) - 3} more...)\n"
                    detailed_results += "\n"
                
                if self.practice_passed:
                    detailed_results += "PASSED! You understood the task correctly."
                    self.status_var.set("Practice passed successfully!")
                    self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
                else:
                    detailed_results += (
                        "FAILED. Please try again and remember:\n"
                        f"- Click ONLY when you hear a tactile stimulus\n"
                        f"- You must click within {RESPONSE_WINDOW} seconds after the stimulus\n"
                        f"- Don't click for looming sounds without tactile stimuli"
                    )
                    self.status_var.set("Practice failed - try again")
                
                self.results_var.set(detailed_results)
                
                # Show Try Again button instead of Start
                self.start_button.pack_forget()
                self.try_again_button.pack(side=tk.LEFT, padx=5)
                
                # Enable diagnostic button
                self.show_diagnostic_button.config(state=tk.NORMAL)
                
                # Ensure buttons are visible by scrolling to bottom if needed
                self.main_canvas.yview_moveto(1.0)
            except Exception as e:
                print(f"Error updating UI with results: {e}")
        
        self.root.after(0, update_ui)
    
    def save_log(self):
        """Save the mouse click log to a CSV file."""
        if not self.mouse_clicks:
            return
        try:
            df = pd.DataFrame(self.mouse_clicks)
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()

Loaded previously played iterations: {1, 2, 3}
Mouse click binding established
Mouse click detected at screen position (1318, 366)
Click ignored for scoring (experiment not running)
Mouse click detected at screen position (67, 2)
Click ignored for scoring (experiment not running)

===== STARTING PRACTICE TRIAL =====
Selected iteration 4
Verifying mouse tracking...
Clearing timeline markers...
Timeline cleared successfully
Looking for files:
- Looming: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_4_looming.wav
- Tactile: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_4_tactile.wav
- Log: C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot\PracticeStimuli\practice_iteration_4_log.csv
Loaded trial log with 12 trials
    trial_number trial_type  tactile_time_seconds
0              1      catch              2.100000
1              2   baseline              8.856229
2              3   baseline